In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 

from collections import Counter
from numpy import nan as NA
from dateutil.parser import parse
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="huangxq",
          password="f_epz6hDmZ0Hzi0OzI0N",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 1.样本数据整理

## 1_11.获取_租后数据

In [3]:
sql = '''-- 租后应收监控  
SELECT  tprm.* 
,om.`status` as 订单状态值
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
,om.order_finish_date as 订单完成时间,om.order_method
,tmu.true_name ,tmu.id_card_num,tmu.mobile 
,tod.product_name,tod.sku_attributes, cc.channel_type_id
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id
left join db_digua_business.t_channel cc on cc.scene = om.channel
;
'''

df_zhys = query(sql)
# (27787, 62)
df_zhys.shape

(35310, 64)

## 1_12. 剔除商家数据

In [4]:
# df_zhys["年"] = df_zhys["下单月份"].str[:4]
# pd.crosstab(dfzh1["商家名称"],dfzh1["年"],margins=True).loc[["北京海鸟窝科技有限公司","深圳优优大数据科技有限公司","租着用电脑数码","苏州蚁诺宝"],:]
# 剔除商家数据
def drop_sj(df_zhys,col1,col2):
    df_zhys.drop(df_zhys[df_zhys[col1]=="深圳优优大数据科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="优优2店"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="小豚租（代收）"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="苏州蚁诺宝"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="租着用电脑数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="北京海鸟窝科技有限公司"].index,inplace=True)

    df_zhys.drop(df_zhys[df_zhys[col1]=="汇客好租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="澄心优租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="CPS渠道合作"].index,inplace=True)
    
    df_zhys.drop(df_zhys[df_zhys[col1]=="趣智数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="格木木二奢名品"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="广州康基贸易有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="线下小店"].index,inplace=True)  # 计划
    df_zhys.drop(df_zhys[df_zhys[col1]=="小蚂蚁租机"].index,inplace=True)  #拒量订单
   

    df_zhys.drop(df_zhys[df_zhys[col2].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys['merchant_name'] == "乙辉数码"].index, inplace=True)
    return df_zhys

df_zhys = drop_sj(df_zhys,'merchant_name','model_number')
df_zhys.shape

(33134, 64)

## 1_13.历史订单渠道名称补充

In [5]:
# 1148 条历史订单渠道名称补充
qudao_name_df = pd.read_excel('E:/myfile/租后数据/历史订单渠道名称补充.xlsx')
qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)

# dfzh1 = df_zh.merge(qudao_name_df,on = '订单号',how = 'left')
dfzh1 =  df_zhys.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
# dfzh1.columns

# qudao_name_df.shape
# qudao_name_df[:2]
# qudao_name_df['渠道名称'].value_counts()

# 渠道名称缺失补充
# dfzh1["渠道名称"] = dfzh1["渠道名称_x"].fillna(dfzh1.渠道名称_y)
dfzh1["channel_name"] = np.where(dfzh1.渠道名称.notnull(),dfzh1.渠道名称,dfzh1["channel_name"])
dfzh1["channel_name"].isnull().sum()

# dfzh1[['order_number','channel_name','渠道名称']][:10]
# df_zhys[df_zhys['order_number'] == 'A202207131230485'][['order_number','channel_name']]

0

## 1_14.渠道归属

In [6]:
def qudao_type(a,b,c,d):
    a = str(a)
    b = str(b)
    if d in [2, 3, 80] or c == 1:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif d in [48, 81]:
        return "搜索渠道"
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "支付宝直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    elif "付费创本" in a:
        return "付费创本-搜索"
    else:
        return a

In [7]:
# df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
dfzh1["activity_source"] = dfzh1["activity_source"].fillna('未知活动') 
dfzh1["channel_name"] = dfzh1["channel_name"].fillna('未知渠道') 

dfzh1.loc[:,"归属渠道"]=dfzh1.apply(lambda x:qudao_type(x["channel_name"],x["activity_source"],x["order_method"], x['channel_type_id']),axis=1)

# pd.crosstab(dfzh1["下单月份"],dfzh1["归属渠道"],margins=True)

## 1_15.判断颜色 内存

In [8]:
dfzh1["内存"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[0].str.split(" ").str[0]
# dfzh1["内存"].value_counts()[14:]

dfzh1["颜色"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[1].str.split(" ").str[0]
# dfzh1["颜色"].value_counts()[1:10]

## 1_16.判断二手、带锁

In [9]:
# dfzh1['是否二手'] = np.where(dfzh1['model_number'].str.contains('95新')|dfzh1['model_number'].str.contains('99新')|dfzh1['model_number'].str.contains('准新'),"二手","全新")
# note 20241124 修改口径，用product_name判断是否二手
dfzh1['是否二手'] = np.where(dfzh1['product_name'].str.contains('95新')|dfzh1['product_name'].str.contains('99新')|dfzh1['product_name'].str.contains('准新'),"二手","全新")
dfzh1['是否二手'].value_counts()

是否二手
全新    32259
二手      875
Name: count, dtype: int64

In [10]:
#读取导入最新出库台账数据,2023.9及以后的数据
# （1）512 带锁
# （2）非512的多头带锁
# （3）抖音带锁
# note 20241204
# （1）512 带锁，但每天豁免一台 512P 不带锁，目前还没量
# （2）非512的命中多头提示带锁，但出库是否带锁由人工控制
# （3）抖音带锁（苹果11-15）
# 维客壹佰2023&2024年台账1204.xlsz 为截止到20241203的数据
# f_path_ck = "维客壹佰2023&2024年台账1204.xlsx"
f_path_ck = "维客壹佰2023&2024年台账.xlsx"
path = 'F:/myfile/p站数据/台账数据/'
dfck = pd.read_excel(path + f_path_ck,sheet_name="2023")   #,skiprows=0 header 

# dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存']=='512G')|(dfck['内存']=='512GB'), 1, 0)
dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存'].str.contains(r'512')), '带锁', '未带锁')
dfck = dfck[[u'订单号',u'是否带锁']]
dfck.shape
dfck['是否带锁'].value_counts()

dfck.drop_duplicates(subset=["订单号"],inplace=True)

# 合并到主表中去
# dfck 的带锁不一定出库，dfzh1 是出库的
dfck = dfck.rename(columns= {'订单号': 'order_number'})
dfzh1 = dfzh1.merge(dfck, on = 'order_number', how = 'left')
dfzh1['是否带锁'] = dfzh1['是否带锁'].fillna('未带锁') 
dfzh1.shape

pd.pivot_table(dfzh1,index='是否二手',columns='是否带锁',values='order_id',aggfunc='count')

(27824, 2)

是否带锁
未带锁    25728
带锁      2096
Name: count, dtype: int64

(33134, 71)

是否带锁,带锁,未带锁
是否二手,,
二手,174,701
全新,1603,30656


## 1_17.日期处理

In [11]:
dfzh1["下单日期"] = dfzh1['order_create_time'].dt.date
dfzh1["下单日期"] = pd.to_datetime(dfzh1["下单日期"],errors="coerce")
dfzh1["下单日期"].head()

dfzh1["月份"]=dfzh1["下单日期"].dt.month
dfzh1["年份"]=dfzh1["下单日期"].dt.year
# df_total["下单月份"].value_counts()

0   2022-07-05
1   2022-07-13
2   2022-07-15
3   2022-07-16
4   2022-07-17
Name: 下单日期, dtype: datetime64[ns]

In [12]:
def getjidu(s):
    if s<4:
        return "1季度"
    elif 4<=s<7:
        return "2季度"
    elif 7<=s<10:
        return "3季度"
    else:
        return "4季度"
dfzh1["季度"]= dfzh1.apply(lambda x:getjidu(x["月份"]),axis=1)  
dfzh1["季度"].value_counts()

季度
4季度    11213
3季度     9679
2季度     6164
1季度     6078
Name: count, dtype: int64

## 1_21.获取_买断数据

In [13]:
sql1 = ''' --   买断信息 
with out_order as ( 
SELECT too.order_id,too.`status` 
,too.real_pay_money ,too.create_time,too.update_time
,too.pay_date ,too.actual_money 
,rank() over(partition by too.order_id order by too.pay_date desc) as rn 
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断 
-- and too.pay_status= 2    -- '支付状态 1、待支付 2、已支付 3、退款中 4、已退款 6、支付取消 7、已关闭',
-- GROUP BY 1 ,2
-- ORDER BY 5 desc,1
)
# note 20241126 增加distinct，买断可能分多次付款
# 181950     2
# 338547     2
# 1083822    2
# 303756     2
SELECT distinct t1.*
,t2.`status`
,case when t2.`status`=2 then '已买断' when  t2.`status`=3 and om.order_finish_date is not null then '已买断'  else '部分买断' end as if_outpay 
,om.order_finish_date
,case when t2.`status`=2 and om.order_finish_date is null then t1.pay_date else om.order_finish_date end as finish_date_new 
from (
SELECT too.order_id  -- ,too.`status` 
-- ,count(DISTINCT too.`status`) nn 
,sum(too.real_pay_money) as outpay_money     -- ,max(too.update_time) as outpay_time
,max(too.pay_date) as pay_date 
from out_order  too
GROUP BY 1 -- ,2
ORDER BY 2 desc ,1) t1 
left join out_order t2 on t1.order_id = t2.order_id and t2.rn=1 
left join db_digua_business.t_order om on t1.order_id = om.id 
;
'''

df_out = query(sql1)
df_out.shape

(3263, 7)

## 1_22.租后数据 关联 买断数据

In [14]:
dfzh2 = dfzh1.merge(df_out[['order_id', 'outpay_money', 'pay_date', 'status', 'if_outpay']], on = 'order_id', how = 'left')
dfzh2.shape
dfzh1.shape

(33134, 79)

(33134, 75)

In [15]:
dfzh2['finish_date_new'] = dfzh2.apply(lambda x:pd.to_datetime(x.pay_date, errors='coerce') if pd.isnull(x.order_finish_date) and x.status==2
                                       else pd.to_datetime(x.订单完成时间, errors='coerce') if pd.isnull(x.order_finish_date)  and pd.notnull(x.订单完成时间)
                                       else pd.to_datetime(x.order_finish_date, errors='coerce'),axis = 1 ) 
# dfzh2['finish_date_new'] = np.where((pd.isnull(dfzh2.order_finish_date)) & (dfzh2.status==2),pd.to_datetime(dfzh2.pay_date, errors='coerce'),pd.to_datetime(dfzh2.order_finish_date, errors='coerce'))
dfzh2['finish_date_new'].notnull().sum()

4222

In [16]:
dfzh2['if_outpay'] = dfzh2.apply(lambda x: '已买断' if x['status'] ==2 
                                 else  '已买断'  if  x['status'] ==3 and pd.notnull(x.finish_date_new) 
                                 else '部分买断' if x['status'] ==3 and pd.isnull(x.finish_date_new) 
                                 else '未知',axis =1)
dfzh2['if_outpay'].value_counts()

if_outpay
未知      29987
已买断      3078
部分买断       69
Name: count, dtype: int64

## 1_31.获取_分期取消数据

In [17]:
sql1 = '''-- 订单分期表  取消的分期
SELECT tos.order_id,count(*) as nn
,count(tos.reality_refund_date) as  reality_refund_date
,sum(tos.real_pay_money) as real_pay_money
,sum(tprm.purchase_amount)/count(tos.order_id) as purchase_amount
,avg(tprm.advance_periods) as  advance_periods
,avg(tprm.advance_sum) as advance_sum
from db_digua_business.t_order_stages tos 
inner join db_digua_business.t_postlease_receivables_monitoring   tprm 
on tos.order_id = tprm.order_id 
# 订单取消
where tos.`status`=4 
GROUP BY 1 order by 2 desc 
;
'''

df_cancel = query(sql1)
df_cancel.shape

(1911, 7)

## 1_32.租后数据关联分期取消数据

In [18]:
dfzh3 = dfzh2.merge(df_cancel[['order_id', 'nn', 'reality_refund_date', 'real_pay_money']],on = 'order_id',how = 'left')
dfzh3.shape
dfzh2.shape

# 不同还款状态下  完成时间的情况
# dfzh3.groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

# 不同订单状态下  完成时间的情况
# dfzh3.groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})

# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})

(33134, 83)

(33134, 80)

## 1_41.按风控策略调整时间节点划分时间段

In [19]:
#按风控策略调整时间节点划分时间段
dfzh1.loc[:"2023-01-09","时间段"]=" -2023.1.9前"
dfzh1.loc["2023-01-10":"2023-03-03","时间段"]="2023.1.10-2023.3.3"
dfzh1.loc["2023-03-04":"2023-03-24","时间段"]="2023.3.4-2023.3.24"
dfzh1.loc["2023-03-25":"2023-04-28","时间段"]="2023.3.25-2023.4.28"
dfzh1.loc["2023-04-29":"2023-05-17","时间段"]="2023.4.29-2023.5.17"
dfzh1.loc["2023-05-18":"2023-05-31","时间段"]="2023.5.18-2023.5.31"
dfzh1.loc["2023-06-01":"2023-06-30","时间段"]="2023.6.1-2023.6.30"
dfzh1.loc["2023-07-01":"2023-08-11","时间段"]="2023.7.1-2023.8.11"
dfzh1.loc["2023-08-12":"2023-09-15","时间段"]="2023.8.12-2023.9.15"
dfzh1.loc["2023-09-16":"2023-09-22","时间段"]="2023.9.16-2023.9.22"
dfzh1.loc["2023-09-23":"2023-11-16","时间段"]="2023.9.23-2023.11.16"
dfzh1.loc["2023-11-17":"2023-11-28","时间段"]="2023.11.17-2023.11.28"
dfzh1.loc["2023-11-29":"2023-12-05","时间段"]="2023.11.29-2023.12.5"
dfzh1.loc["2023-12-06":"2024-01-05","时间段"]="2023.12.06-2024.1.5"
dfzh1.loc["2024-01-06":"2024-01-23","时间段"]="2024.1.6-2024.1.23"
dfzh1.loc["2024-01-24":"2024-2-21","时间段"]="2024.1.24-2024.2.21"
dfzh1.loc["2024-02-22":"2024-02-29","时间段"]="2024.2.22-2024.2.29"
dfzh1.loc["2024-03-01":"2024-03-12","时间段"]="2024.3.1-2024.3.12"
dfzh1.loc["2024-03-13":"2024-04-03","时间段"]="2024.3.13-2024.4.3"
dfzh1.loc["2024-04-04":"2024-04-10","时间段"]="2024.4.4-2024.4.10"
dfzh1.loc["2024-04-11":"2024-04-18","时间段"]="2024.4.11-2024.4.18"
dfzh1.loc["2024-04-19":"2024-04-24","时间段"]="2024.4.19-2024.4.24"
dfzh1.loc["2024-04-25":"2024-04-27","时间段"]="2024.4.25-2024.4.27"
dfzh1.loc["2024-04-28":,"时间段"]="2024.4.28- "

## 1_51.获取_账期数据（含续租）

In [20]:
sql_xz = ''' 
-- 状态,1:未支付  2:扣款失败 3:已支付 4：已取消 5、申请扣款、6已退款',
-- '是否续租账期,0非续租账期，1预授权账期，2自定义续租账期',
SELECT ymos.*
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
;
'''

df_xzfq = query(sql_xz)
df_xzfq.shape

(439193, 23)

In [21]:
# note 20241227 号卡 应付日期更新
# 分期表
sql = '''-- 租后应收监控  
SELECT distinct hk.order_number, tos.order_id, tos.refund_date as refund_date_1
from db_digua_business.t_order_stages tos
inner join
    (select distinct om.id as order_id, om.order_number
    from db_digua_business.t_order om
    left join db_digua_business.t_platform_activity tpa 
    on tpa.id=om.activity_id
    where tpa.type = 4) hk
on tos.order_id = hk.order_id
'''
tmp_hk = query(sql)
tmp_hk['refund_date_ym'] = tmp_hk['refund_date_1'].dt.strftime('%Y-%m')
tmp_hk.shape
tmp_hk[:2]

df_xzfq['refund_date_ym'] = df_xzfq['refund_date'].dt.strftime('%Y-%m')
df_xzfq = df_xzfq.merge(tmp_hk[['order_id', 'refund_date_1', 'refund_date_ym']], on = ['order_id', 'refund_date_ym'], how = 'left')
df_xzfq['refund_date_2'] = np.where(df_xzfq.refund_date_1.notnull(), df_xzfq.refund_date_1, df_xzfq.refund_date)
df_xzfq.shape

df_xzfq.drop(columns=['refund_date_1','refund_date_ym','refund_date'],inplace = True)
df_xzfq = df_xzfq.rename(columns = {'refund_date_2':'refund_date'})
df_xzfq.shape

(4404, 4)

,order_number,order_id,refund_date_1,refund_date_ym
0,A20240927154904302,1533488,2024-09-30,2024-09
1,A20240927154904302,1533488,2024-10-30,2024-10


(439193, 26)

(439193, 23)

In [22]:
# 数据检查
# tmp1 = df_xzfq.drop_duplicates(subset = ['order_id'])[['order_id']]
# tmp2 = tmp_hk.drop_duplicates(subset = ['order_id'])[['order_id', 'order_number']]
# tmp1.shape
# tmp2.shape

# tmp3 = tmp1.merge(tmp2, on = 'order_id', how = 'inner')
# tmp3.shape

# tmp4 = tmp3.merge(df_xzfq, on = 'order_id', how = 'inner')
# tmp4.shape

# tmp4["日"]=tmp4["refund_date"].dt.day
# tmp4["日"].value_counts()
# tmp4[tmp4.日.isin([2,11])][['order_number','refund_date']]

# df_xzfq[df_xzfq.refund_date_2 != df_xzfq.refund_date].shape
# df_xzfq[df_xzfq.refund_date_2 == df_xzfq.refund_date].shape
# df_xzfq[df_xzfq.refund_date_2 != df_xzfq.refund_date][['order_id', 'refund_date', 'refund_date_1', 'refund_date_2', 'refund_date_ym']]
# 以下四个订单的应付日期<15号，订单分期表更正了  正常的订单的应付日期
# A20240929094315174 1539249  已退款
# A2024092919033313  1541755  正常
# A2024092919182056  1541798  已退款
# A2024100810242488  1568606  已退款

# tmp_hk[tmp_hk.order_number.isin(['A2024100810242488'])]
# df_xzfq[df_xzfq.order_id.isin([1539249,1541755,1541798,1568606])][['order_id', 'refund_date', 'refund_date_1', 'refund_date_2', 'refund_date_ym']]

In [23]:
# df_xz_concat[df_xz_concat.order_number.isin(['A2024080919264964','A2024092919033313','A20241218184711224'])][['order_number','应付日期','实付日期','逾期天数']]
# A2024080919264964
# A2024092919033313
# tmp = df_xz_concat[df_xz_concat.逾期天数 > 30].drop_duplicates(subset=['order_number'])[['order_number']]
# tmp = tmp.merge(df_hk, on = 'order_number', how = 'inner')
# tmp.shape
# tmp

## 1_52.重命名

In [24]:
df_xzfq = df_xzfq.rename(columns={'money':'当前应付租金','part_payment':'实付金额','refund_date':'应付日期','reality_refund_date':'实付日期','sort':'当前期数'})

## 1_53.实还金额核查

In [25]:
# total_check = df_ddfq.merge(df_xzfq,on=['订单ID','当前期数'],how='inner')
# total_check.info()

# total_check1= total_check[(total_check.实付金额_y < total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check1= total_check[(total_check.实付金额 < total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check1

# total_check2= total_check[(total_check.实付金额_y > total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check2= total_check[(total_check.实付金额 > total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check2

# total_check2.to_excel('E:/myfile/业务相关数据/total_check_0709-1.xlsx')

## 1_61.数据合并

In [26]:
df_xz_concat = df_xzfq.merge(dfzh3,how='right',on ='order_id')
# df_xz_concat = df_fq.merge(dfzh1,how='right',right_on ='订单id',left_on='订单ID')
# note 20241126 增加
# df_xz_concat[df_xz_concat.当前应付租金.isnull()].shape
# df_xz_concat[df_xz_concat.当前应付租金.isna()].shape
# 实付日期为空，但当前应付租金为空
# 1681669 A20241104144812228
df_xz_concat.实付金额 = df_xz_concat.实付金额 + df_xz_concat.sesame_promo_money_pay + df_xz_concat.promo_money
df_xz_concat = df_xz_concat[df_xz_concat.当前应付租金.notnull()]

# order_id == 1681669 的当前期数为空
# 1681669	A20241104144812228
# 1669581	A2024110109350797
df_xz_concat['当前期数'].notnull().sum()
df_xz_concat.shape
df_xzfq.shape
dfzh3.shape

# df_xz_concat.drop_duplicates(subset=["order_id"]).shape
# dfzh3.drop_duplicates(subset=["order_id"]).shape

# df_xz_concat['order_id'].nunique()
# dfzh3['order_id'].nunique()
# df_xz_concat[["order_id","当前期数"]].info()

439126

(439126, 105)

(439193, 23)

(33134, 83)

## 1_62.去重

In [27]:
check = df_xz_concat[["order_id","当前期数"]]  # ,"逾期天数"
all_duplicates = check[check.duplicated(keep=False)]  
# all_duplicates.to_excel('E:/myfile/业务相关数据/all_duplicates.xlsx')
print(all_duplicates.sort_values(by=['order_id','当前期数']))

        order_id  当前期数
146250    869632   1.0
146262    869632   1.0
423703    869632   1.0
423715    869632   1.0
146251    869632   2.0
...          ...   ...
436889    911772  11.0
159357    911772  12.0
159369    911772  12.0
436878    911772  12.0
436890    911772  12.0

[240 rows x 2 columns]


In [28]:
#删除重复订单
df_xz_concat.drop_duplicates(subset=["order_id","当前期数"],inplace=True)
df_xz_concat.shape

(438946, 105)

In [29]:
# 119321
# 1083822 303756
# all_duplicates[all_duplicates['order_id'] == 181950].shape
# df_xz_concat[df_xz_concat['order_id'] == 181950].shape

# df_xz_concat.to_excel('E:/myfile/业务相关数据/df_xz_concat.xlsx')

# 3.计算逾期天数

## 3_11.异常查询

In [30]:
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.status_x==1)][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

# 实付金额>0 实付日期为空 finish_date_new非空 :实付日期=订单完成时间
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

# part_payment==0 reality_refund_date.notnull() 还款状态== '续租中' part_payment=money 还款状态== '已逾期' reality_refund_date=pd.na
# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test

# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test

# 实付为0，实付日期非空，完成时间为空，无期数取消：实付日期清空
# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test.订单状态值.value_counts()

# 实付日期为空，完成时间为空，取消期数>=1 ：状态=已取消
# dd_test = df_xz_concat[ (df_xz_concat.实付日期.isnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn>=1)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test.订单状态值.value_counts()

# 部分还款，实付日期非空，完成时间为空 用户： 实付日期清空
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test1

# 部分还款，实付日期为空，完成时间非空 用户： 实付日期=完成时间
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test1.订单状态值.value_counts()

# 已买断 但无实付日期及金额：实付日回写
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# # dd_test1.订单状态值.value_counts()
# dd_test1

## 3_12.实还日期修正

In [31]:
df_xz_concat['实付日期'] = pd.to_datetime(df_xz_concat['实付日期']).dt.date
df_xz_concat['finish_date_new'] = pd.to_datetime(df_xz_concat['finish_date_new']).dt.date

In [32]:
# （1）部分还款 用户 实付日期非空 且非买断 非完成 ： 实付日期清空  
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull()),pd.NaT,df_xz_concat.实付日期)

df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()     

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
331354,A2024110123471583,1672926,499.85,499.85,2025-02-04,2025-02-01,3.0,4.0,还款中,NaT,4,NaT,NaN,NaT
334977,A20241106135426228,1689980,499.85,499.85,2024-12-09,2024-12-08,3.0,2.0,已逾期,NaT,4,NaT,NaN,NaT
334978,A20241106135426228,1689980,499.85,499.85,2025-01-09,2025-01-05,3.0,3.0,已逾期,NaT,4,NaT,NaN,NaT
335542,A2024110708472957,1692436,457.98,457.98,2025-01-10,2025-01-08,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT
335543,A2024110708472957,1692436,457.98,457.98,2025-02-10,2025-02-08,3.0,4.0,还款中,NaT,4,NaT,NaN,NaT


In [33]:
# （2）已完成 还款 用户  实付日期 补充 订单完成时间       实付金额>0  实付日期为空  finish_date_new非空  :实付日期=订单完成时间finish_date_new
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()       

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
334978,A20241106135426228,1689980,499.85,499.85,2025-01-09,2025-01-05,3.0,3.0,已逾期,NaT,4,NaT,NaN,NaT
335542,A2024110708472957,1692436,457.98,457.98,2025-01-10,2025-01-08,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT
335543,A2024110708472957,1692436,457.98,457.98,2025-02-10,2025-02-08,3.0,4.0,还款中,NaT,4,NaT,NaN,NaT
372624,A20241210113506313,1839860,695.21,665.65,2025-01-13,NaT,1.0,2.0,还款中,NaT,4,NaT,NaN,NaT
372625,A20241210113506313,1839860,695.21,665.65,2025-02-13,NaT,1.0,3.0,还款中,NaT,4,NaT,NaN,NaT


In [34]:
# （3）租完即送用户 A2023081019051714 补还款时间 
df_xz_concat['实付日期1'] = np.where((df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12),'2024-07-22',df_xz_concat['实付日期1'])
df_xz_concat[(df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12)][['实付日期1']]

,实付日期1
60964,2024-07-22


In [35]:
# （4）未还款 且 未完成 未取消用户 ： 实付日期清空
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull()),pd.NaT,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
8862,A2023021015233353,125911,633.21,0.0,2023-03-13,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
9558,A2023021414250354,132517,334.26,0.0,2023-03-17,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
9636,A2023021504342232,133713,396.66,0.0,2023-03-18,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
10650,A2023022114312437,145022,556.04,0.0,2023-03-24,2023-03-20,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
12255,A2023030700325746,166163,288.77,0.0,2024-01-10,2023-04-17,1.0,11.0,已买断,NaT,8,NaT,NaN,NaT


In [36]:
# （5）实付为0，实付日期为空，但 完成时间非空，买断金额>0： 实付日期回写

df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.notnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']] .tail()

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
182248,A2024041316125534,995895,1061.16,0.0,2024-12-16,NaT,1.0,9.0,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
182249,A2024041316125534,995895,1061.16,0.0,2025-01-16,NaT,1.0,10.0,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
182250,A2024041316125534,995895,1061.16,0.0,2025-02-16,NaT,1.0,11.0,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
182251,A2024041316125534,995895,1063.94,0.0,2025-03-16,NaT,1.0,12.0,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
212009,A2024060716074819,1111255,1085.68,0.0,2025-05-10,NaT,1.0,12.0,已买断,2024-12-14 14:59:21,8,2024-12-14,7.0,2024-12-14


In [37]:
# （6）部分还款 
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [38]:
# （7）未完成 但含有分期取消的 订单
# check = df_xz_concat[(df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull()) & df_xz_concat.nn>=1][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# check.订单状态值.value_counts()

In [39]:
# （8）实付0，实付日期为空，但 完成时间非空：为取消订单，实付日期=完成时间
df_xz_concat['实付日期new'] = np.where( (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)
df_xz_concat[ (df_xz_concat.实付日期new.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期1,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [40]:
# （9）属于部分还款 :无实付日期
# dd_test2 = df_xz_concat[(df_xz_concat.实付金额 >0) & (df_xz_concat.实付日期new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# dd_test2

In [41]:
# （10）df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [42]:
# （11）A202207050851373、A20241104144812228 应付日期修复
df_xz_concat['应付日期'] = np.where((df_xz_concat.应付日期.isnull()) & (df_xz_concat.order_number.isin(['A202207050851373','A20241104144812228'])),pd.to_datetime('2023-07-05') ,df_xz_concat.应付日期 )
df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期new,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [43]:
# （12）实付金额>0  实付日期.isnull()  还款状态==已买断  :  逾期天数=0  实付日期=订单完成时间
# 实付金额>0  实付日期.isnull()  还款状态==已逾期  :实付金额 =0  
# 实付金额==0  实付日期.notnull()  还款状态== '续租中' 实付金额=当前应付租金   还款状态== '已逾期' 实付日期=pd.na
# 0<实付金额<当前应付租金  实付日期.notnull()  还款状态== '已逾期'  :  实付金额=0  & 实付日期=pd.na

# df_xz_concat['实付金额new'] = df_xz_concat.apply(lambda x:
#     0 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已逾期'
#     else 0 if x.实付金额>0 and x.实付金额<x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.当前应付租金 if x.实付金额==0 and pd.notna(x.实付日期) and x.还款状态=='续租中'
#     else x.实付金额
#     ,axis = 1)

In [44]:
# （13）df_xz_concat['实付日期new'] = df_xz_concat.apply(lambda x:
#     x.订单完成时间 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已买断'
#     else np.nan if x.实付金额>=0 and x.实付金额 < x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.实付日期
#     ,axis = 1)

## 3_13.获取逾期截止时间

In [45]:
df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['now_date'].head()

0   2025-02-23
1   2025-02-23
2   2025-02-23
3   2025-02-23
4   2025-02-23
Name: now_date, dtype: datetime64[s]

## 3_14.重新定义状态

In [46]:
df_xz_concat[(df_xz_concat.实付日期new.isnull()) & (~df_xz_concat.订单状态值.isin([8,10]))][['rembursement_status']].value_counts()
df_xz_concat[(df_xz_concat.实付日期new.isnull())& (pd.isna(df_xz_concat.finish_date_new))][['订单状态']].value_counts()

rembursement_status
还款中                    89623
已逾期                    44072
续租中                    16939
未到首期还款日                14485
检测中                       48
已完成                       24
Name: count, dtype: int64

订单状态
租赁中     155697
待归还       6184
已退款       4548
待收货       2538
待发货        664
已完成        220
订单取消        60
检测中         48
Name: count, dtype: int64

In [47]:
conditions = [  
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.isna(df_xz_concat['finish_date_new'])) & (df_xz_concat['订单状态值'].isin([8, 10]))),  
    ((pd.to_datetime(df_xz_concat['now_date']) <= pd.to_datetime(df_xz_concat['应付日期'])) ),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) <= pd.to_datetime(df_xz_concat['应付日期']))),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) > pd.to_datetime(df_xz_concat['应付日期']))) ,
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['now_date']) > pd.to_datetime(df_xz_concat['应付日期'])) )  

]  
  
choices = ['已取消', '未到还款日', '正常还款', '已逾期支付','已逾期']  
  
# 使用numpy.where来应用条件  
df_xz_concat['状态'] = np.select(conditions, choices, default='其他')

df_xz_concat['状态'].value_counts()

状态
正常还款     208135
未到还款日    166224
已逾期支付     31221
已逾期       28598
已取消        4768
Name: count, dtype: int64

In [48]:
# df_xz_concat['实付日期new'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')
# df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')

# # E:/myfile/业务相关数据/风控相关数据输出_{Today}.xlsx
# df_xz_concat.to_excel('E:/myfile/业务相关数据/报错表0902.xlsx')

## 3_15.重新定义逾期天数

In [49]:
df_xz_concat['实付日期new'].isnull().sum()
empty_rows = df_xz_concat[df_xz_concat['应付日期'].isna()]
empty_rows.order_number  

169959

Series([], Name: order_number, dtype: object)

In [50]:
# 异常值删除，应付时间为空
df_xz_concat.shape
df_xz_concat=df_xz_concat[~df_xz_concat['应付日期'].isnull()]
df_xz_concat.shape

(438946, 109)

(438946, 109)

In [51]:
#实付日期为空且完成时间为空  订单状态为已退款或已完成  :  逾期天数=0 
#note 20241127 逾期天数 截止目前的逾期天数 ；逾期一直未还的客户，应付日期越靠前，逾期天数越高
# A202207181530165
# A2024092919033313
# df_xz_concat[df_xz_concat.order_number == 'A202207181530165'][['order_number','逾期天数','应付日期','']]
# df_xz_concat[df_xz_concat.order_number == 'A2024092919033313'][['应付日期','实付日期','实付日期new','实付日期1','finish_date_new','当前期数','逾期天数','状态'
# ,'实付金额','当前应付租金','sesame_promo_money_pay','promo_money']]

df_xz_concat['逾期天数'] = df_xz_concat.apply(lambda x:
    0 if x['状态'] in ['未到还款日','正常还款','已取消']  
    else (pd.to_datetime(x['实付日期new']) - pd.to_datetime(x['应付日期'])).days  if pd.to_datetime(x['实付日期new']) >= pd.to_datetime(x['应付日期'])
    else  (pd.to_datetime(x['now_date']) - pd.to_datetime(x['应付日期'])).days   if   pd.to_datetime(x['now_date']) > pd.to_datetime(x['应付日期']) and pd.isna(x['实付日期new'])
    else 0 
    ,  axis=1)

# df_xz_concat['逾期天数'].value_counts()
df_xz_concat[df_xz_concat['逾期天数']>0]['rembursement_status'].value_counts()

rembursement_status
已逾期        41972
还款中         6454
续租中         5266
已买断         5080
已完成          861
检测中          147
未到首期还款日       26
Name: count, dtype: int64

In [52]:
# liudan
# # 定义一个函数来计算逾期天数
# def calculate_overdue_days(row):
#     if row['状态'] in ['未到还款日', '正常还款', '已取消']:
#         return 0
    
#     # 转换日期为 datetime 类型，并处理 None 和 NaN
#     实付日期new = pd.to_datetime(row['实付日期new'], errors='coerce')
#     应付日期 = pd.to_datetime(row['应付日期'], errors='coerce')
#     now_date = pd.to_datetime(row['now_date'], errors='coerce')

#     if pd.isna(实付日期new) or pd.isna(应付日期):
#         return 0
    
#     if 实付日期new >= 应付日期:
#         return (实付日期new - 应付日期).days
    
#     if pd.isna(实付日期new) and now_date > 应付日期:
#         return (now_date - 应付日期).days
    
#     return 0

# # 应用函数并创建新的 '逾期天数' 列
# df_xz_concat['逾期天数'] = df_xz_concat.apply(calculate_overdue_days, axis=1)

# # 显示结果
# print(df_xz_concat['逾期天数'].value_counts())

## 3_16.逾期天数处理

In [53]:
def over_due_days(df):  
    # 转换日期为pandas的日期类型
    df['实付日期'] = pd.to_datetime(df['实付日期'])  
    df['应付日期'] = pd.to_datetime(df['应付日期'])  
      
    # 创建一个新的列来存储结果  
    df['逾期天数new'] = np.nan  # 初始化为NaN  
      
    # 应用逻辑判断  
    mask1 = df['状态'].isin(['已逾期', '未到还款日', '已取消','已提前支付'])  
    mask2 = df['实付日期'].isnull()  
    mask3 = (df['实付日期'] - df['应付日期']).dt.days <= 0  
      
    # 更新计算天数列  
    df.loc[mask1, '逾期天数new'] = df.loc[mask1, '逾期天数']  
    df.loc[~mask1 & mask2, '逾期天数new'] = 0   
    df.loc[~mask1 & ~mask2 & mask3, '逾期天数new'] = 0  
    df.loc[~mask1 & ~mask2 & ~mask3, '逾期天数new'] = (df['实付日期'] - df['应付日期']).dt.days  
      
    # 实付日期为空时设置计算天数为0：  
    # df.loc[df['实付日期'].isnull(), '逾期天数new'] = 0  
      
    return df 

    # df_ddfq = over_due_days(df_ddfq)

## 3_17.稽查

In [54]:
# df_ddfq.columns
# df_xz_concat.shape
# check_fq_df = df_xz_concat.merge(df_ddfq,on=['订单ID','当前期数'],how='left')
# check_fq_df.shape
# check_fq_df.columns
# check_fq_df[['订单ID', '当前应付租金_x', '实付金额','应付日期_x', '实付日期_x','status', '当前期数', 'is_relet','订单号_x','下单月份', '下单日期','还款状态', '实付金额new', '实付日期new', '逾期天数_x','状态_x'
            #  ,'当前应付租金_y', '实际支付金额', '应付日期_y', '实付日期_y', '状态_y', '逾期天数_y','逾期天数new']].to_excel('E:/myfile/业务相关数据/check_fq_df.xlsx')


## 3_18.确认是否到表现期

In [55]:
# import os
# file_name = os.path.basename(f_path_ck)  

# df_xz_concat['date_str'] = file_name.split('excel')[1].split('.')[0]

# df_xz_concat['date_str'].head(1)

In [56]:
# df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
# df_xz_concat['now_date']

In [57]:
# dfzh1 = over_due_days(dfzh1)

df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']).dt.date
# df_xz_concat['agr_days'] = (pd.to_datetime(df_xz_concat['now_date']) - df_xz_concat['应付日期']).dt.days 

In [58]:
# from datetime import timedelta
df_xz_concat['agr_1d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=1)
df_xz_concat['agr_4d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=4)
df_xz_concat['agr_7d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=7)
df_xz_concat['agr_15d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=15)
df_xz_concat['agr_30d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=30)

df_xz_concat[['agr_30d','agr_15d','agr_7d','agr_4d','agr_1d','应付日期']].head()

,agr_30d,agr_15d,agr_7d,agr_4d,agr_1d,应付日期
0,2022-08-04,2022-07-20,2022-07-12,2022-07-09,2022-07-06,2022-07-05
1,2022-09-04,2022-08-20,2022-08-12,2022-08-09,2022-08-06,2022-08-05
2,2022-10-05,2022-09-20,2022-09-12,2022-09-09,2022-09-06,2022-09-05
3,2022-11-04,2022-10-20,2022-10-12,2022-10-09,2022-10-06,2022-10-05
4,2022-12-05,2022-11-20,2022-11-12,2022-11-09,2022-11-06,2022-11-05


In [59]:
import os

# file_name = os.path.basename(f_path_ck)  
df_xz_concat['date_str'] = df_xz_concat['now_date']
df_xz_concat['agr_1d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_1d) <= pd.to_datetime(df_xz_concat.date_str),1,0)

df_xz_concat['agr_4d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_4d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_7d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_7d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_15d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_15d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_30d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_30d) <= pd.to_datetime(df_xz_concat.date_str),1,0)


df_xz_concat[['agr_30d','应付日期','date_str','agr_30d_cum']].head(5)
# df_xz_concat[(df_xz_concat.agr_7d_cum == 0) & (df_xz_concat.agr_4d_cum == 1)].head()
# df_xz_concat[df_xz_concat.order_id == 254588]

,agr_30d,应付日期,date_str,agr_30d_cum
0,2022-08-04,2022-07-05,2025-02-23,1
1,2022-09-04,2022-08-05,2025-02-23,1
2,2022-10-05,2022-09-05,2025-02-23,1
3,2022-11-04,2022-10-05,2025-02-23,1
4,2022-12-05,2022-11-05,2025-02-23,1


In [60]:
# note 20241211 数据检查
# df_xz_concat[(df_xz_concat.应付日期>='2024-12-01')&(df_xz_concat.应付日期<='2024-12-31')&(df_xz_concat.order_number.isin(
# ['A20241101174043208'
# ,'A2024110313351882'
# ,'A20241103155713186'
# ,'A2024110318311457'
# ,'A20241104145450273'
# ,'A20241104164506212'
# ,'A20241105105325369'
# ,'A20241105154302243'
# ,'A2024110515050030'
# ,'A2024110610041421'
# ,'A20241106144302240'
# ,'A2024110709254879'
# ,'A20241107103123201'
# ,'A20241107195535157'
# ,'A20241108104251268'
# ,'A20241108155751456'
# ,'A2024110715142073'
# ,'A2024110817092834'
# ]))][['order_number','agr_1d_cum','agr_4d_cum','agr_7d_cum','agr_4d','应付日期','date_str','agr_1d','下单日期']]

## 3_19.观察日定义 每月月底时间

In [61]:
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    # note n 续租的没有统计
    for n in range(0,13):
    # 2025.2.8 
    # for n in range(0,16):  
        # df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1)
        # note 20241205 修改 月末的的跳级了，所以不要加1，mob0对齐
        # df[f'mob_date_{n}'] = np.where(df["下单日期"].dt.day.isin([28,29,30,31]), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = np.where((df["下单日期"].dt.day == 31)|(df["下单日期"].isin(['2023-02-28','2024-02-29','2025-02-28']))|((df["下单日期"].dt.month.isin([4,6,9,11]))&(df["下单日期"].dt.day == 30)), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}'])
    return df

df_xz_concat.shape
df_xz_concat = mob_date(df_xz_concat)
df_xz_concat.shape

# A2023072814030812  2023-7-28 加收二期，月末为账单日
# A2023073012170638  2023-7-30 3+1，2023.8无账单日
# A2023070115535442  2023-7-1
# A2023073114301076  2023-7-31 1元+1期，续租

# A202301011402371 2023-01-01
# A202301290815565 2023-01-29
# A2023013100341134 2023-01-31
# A2023022801142011 2023-02-28
# A2023022709283120  2023-02-27
# A202302050022319  2023-02-05
# A2023040503314720 2023-04-05
# A2023042803470516  2023-04-28
# A2023043000524068 2023-04-30
# A2023063003392242 2023-06-30
# A202306280105556 2023-06-28
# A202306050324407  2023-06-05
# A202310050100222 2023-10-05
# A2023102901224646 2023-10-29
# A2023103101255825 2023-10-31

(438946, 121)

(438946, 160)

In [62]:
df_xz_concat['date_str']  = pd.to_datetime( df_xz_concat['date_str'] )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']) 

In [63]:
# var = ['order_number','当前期数','mob_date_12','paid_date_mob12','paid_money_mob12','mob_date_13','paid_date_mob13','paid_money_mob13','mob_date_14','paid_date_mob14','paid_money_mob14','mob_date_15','paid_date_mob15','paid_money_mob15']
# df_xz_concat[df_xz_concat.order_number.isin(['A2024060200290566','A2024063001194047','A202308262317036','A2024031113200319','A202207050851373'])][var]
# df_xz_concat[df_xz_concat.order_number.isin(['A202207050851373'])][var]

## 3_20.更新逾期天数

In [64]:
# （1）逾期天数校正
# note 20241213
# 首期实际付款日期为空的订单，为首期取消或者待发货的，贷后表现逾期天数更新为0（各期更新为0）

# 把符合条件的订单拿出来，逾期天数全部更新为0
sql_xz = ''' 
SELECT distinct tprm.order_number,1 as label
# ,ymos.reality_refund_date,ymos.sort as '当前期数',ymos.status
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
where ymos.sort=1 and ymos.reality_refund_date is null
;
'''
tmp = query(sql_xz)
tmp.shape
# tmp[:2]

df_xz_concat = df_xz_concat.merge(tmp, on = 'order_number', how = 'left')
df_xz_concat['label'] = df_xz_concat['label'].fillna(0)
df_xz_concat['label'].value_counts()

df_xz_concat[df_xz_concat['label'] == 1].drop_duplicates(subset = 'order_number').shape

df_xz_concat['逾期天数'] = np.where(df_xz_concat['label'] == 1, 0, df_xz_concat['逾期天数'])
# df_xz_concat['逾期天数_new'] = np.where(df_xz_concat['label'] == 1, 0, df_xz_concat['逾期天数'])
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].shape
# df_xz_concat[(df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']) & (df_xz_concat.逾期天数 > 30)].shape
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']][['order_number','逾期天数','逾期天数_new']]
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].逾期天数.max()
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].逾期天数.min()
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']][['order_number','逾期天数','逾期天数_new']].to_excel('tmp.xlsx', index = False)

# A202207181530165
# A2024092919033313 号卡
# A2023011718580545 vintage分子是月末状态，FSTQPD是发生过逾期
# df_xz_concat[df_xz_concat.order_number == 'A2023011718580545'][['order_number','应付日期','实付日期','实付金额','逾期天数']]
# df_mob_total[df_mob_total.order_number == 'A2023011718580545'][['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','MOB','overdue_days_1','overdue_days_4','overdue_days_7','overdue_days_15','overdue_days_31']]

# （2）2025.2.21
df_xz_concat['逾期天数'] = np.where(df_xz_concat.当前期数 == 1, 0, df_xz_concat.逾期天数)

(723, 2)

label
0.0    430414
1.0      8532
Name: count, dtype: int64

(702, 161)

## 3_21.更新租期到期的逾期天数

In [65]:
# # （1）更新归还逾期的订单的逾期天数
# # 本次未统计在内，应该用 df_xz_concat

# # （1-1）未归还逾期或未到期或系统当天到期
# df_xz_concat.shape
# conditions_1 = (df_xz_concat.is_relet == 0)
# tmp1 = df_xz_concat[(conditions_1 & (df_xz_concat.end_date >= df_xz_concat.finish_date_new)) | (conditions_1 & (df_xz_concat.end_date >= df_xz_concat.date_str))]
# tmp1.shape

# # （1-2）续租
# tmp2 = df_xz_concat[df_xz_concat.is_relet > 0 ]
# tmp2.shape
# # （1-3）出现过归还逾期（包含租期一直逾期直到归还逾期的）--仅首续有归还逾期，被减数为最近未支付的应付日期
# conditions_2 = (df_xz_concat.end_date < df_xz_concat.finish_date_new)
# conditions_3 = (df_xz_concat.finish_date_new.isnull())
# conditions_4 = (df_xz_concat.end_date < df_xz_concat.date_str)
# tmp3 = df_xz_concat[(conditions_1 & conditions_2) | (conditions_1 & conditions_3 & conditions_4)]
# tmp3.shape

# # 367644 + 55490

# # 计算到期后逾期天数--订单已经完成的： 逾期天数为 finish_date_new- end_date；订单未完成的，逾期天数为 date_str- 实付日期new为空的最小的 应付日期
# # ((df_xz_concat['finish_date_new'] - df_xz_concat['end_date']).dt.days > 0) 
# # A2024020317504938--订单已完成
# # A202207171820512 --订单已完成
# # A202207181530165--订单未完成
# # var = ['order_number','当前期数','应付日期','实付日期new','end_date','date_str','逾期天数','pay_date','order_finish_date','订单完成时间','finish_date_new']
# # df_xz_concat[df_xz_concat.order_number == 'A202207181530165'][var]

In [66]:
# （2）数据检查
# df_fq_concat[df_fq_concat.下单日期 == '2024-06-30'][['order_number']][:2]
# A2024060200290566  A2024063001194047 A202308262317036（续租） A2024031113200319（提早归还） A202207050851373（续租提前买断）
# var = ['order_number', '当前期数', '下单日期', '应付日期', '首期应还日期', '当前应付租金','rembursement_status','订单状态','if_outpay','状态','是否提前还款','实付日期new','逾期天数','order_finish_date','finish_date_new']
# df_fq_concat[df_fq_concat.order_number.isin(['A2024060200290566','A2024063001194047','A202308262317036','A2024031113200319','A202207050851373'])][var]

# var = ['order_number', '当前期数', '下单日期', '应付日期', '当前应付租金','rembursement_status','订单状态','if_outpay','状态','实付日期new','finish_date_new','status_x','paid_periods','实付金额','date_tmp_1','date_tmp_2','订单状态_new','pay_date']
# var = ['order_number','is_relet','当前期数', '下单日期', '应付日期', '当前应付租金', '实付金额','rembursement_status','订单状态','if_outpay','状态','首期应还日期', '买断价', '应付总租金', '买断尾款', '订单状态_new','租赁方案']
# df_xz_concat[df_xz_concat.order_number.isin(['A2024060200290566','A2024063001194047','A2024031113200319'])][var]
# df_xz_concat[df_xz_concat.order_number.isin(['A202308262317036'])][var]
# df_xz_concat[df_xz_concat.order_number.isin(['A202207050851373'])][var] 

# 4.加工资方所需字段

In [67]:
# # 20250212
# # 限定'芝麻租物','搜索渠道'渠道
# df_xz_concat.shape
# df_xz_concat[df_xz_concat.归属渠道.isnull()].shape
# df_xz_concat[df_xz_concat.归属渠道.isin(['芝麻租物','搜索渠道'])].shape
# # df_xz_concat.归属渠道.value_counts()

# df_xz_concat = df_xz_concat[df_xz_concat.归属渠道.isin(['芝麻租物','搜索渠道'])]

In [68]:
# 20250207
df_xz_concat_1 = df_xz_concat.copy()
df_xz_concat_1.loc[:,"租赁方案"]=np.where(df_xz_concat_1["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df_xz_concat_1['finish_date_new'] = pd.to_datetime(df_xz_concat_1['finish_date_new'])

# （1）取首期应还日期，区分续租和非续租  首期应还日期
tmp1 = df_xz_concat_1[(df_xz_concat_1.is_relet == 0) & (df_xz_concat_1.当前期数 == 1)][['order_number','应付日期']]
tmp1 = tmp1.rename(columns = {'应付日期': '首期应还日期'})
tmp2 = df_xz_concat_1[df_xz_concat_1.is_relet == 0].merge(tmp1, on = 'order_number', how = 'left')

tmp3 = df_xz_concat_1[df_xz_concat_1.is_relet != 0].groupby(['order_number']).agg({'应付日期':'min'}).reset_index()
tmp3 = tmp3.rename(columns = {'应付日期': '首期应还日期'})
tmp4 = df_xz_concat_1[df_xz_concat_1.is_relet != 0].merge(tmp3, on = 'order_number', how = 'left')

df_xz_concat_1 = pd.concat([tmp2, tmp4])
df_xz_concat_1.shape

tmp5 = df_xz_concat_1.groupby(['order_number']).agg({'应付日期':'max'}).reset_index()
tmp5 = tmp5.rename(columns = {'应付日期': '最大应还日期'})
df_xz_concat_1 = df_xz_concat_1.merge(tmp5, on = 'order_number', how = 'left')
df_xz_concat_1.shape

del tmp1
del tmp2
del tmp3
del tmp4
del tmp5

# （2）提取买断金等信息，区分续租和非续租 买断价，续租的买断价 包含了 首租的买断价
# 续租买断尾款 为续租总租金，续租买断尾款其实为0
# 首租订单，首租买断尾款为续租总租金，实际中，续租总租金 大于 首租买断尾款
# all_money：续租的已完成的话，为首租+续租总租金；续租的未完成的话，为首租的总租  'A202308262317036','A202207050851373'
sql = '''
SELECT tprm.order_id, tprm.order_number, tprm.order_create_time 下单日期
    ,tod.actual_money 首租买断价
    ,tod.new_actual_money 续租买断价
    , tprm.all_deposit 总押金
    , tprm.total_receivable 总应收
    , tprm.total_received 总已收
    # , om.all_money 首租总租金
    # , (tod.new_actual_money-om.all_money) 续租总租金
    # , (tod.actual_money-om.all_money) 首租买断尾款
    , tprm.purchase_amount 采购金额
    , om.has_actual 是否买断
    , om.relet_status 是否续租
    , tprm.paid_periods 已还期数
    , tod.early_completion_mark 是否提前完成
    , tmu.id_card_num 身份证号
    , tol.delivery_province_name 地区
    , pa.type 活动类型
    , pa.type, too.money 滞纳金, tprm.liquidated_damages 违约金
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_digua_business.t_platform_activity pa on om.activity_id=pa.id
left join db_digua_business.t_member_user tmu on om.user_id=tmu.id
left join db_digua_business.t_order_details tod on tod.order_id=tprm.order_id
left join db_digua_business.t_order_logistics tol on tol.order_id=tprm.order_id
left join (SELECT t.order_id,sum(case when t.capital_type =6 and t.`status`=2 then t.money end) as other_money 
,sum(case when t.capital_type != 5 AND  t.`status` = 2 then t.money end) as money
from db_digua_business.t_order_other t GROUP BY 1 ) too on too.order_id=om.id
'''

df_temp_1 = query(sql)
df_temp_1.shape
df_temp_1 = df_temp_1.drop_duplicates(subset = ['order_number'])
df_temp_1.shape

df_temp_1_1 = df_xz_concat_1[(df_xz_concat_1.is_relet == 0)].merge(df_temp_1[['order_number','首租买断价']], on = 'order_number', how = 'left')
df_temp_1_1 = df_temp_1_1.rename(columns = {'首租买断价': '买断价'})
df_temp_1_1.shape

df_temp_1_2 = df_xz_concat_1[(df_xz_concat_1.is_relet != 0)].merge(df_temp_1[['order_number','续租买断价']], on = 'order_number', how = 'left')
df_temp_1_2 = df_temp_1_2.rename(columns = {'续租买断价': '买断价'})
df_temp_1_2.shape

df_xz_concat_1 = pd.concat([df_temp_1_1, df_temp_1_2])

# （3）加工应付总租金，区分续租和非续租 应付总租金
df_temp_2 = df_xz_concat_1[(df_xz_concat_1.is_relet == 0)].groupby(['order_number']).agg({'当前应付租金':'sum'}).reset_index()
df_temp_2 = df_temp_2.rename(columns = {'当前应付租金': '应付总租金'})
df_temp_2.shape

df_temp_3 = df_xz_concat_1[(df_xz_concat_1.is_relet != 0)].groupby(['order_number']).agg({'当前应付租金':'sum'}).reset_index()
df_temp_3 = df_temp_3.rename(columns = {'当前应付租金': '应付总租金'})
df_temp_3.shape

df_temp_4 = df_xz_concat_1[(df_xz_concat_1.is_relet == 0)].merge(df_temp_2, on = 'order_number', how = 'left')
df_temp_4.shape

df_temp_5 = df_xz_concat_1[(df_xz_concat_1.is_relet != 0)].merge(df_temp_3, on = 'order_number', how = 'left')
df_temp_5.shape

df_xz_concat_1 = pd.concat([df_temp_4, df_temp_5])

#（4）买断金 续租的买断尾款为 0 
df_xz_concat_1['买断尾款'] = df_xz_concat_1['买断价'] - df_xz_concat_1['应付总租金']
df_xz_concat_1['买断尾款'] = np.where(df_xz_concat_1.is_relet == 0, df_xz_concat_1['买断尾款'], 0)
df_xz_concat_1.shape

del df_temp_1_1
del df_temp_1_2
del df_temp_1
del df_temp_2
del df_temp_3
del df_temp_4
del df_temp_5

# （5）补三期数据  为了关联买断数据
# 当前期数、应付日期 改变，其他属性不变
tmp1 = df_xz_concat_1.groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
var = ['order_number','order_id','is_relet','当前期数', '下单日期', '应付日期','首期应还日期', '买断价', '应付总租金', '买断尾款','租赁方案','rembursement_status','finish_date_new']
tmp1.shape

tmp2 = tmp1.merge(df_xz_concat_1[var], on = ['order_number', '当前期数'], how = 'inner')
tmp2['当前期数'] = tmp2['当前期数'] + 1
tmp2['应付日期'] = tmp2['应付日期'] + pd.DateOffset(months = 1)

tmp3 = tmp1.merge(df_xz_concat_1[var], on = ['order_number', '当前期数'], how = 'inner')
tmp3['当前期数'] = tmp3['当前期数'] + 2
tmp3['应付日期'] = tmp3['应付日期'] + pd.DateOffset(months = 2)

tmp4 = tmp1.merge(df_xz_concat_1[var], on = ['order_number', '当前期数'], how = 'inner')
tmp4['当前期数'] = tmp4['当前期数'] + 3
tmp4['应付日期'] = tmp4['应付日期'] + pd.DateOffset(months = 3)

tmp5 = pd.concat([tmp2, tmp3, tmp4])
tmp5.shape

df_xz_concat_1 = pd.concat([df_xz_concat_1, tmp5])
df_xz_concat_1.shape
# tmp1[tmp1.下单日期 == '2024-01-27'][:1]
# A2024012707300311 1-30 02-28
# A2023032611070524 2-29 3-29
# A2022092917545514 3-31 4-30
# A202207050851373 5-31 6-30
# tmp5[tmp5.order_number.isin(['A202207050851373'])]

# （6）订单状态定位到 当月
df_xz_concat_1['date_tmp_1'] = df_xz_concat_1['应付日期'].dt.strftime('%Y-%m')
df_xz_concat_1['date_tmp_1'] = pd.to_datetime(df_xz_concat_1['date_tmp_1'])
df_xz_concat_1['date_tmp_2'] = np.where(df_xz_concat_1['finish_date_new'].notnull(), pd.to_datetime(df_xz_concat_1['finish_date_new']).dt.strftime('%Y-%m'), np.nan)
df_xz_concat_1['date_tmp_2'] = pd.to_datetime(df_xz_concat_1['date_tmp_2'])

# 判断订单结束日期非空的订单是买断还是非买断，未结束的订单不判断（资方所需的是是否结清，分买断或完成）
# 判断前15期的状态
df_xz_concat_1['订单状态_new'] = df_xz_concat_1.apply(lambda x: '已买断' if x.date_tmp_1 == x.date_tmp_2 and x.rembursement_status == '已买断'
                                              else '已完成' if x.date_tmp_1 == x.date_tmp_2 and x.rembursement_status == '已完成'
                                              else '还款中' if x.date_tmp_1 < x.date_tmp_2
                                              else '已结束' if x.date_tmp_1 > x.date_tmp_2
                                              else '其他', axis = 1
                                              )
df_xz_concat_1['提前状态'] = df_xz_concat_1.apply(lambda x: '提前买断' if x.finish_date_new < x.最大应还日期 and x.订单状态_new == '已买断'
                                              else '提前完成' if x.finish_date_new < x.最大应还日期 and x.订单状态_new == '已完成'
                                              else '其他', axis = 1
                                              )

# df_xz_concat_1 = df_xz_concat_1.drop(columns = ['date_tmp_1', 'date_tmp_2'])
df_xz_concat_1.shape

# (7)关联买断数据 只考虑总期数往后加三期的 
sql1 = '''-- 买断信息   
SELECT too.order_id
,too.real_pay_money 
,too.pay_date
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断
and too.real_pay_money > 0 and too.pay_date is not null
'''
df_out_1 = query(sql1)
df_out_1.shape
df_out_1["pay_date"] = df_out_1['pay_date'].dt.date
df_out_1["pay_date"] = pd.to_datetime(df_out_1["pay_date"],errors="coerce")
df_out_1['pay_date_m'] = df_out_1['pay_date'].dt.strftime('%Y-%m')
df_out_1['pay_date_m'] = pd.to_datetime(df_out_1['pay_date_m'])

df_out_2 = df_out_1.groupby(['order_id','pay_date_m']).agg({'real_pay_money':'sum'}).reset_index()
df_out_2 = df_out_2.rename(columns = {'real_pay_money': 'real_pay_money_new'})
df_out_2.shape

df_xz_concat_1 = df_xz_concat_1.merge(df_out_2[['order_id', 'real_pay_money_new', 'pay_date_m']], left_on = ['order_id', 'date_tmp_1'], right_on = ['order_id','pay_date_m'], how = 'left')
df_xz_concat_1 = df_xz_concat_1.drop(columns = ['date_tmp_1', 'date_tmp_2', 'pay_date_m'])

# 计算剩余买断尾款
df_xz_concat_1['real_pay_money_new'] = df_xz_concat_1['real_pay_money_new'].fillna(0)
df_xz_concat_1 = df_xz_concat_1.rename(columns = {'real_pay_money_new': '当月已收买断金'})
df_xz_concat_1 = df_xz_concat_1.sort_values(by = ['order_number','当前期数'])
df_xz_concat_1['当前累计已收买断金'] = df_xz_concat_1.groupby(['order_number'])['当月已收买断金'].cumsum()

df_xz_concat_1['剩余总应收买断金'] = df_xz_concat_1['买断尾款'] - df_xz_concat_1['当前累计已收买断金'] 
df_xz_concat_1['剩余总应收买断金'] = np.where(df_xz_concat_1['剩余总应收买断金'] < 0 , 0, df_xz_concat_1['剩余总应收买断金'])
df_xz_concat_1['剩余总应收买断金'] = np.where(df_xz_concat_1['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_xz_concat_1['剩余总应收买断金'])

df_xz_concat_1.shape
df_xz_concat_1[:1]
	
# var1 = ['order_number','is_relet','当前期数', '下单日期', '应付日期', '当前应付租金', '实付日期new','实付金额','rembursement_status','订单状态','if_outpay','状态','订单状态_new']
# var2 = ['mob_date_11','paid_date_mob11','paid_money_mob11','mob_date_12','paid_date_mob12','paid_money_mob12','mob_date_13','paid_date_mob13','paid_money_mob13','mob_date_14','paid_date_mob14','paid_money_mob14','mob_date_15','paid_date_mob15','paid_money_mob15']
# var3 = ['order_id','当前期数','应付日期','实付日期new','实付金额','finish_date_new','订单状态_new','首期应还日期','买断价','应付总租金','买断尾款','订单状态_new','real_pay_money_new','剩余总应收买断金']
# var = var1 + var2 + var3

# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024060200290566'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024063001194047'])][var3]
# 订单已完成
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024031113200319'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202308262317036'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207050851373'])][var3]
# 在15期之前买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024020317504938'])][var3]
# 在15期之后买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207171820512'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207181530165'])][var3]
# 已买断但 买断尾款	!= real_pay_money_new  可能使用优惠劵
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207151707153'])][var3]
# 续租的实付买断金非0
# 续租总租金 = 最新的买断价-首租总租金
# df_xz_concat_1[df_xz_concat_1.order_number == 'A2024072414134141']

(438946, 163)

(438946, 164)

(35310, 19)

(35304, 19)

(398010, 165)

...

(538306, 171)

(2990, 3)

(2908, 3)

(538306, 172)

,id_x,order_id,当前应付租金,实付金额,interest_money,实付日期,status_x,当前期数,order_pay_id,create_time_x,...,首期应还日期,最大应还日期,买断价,应付总租金,买断尾款,订单状态_new,提前状态,当月已收买断金,当前累计已收买断金,剩余总应收买断金
0,6506.0,8924,357.95,357.95,0.0,2022-07-05,3.0,1.0,9576,2022-07-05 08:51:37,...,2022-07-05,2024-05-31,7530.0,4295.4,3234.6,还款中,其他,0.0,0.0,3234.6


# 5.非续租订单

<!-- （1）每个mob 观察日 对应的逾期天数
（2）实付日期逻辑：如实付日期大于观察日，实付日期清空
（3）达到表现期逻辑：应付日期小于观察日，且应付日期小于当前日期
（4）逾期天数判断逻辑:如状态为已取消状态则0，如实付日期不为空则 0，如实付日期为空且观察日期大于当前日期则 当前日期-应付日期，否则 观察日期-应付日期 -->

In [69]:
# （1）非续租租期数据
# (384318, 161)
# df_fq_concat = df_xz_concat[df_xz_concat.is_relet==0]
# 2025.2.9
df_xz_concat_2 = df_xz_concat.merge(df_xz_concat_1[['order_number', '订单状态_new','当前期数']],on = ['order_number','当前期数'], how = 'inner')
# conditions_1 = (df_xz_concat_2.订单状态_new != '已结束') | ((df_xz_concat_2.订单状态_new == '已结束') & (df_xz_concat_2.实付金额 > 0))
# df_xz_concat_2 = df_xz_concat_2[conditions_1]
df_fq_concat = df_xz_concat_2[df_xz_concat_2.is_relet==0]
df_fq_concat['当前应付租金'] = np.where((df_fq_concat.订单状态_new == '已结束') & (df_fq_concat.实付金额 == 0), 0, df_fq_concat.当前应付租金)

df_fq_concat.shape
df_xz_concat.is_relet.value_counts()

# note 20241205 首个应付日期更新为下单日期，避免 首个应付日期为下个月的订单没有 把mob0的已付统计在内，首个应收日期为下个月的订单，mob1没有账单日
df_fq_concat['应付日期_new'] = np.where(df_fq_concat.当前期数 == 1, df_fq_concat.下单日期, df_fq_concat.应付日期)
# note 20241206 更新 应付日期，提前还款的统计在内
df_fq_concat['应付日期_new'] = np.where((df_fq_concat['实付日期new'].notna())&((df_fq_concat['应付日期_new'] - df_fq_concat['实付日期new']).dt.days >= 0),df_fq_concat['实付日期new'],df_fq_concat['应付日期_new'])
df_fq_concat.shape

# A2023072814030812  2023-7-28 加收二期，月末为账单日
# A2023073012170638  2023-7-30 3+1，2023.8无账单日
# A2023070115535442  2023-7-1
# A2023073114301076  2023-7-31 1元+1期，续租

(398010, 162)

is_relet
0.0    398010
1.0     40872
2.0        64
Name: count, dtype: int64

(398010, 163)

In [70]:
# （2）12期加工
# note 20241205 每月28.29.30号下单的没有统计mob_0已付的
df_mob_list = []
for i in range(0,13):
# 2025.2.9
# for i in range(0,16):
    # note 20241205 增加 (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])，不然预收的没有在第1期体现出来， A2024063001194047 下单日期!=实付日期new（第一期）
    # note 20241205 增加=，不然28或27号的没有mob0，导致 paid_money_mob 计算不正确，即还款日为月末的计算不正确
    # df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days > 0) 
    df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期_new']).dt.days >= 0) 
                        #   & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                        # 2025.2.9
                          & ((df_fq_concat['date_str'] - df_fq_concat['应付日期_new']).dt.days > 0) 
                          & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                          | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])]  # &((dfzh1['date_str'] - dfzh1[f'mob_date_{i}']).dt.days > 0)
# df_mob2 = dfzh1[((dfzh1['mob_date_2'] - dfzh1['应付日期']).dt.days > 0) & (dfzh1.状态 != '已取消') & ((dfzh1['date_str'] - dfzh1['mob_date_2']).dt.days > 0)]

    # df_mob['overdue_days_mob'] = df_mob.apply(lambda x:
    #                                           0 if x['逾期天数']==0  
    #                                           else 0  if pd.notnull(x[f'paid_date_mob{i}']) 
    #                                           else (x['date_str'] - x['应付日期']).days  if  ((x[f'mob_date_{i}'] - x['date_str']).days > 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else (x[f'mob_date_{i}'] - x['应付日期']).days if ((x[f'mob_date_{i}'] - x['date_str']).days <= 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else x['逾期天数'] ,axis=1 )
    # # np.where(df_mob[f'paid_date_mob{i}'].isnull(),(np.minimum(df_mob['date_str'],df_mob[f'mob_date_{i}']) - df_mob['应付日期']).dt.days  ,(df_mob[f'paid_date_mob{i}'] - df_mob['应付日期']).dt.days)
    # df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # df_mob_group['MOB'] = f'mob_{i}'

    # df_mob_list.append(df_mob_group)

    conditions = [ ( ( df_mob['逾期天数']==0) | (pd.notnull(df_mob[f'paid_date_mob{i}']))),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days > 0)  & (pd.isnull(df_mob[f'paid_date_mob{i}'])) ),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days <= 0) & ( pd.isnull(df_mob[f'paid_date_mob{i}'])))]  
  
    choices = [0,(df_mob['date_str'] - df_mob['应付日期']).dt.days,(df_mob[f'mob_date_{i}'] - df_mob['应付日期']).dt.days]  
  
    # 使用numpy.where来应用条件  
    # note 20241127 overdue_days_mob 截止每个mob观察月的逾期天数，月末状态，若月末不逾期了，则为0
    df_mob['overdue_days_mob'] = np.select(conditions, choices, default = df_mob['逾期天数'])

    df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # note 20241210 修改 提前支付的剔除未到期应收
    df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    df_mob_group_1 = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days >= 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                      | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])
                      ].groupby(by=['order_number']).agg({'当前应付租金':'sum'}).reset_index()
    
    # 2025.2.11，当前最大期数不用 (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])
    df_mob_group_1_1 = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days >= 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                      ].groupby(by=['order_number']).agg({'当前期数':'max'}).reset_index()
    
    df_mob_group_1 = df_mob_group_1.merge(df_mob_group_1_1, on = 'order_number', how = 'inner')

    df_mob_group = df_mob_group.merge(df_mob_group_1, on = 'order_number', how = 'left')

    df_mob_group['MOB'] = f'mob_{i}'

    df_mob_list.append(df_mob_group)

df_mob_total = pd.concat(df_mob_list, ignore_index=True)

# 2025.2.9
df_mob_total.shape
df_mob_total = df_mob_total[df_mob_total.当前应付租金 >= 0]
df_mob_total = df_mob_total.drop_duplicates(subset = ['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','当前期数'])
df_mob_total.shape
df_mob_total = df_mob_total.sort_values(by = ['order_number','当前期数','overdue_days_mob'])
df_mob_total = df_mob_total.drop_duplicates(subset = ['order_number','当前期数'])

# df_mob_total[df_mob_total.order_number == 'A202207151707153']
# var3 = ['order_id','当前期数','应付日期','当前应付租金','实付日期new','实付金额','finish_date_new','订单状态_new','首期应还日期','买断价','应付总租金','买断尾款','订单状态_new','real_pay_money_new','剩余总应收买断金']
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024060200290566'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024063001194047'])][var3]
# 订单已完成
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024031113200319'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202308262317036'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207050851373'])][var3]
# # 在15期之前买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024020317504938'])][var3]
# 在15期之后买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207171820512'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207181530165'])][var3] --2个12期
# 已买断但 买断尾款	!= real_pay_money_new  可能使用优惠劵
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207151707153'])][var3]

# paid_money_mob：当前累计实付租金  当前应付租金：当前累计应付租金
df_mob_total = df_mob_total.rename(columns = {'当前应付租金': '当前累计应付租金', 'paid_money_mob': '当前累计实付租金'})
df_mob_total.tail(10)

df_xz_concat_1.shape
df_xz_concat.shape
df_fq_concat.shape
df_mob_total.shape

# （2）
# note 20241205 每月28.29.30号下单的没有统计mob_0已付的
df_mob_list = []
for i in range(0,13):
# 2025.2.9
# for i in range(0,16):
    # note 20241205 增加 (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])，不然预收的没有在第1期体现出来， A2024063001194047 下单日期!=实付日期new（第一期）
    # note 20241205 增加=，不然28或27号的没有mob0，导致 paid_money_mob 计算不正确，即还款日为月末的计算不正确
    # df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days > 0) 
    df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期_new']).dt.days >= 0) 
                        #   & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                        # 2025.2.9
                          & ((df_fq_concat['date_str'] - df_fq_concat['应付日期_new']).dt.days > 0) 
                          & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                          | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])]  # &((dfzh1['date_str'] - dfzh1[f'mob_date_{i}']).dt.days > 0)
# df_mob2 = dfzh1[((dfzh1['mob_date_2'] - dfzh1['应付日期']).dt.days > 0) & (dfzh1.状态 != '已取消') & ((dfzh1['date_str'] - dfzh1['mob_date_2']).dt.days > 0)]

    # df_mob['overdue_days_mob'] = df_mob.apply(lambda x:
    #                                           0 if x['逾期天数']==0  
    #                                           else 0  if pd.notnull(x[f'paid_date_mob{i}']) 
    #                                           else (x['date_str'] - x['应付日期']).days  if  ((x[f'mob_date_{i}'] - x['date_str']).days > 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else (x[f'mob_date_{i}'] - x['应付日期']).days if ((x[f'mob_date_{i}'] - x['date_str']).days <= 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else x['逾期天数'] ,axis=1 )
    # # np.where(df_mob[f'paid_date_mob{i}'].isnull(),(np.minimum(df_mob['date_str'],df_mob[f'mob_date_{i}']) - df_mob['应付日期']).dt.days  ,(df_mob[f'paid_date_mob{i}'] - df_mob['应付日期']).dt.days)
    # df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # df_mob_group['MOB'] = f'mob_{i}'

    # df_mob_list.append(df_mob_group)

    conditions = [ ( ( df_mob['逾期天数']==0) | (pd.notnull(df_mob[f'paid_date_mob{i}']))),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days > 0)  & (pd.isnull(df_mob[f'paid_date_mob{i}'])) ),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days <= 0) & ( pd.isnull(df_mob[f'paid_date_mob{i}'])))]  
  
    choices = [0,(df_mob['date_str'] - df_mob['应付日期']).dt.days,(df_mob[f'mob_date_{i}'] - df_mob['应付日期']).dt.days]  
  
    # 使用numpy.where来应用条件  
    # note 20241127 overdue_days_mob 截止每个mob观察月的逾期天数，月末状态，若月末不逾期了，则为0
    df_mob['overdue_days_mob'] = np.select(conditions, choices, default = df_mob['逾期天数'])

    df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # note 20241210 修改 提前支付的剔除未到期应收
    df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    df_mob_group_1 = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days >= 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                      | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])
                      ].groupby(by=['order_number']).agg({'当前应付租金':'sum'}).reset_index()
    
    # 2025.2.11，当前最大期数不用 (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])
    df_mob_group_1_1 = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days >= 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                      ].groupby(by=['order_number']).agg({'当前期数':'max'}).reset_index()
    
    df_mob_group_1 = df_mob_group_1.merge(df_mob_group_1_1, on = 'order_number', how = 'inner')

    df_mob_group = df_mob_group.merge(df_mob_group_1, on = 'order_number', how = 'left')

    df_mob_group['MOB'] = f'mob_{i}'

    df_mob_list.append(df_mob_group)

df_mob_total = pd.concat(df_mob_list, ignore_index=True)

# 2025.2.9
df_mob_total.shape
df_mob_total = df_mob_total[df_mob_total.当前应付租金 >= 0]
df_mob_total = df_mob_total.drop_duplicates(subset = ['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','当前期数'])
df_mob_total.shape
df_mob_total = df_mob_total.sort_values(by = ['order_number','当前期数','overdue_days_mob'])
df_mob_total = df_mob_total.drop_duplicates(subset = ['order_number','当前期数'])

# df_mob_total[df_mob_total.order_number == 'A202207151707153']
# var3 = ['order_id','当前期数','应付日期','当前应付租金','实付日期new','实付金额','finish_date_new','订单状态_new','首期应还日期','买断价','应付总租金','买断尾款','订单状态_new','real_pay_money_new','剩余总应收买断金']
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024060200290566'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024063001194047'])][var3]
# 订单已完成
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024031113200319'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202308262317036'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207050851373'])][var3]
# # 在15期之前买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024020317504938'])][var3]
# 在15期之后买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207171820512'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207181530165'])][var3] --2个12期
# 已买断但 买断尾款	!= real_pay_money_new  可能使用优惠劵
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207151707153'])][var3]

# paid_money_mob：当前累计实付租金  当前应付租金：当前累计应付租金
df_mob_total = df_mob_total.rename(columns = {'当前应付租金': '当前累计应付租金', 'paid_money_mob': '当前累计实付租金'})
df_mob_total.tail(10)

df_xz_concat_1.shape
df_xz_concat.shape
df_fq_concat.shape
df_mob_total.shape

# (520033, 170)
# (424063, 161)
# (384318, 163)
# (234387, 7)

(408321, 7)

(236931, 7)

,order_number,下单日期,当前累计实付租金,overdue_days_mob,当前累计应付租金,当前期数,MOB
31685,A2025012615480550,2025-01-26,719.04,0,719.04,1.0,mob_0
31686,A2025012617483443,2025-01-26,537.94,0,537.94,1.0,mob_0
31687,A202501261811289,2025-01-26,2151.96,0,1.00,1.0,mob_0
31688,A2025012619102611,2025-01-26,599.75,0,599.75,1.0,mob_0
31689,A2025012622475925,2025-01-26,459.82,0,459.82,1.0,mob_0
31690,A2025012709163214,2025-01-27,719.04,0,719.04,1.0,mob_0
31691,A2025012710270720,2025-01-27,1082.94,0,1082.94,1.0,mob_0
31692,A2025012713364018,2025-01-27,455.93,0,455.93,1.0,mob_0
31693,A2025012714175415,2025-01-27,455.93,0,455.93,1.0,mob_0
31694,A2025012716123412,2025-01-27,744.53,0,744.53,1.0,mob_0


(538306, 172)

(438946, 161)

...

,order_number,下单日期,当前累计实付租金,overdue_days_mob,当前累计应付租金,当前期数,MOB
31685,A2025012615480550,2025-01-26,719.04,0,719.04,1.0,mob_0
31686,A2025012617483443,2025-01-26,537.94,0,537.94,1.0,mob_0
31687,A202501261811289,2025-01-26,2151.96,0,1.00,1.0,mob_0
31688,A2025012619102611,2025-01-26,599.75,0,599.75,1.0,mob_0
31689,A2025012622475925,2025-01-26,459.82,0,459.82,1.0,mob_0
31690,A2025012709163214,2025-01-27,719.04,0,719.04,1.0,mob_0
31691,A2025012710270720,2025-01-27,1082.94,0,1082.94,1.0,mob_0
31692,A2025012713364018,2025-01-27,455.93,0,455.93,1.0,mob_0
31693,A2025012714175415,2025-01-27,455.93,0,455.93,1.0,mob_0
31694,A2025012716123412,2025-01-27,744.53,0,744.53,1.0,mob_0


(538306, 172)

(438946, 161)

(398010, 163)

(234387, 7)

In [71]:
# （3）关联15期的表
# 2025.2.9
var_1 = ['order_number','下单日期','租赁方案','当前期数','应付日期', '当前应付租金', '实付金额','实付日期new','首期应还日期','买断价','应付总租金','买断尾款','订单状态_new','当月已收买断金','当前累计已收买断金','剩余总应收买断金','提前状态','purchase_amount','归属渠道']
df_mob_total_tmp = df_xz_concat_1[df_xz_concat_1.is_relet == 0][var_1].merge(df_mob_total[['order_number','当前期数','当前累计应付租金','当前累计实付租金','overdue_days_mob']], on = ['order_number','当前期数'], how = 'left')
df_xz_concat_1[df_xz_concat_1.is_relet == 0].shape
df_mob_total_tmp.shape

tmp_2 = df_xz_concat.groupby(['order_number']).agg({'is_relet':'max'}).reset_index()
tmp_2 = tmp_2.rename(columns = {'is_relet': 'is_relet_max'})
tmp_2.shape

df_mob_total_tmp = df_mob_total_tmp.merge(tmp_2, on = 'order_number', how ='inner')
df_mob_total_tmp.shape
del tmp_2

# 加工 df_mob_total_tmp
# 续租的订单只有12期，第12期的订单状态修改为已完成
df_mob_total_tmp['订单状态_new'] = np.where((df_mob_total_tmp.is_relet_max >= 1) & (df_mob_total_tmp.当前期数 == 12), '已完成', df_mob_total_tmp['订单状态_new'])
df_mob_total_tmp['剩余总应收买断金'] = np.where(df_mob_total_tmp['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_mob_total_tmp['剩余总应收买断金'])
# 删去订单已结束的订单
df_mob_total_tmp = df_mob_total_tmp[df_mob_total_tmp.订单状态_new != '已结束']
df_mob_total_tmp.shape
# 订单未结束的，删去未到期的，最新时间为 2025.2.10
df_mob_total_tmp = df_mob_total_tmp[df_mob_total_tmp.应付日期 <= '2025-01-31']
df_mob_total_tmp.shape
# 订单未结束的，到期未结束的则取完15期的数据，若租期一直逾期，则衍生到15期，若租期逾期，则默认不逾期
# A2024020317504938 2025.2 买断，还没表现出来
# 'A202207171820512' -- 9683，在15期之外买断，不加工租期逾期
# 'A202207181530165' --一直逾期，直至租期到了之后也逾期

df_mob_total_tmp['overdue_days_mob'] = df_mob_total_tmp['overdue_days_mob'].fillna(0)
df_mob_total_tmp['当前累计应付租金'] = df_mob_total_tmp['当前累计应付租金'].fillna(0)
df_mob_total_tmp['当前累计实付租金'] = df_mob_total_tmp['当前累计实付租金'].fillna(0)

tmp_2 = df_mob_total.groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp_2 = tmp_2.rename(columns = {'当前期数': '当前期数'})
tmp_3 = df_mob_total[['order_number','当前期数','当前累计实付租金','当前累计应付租金','overdue_days_mob']].merge(tmp_2, on = ['order_number','当前期数'], how = 'inner')
tmp_2.shape
tmp_3.shape
tmp_3['当前期数'] = tmp_3['当前期数'] + 1
df_mob_total_tmp = df_mob_total_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp['overdue_days_mob_y'] >= 0, df_mob_total_tmp['overdue_days_mob_y'] + 30, df_mob_total_tmp['overdue_days_mob_x'])
df_mob_total_tmp['当前累计应付租金'] = np.where(df_mob_total_tmp['当前累计应付租金_y'] > df_mob_total_tmp['当前累计应付租金_x'], df_mob_total_tmp['当前累计应付租金_y'], df_mob_total_tmp['当前累计应付租金_x'])
df_mob_total_tmp['当前累计实付租金'] = np.where(df_mob_total_tmp['当前累计实付租金_y'] > df_mob_total_tmp['当前累计实付租金_x'], df_mob_total_tmp['当前累计实付租金_y'], df_mob_total_tmp['当前累计实付租金_x'])
df_mob_total_tmp = df_mob_total_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

tmp_3['当前期数'] = tmp_3['当前期数'] + 1
df_mob_total_tmp = df_mob_total_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp['overdue_days_mob_y'] >= 0, df_mob_total_tmp['overdue_days_mob_y'] + 60, df_mob_total_tmp['overdue_days_mob_x'])
df_mob_total_tmp['当前累计应付租金'] = np.where(df_mob_total_tmp['当前累计应付租金_y'] > df_mob_total_tmp['当前累计应付租金_x'], df_mob_total_tmp['当前累计应付租金_y'], df_mob_total_tmp['当前累计应付租金_x'])
df_mob_total_tmp['当前累计实付租金'] = np.where(df_mob_total_tmp['当前累计实付租金_y'] > df_mob_total_tmp['当前累计实付租金_x'], df_mob_total_tmp['当前累计实付租金_y'], df_mob_total_tmp['当前累计实付租金_x'])
df_mob_total_tmp = df_mob_total_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

tmp_3['当前期数'] = tmp_3['当前期数'] + 1
df_mob_total_tmp = df_mob_total_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp['overdue_days_mob_y'] >= 0, df_mob_total_tmp['overdue_days_mob_y'] + 90, df_mob_total_tmp['overdue_days_mob_x'])
df_mob_total_tmp['当前累计应付租金'] = np.where(df_mob_total_tmp['当前累计应付租金_y'] > df_mob_total_tmp['当前累计应付租金_x'], df_mob_total_tmp['当前累计应付租金_y'], df_mob_total_tmp['当前累计应付租金_x'])
df_mob_total_tmp['当前累计实付租金'] = np.where(df_mob_total_tmp['当前累计实付租金_y'] > df_mob_total_tmp['当前累计实付租金_x'], df_mob_total_tmp['当前累计实付租金_y'], df_mob_total_tmp['当前累计实付租金_x'])
df_mob_total_tmp = df_mob_total_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

df_mob_total_tmp.shape
df_mob_total_tmp['search_time'] = df_mob_total_tmp['下单日期'].dt.strftime('%Y-%m')

tmp = df_xz_concat[df_xz_concat.is_relet == 0].groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp = tmp.rename(columns = {'当前期数': '最大期数'})
df_mob_total_tmp = df_mob_total_tmp.merge(tmp, on = 'order_number', how = 'left')
df_mob_total_tmp.shape

df_mob_total_tmp['买断尾款'] = np.where(df_mob_total_tmp.租赁方案 == '租完即送', 0, df_mob_total_tmp.买断尾款)
df_mob_total_tmp['剩余总应收买断金'] = np.where(df_mob_total_tmp.租赁方案 == '租完即送', 0, df_mob_total_tmp.剩余总应收买断金)

df_mob_total_tmp['应付总租金'] = df_mob_total_tmp['应付总租金'].fillna(0)
df_mob_total_tmp['当前累计实付租金'] = df_mob_total_tmp['当前累计实付租金'].fillna(0)
df_mob_total_tmp['剩余总应收租金'] = df_mob_total_tmp['应付总租金'] - df_mob_total_tmp['当前累计实付租金']
df_mob_total_tmp['剩余总应收租金'] = np.where(df_mob_total_tmp['剩余总应收租金'] < 0, 0, df_mob_total_tmp['剩余总应收租金'])
df_mob_total_tmp['剩余总应收租金'] = np.where(df_mob_total_tmp['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_mob_total_tmp['剩余总应收租金'])

df_mob_total_tmp['剩余总应收'] = df_mob_total_tmp['剩余总应收租金'] + df_mob_total_tmp['剩余总应收买断金']
df_mob_total_tmp['剩余总应收'] = np.where(df_mob_total_tmp['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_mob_total_tmp['剩余总应收'])

# A2023112717153031
df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp.订单状态_new.isin(['已买断','已完成','已结束']), 0 , df_mob_total_tmp.overdue_days_mob)
df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp.当前期数 == 1, 0 , df_mob_total_tmp.overdue_days_mob)

# df_mob_total_tmp.shape
# tmp1 = df_xz_concat[['order_number', '当前期数','当前应付租金']]
# tmp1 = tmp1.rename(columns = {'当前应付租金': '当前应付租金_ori'})
# tmp1 = tmp1.drop_duplicates(subset = ['order_number', '当前期数'])
# df_mob_total_tmp = df_mob_total_tmp.merge(tmp1, on = ['order_number', '当前期数'], how = 'left')
# df_mob_total_tmp.shape

df_mob_total_tmp[:1]
# (239652, 22)

(485706, 172)

(485706, 22)

(33120, 2)

(485706, 23)

(465179, 23)

...

(31695, 2)

(31695, 5)

(239651, 23)

(239651, 25)

,order_number,下单日期,租赁方案,当前期数,应付日期,当前应付租金,实付金额,实付日期new,首期应还日期,买断价,...,purchase_amount,归属渠道,is_relet_max,overdue_days_mob,当前累计应付租金,当前累计实付租金,search_time,最大期数,剩余总应收租金,剩余总应收
0,A202207050851373,2022-07-05,租完归还,1.0,2022-07-05,357.95,357.95,2022-07-05,2022-07-05,7530.0,...,5150.0,未知渠道,2.0,0.0,357.95,357.95,2022-07,12.0,3937.45,7172.05


In [ ]:
# （4）统计
conditions_1 = ((df_mob_total_tmp.租赁方案 == '租完归还') & (df_mob_total_tmp.最大期数 == 12) & (df_mob_total_tmp.当前期数 == 1))
conditions_2 = ((df_mob_total_tmp.租赁方案 == '租完即送') & (df_mob_total_tmp.最大期数 == 12) & (df_mob_total_tmp.当前期数 == 1))

conditions_3 = ((df_mob_total_tmp.租赁方案 == '租完归还') & (df_mob_total_tmp.最大期数 == 12) & (~df_mob_total_tmp.订单状态_new.isin(['已买断','已完成','已结束'])))
conditions_5 = ((df_mob_total_tmp.租赁方案 == '租完归还') & (df_mob_total_tmp.最大期数 == 12) & (df_mob_total_tmp.overdue_days_mob > 1 ))
conditions_6 = ((df_mob_total_tmp.租赁方案 == '租完归还') & (df_mob_total_tmp.最大期数 == 12) & (df_mob_total_tmp.overdue_days_mob > 30 ))
conditions_7 = ((df_mob_total_tmp.租赁方案 == '租完归还') & (df_mob_total_tmp.最大期数 == 12) & (df_mob_total_tmp.overdue_days_mob > 60 ))
conditions_8 = ((df_mob_total_tmp.租赁方案 == '租完归还') & (df_mob_total_tmp.最大期数 == 12) & (df_mob_total_tmp.overdue_days_mob > 90 ))

conditions_4 = ((df_mob_total_tmp.租赁方案 == '租完即送') & (df_mob_total_tmp.最大期数 == 12) & (~df_mob_total_tmp.订单状态_new.isin(['已买断','已完成','已结束'])))
conditions_9 = ((df_mob_total_tmp.租赁方案 == '租完即送') & (df_mob_total_tmp.最大期数 == 12) & (df_mob_total_tmp.overdue_days_mob > 1 ))
conditions_10 = ((df_mob_total_tmp.租赁方案 == '租完即送') & (df_mob_total_tmp.最大期数 == 12) & (df_mob_total_tmp.overdue_days_mob > 30 ))
conditions_11 = ((df_mob_total_tmp.租赁方案 == '租完即送') & (df_mob_total_tmp.最大期数 == 12) & (df_mob_total_tmp.overdue_days_mob > 60 ))
conditions_12 = ((df_mob_total_tmp.租赁方案 == '租完即送') & (df_mob_total_tmp.最大期数 == 12) & (df_mob_total_tmp.overdue_days_mob > 90 ))

# df_xz_concat[df_xz_concat.当前期数 == 1].shape
# df_xz_concat_1[df_xz_concat_1.当前期数 == 1].shape
# df_mob_total_tmp[df_mob_total_tmp.当前期数 == 1].shape
# 833
# df_mob_total_tmp[conditions_2 & (df_mob_total_tmp.买断尾款 > 0)].shape
# 26722
# df_mob_total_tmp[conditions_2 & (df_mob_total_tmp.买断尾款 > 0)].买断尾款.sum()

# '应付总租金', '买断尾款'
df_mob_total_tmp[conditions_1].groupby(['search_time']).agg({'order_number':'count', '应付总租金':'sum','买断尾款':'sum','purchase_amount':'sum'}).reset_index()
df_mob_total_tmp[conditions_2].groupby(['search_time']).agg({'order_number':'count', '应付总租金':'sum','买断尾款':'sum','purchase_amount':'sum'}).reset_index()


# pd.pivot_table(df_mob_total_tmp[conditions_3],index='search_time',columns='当前期数',values='order_number',aggfunc='count')
tmp1 = df_mob_total_tmp[conditions_3].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收租金':'sum','剩余总应收买断金':'sum'}).reset_index()
tmp1 = tmp1.rename(columns = {'order_number': '表现月末未结清客户数','剩余总应收租金': '截至表现月份剩余总应收（租金）','剩余总应收买断金':'截至表现月份剩余总应收（买断金）'})
tmp2 = df_mob_total_tmp[conditions_5].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收': 'sum'}).reset_index()
tmp2 = tmp2.rename(columns = {'order_number': '表现月末逾期客户数dpd1+','剩余总应收': '表现月末逾期dpd1+剩余总应收（应收租金+买断金）'})
tmp3 = df_mob_total_tmp[conditions_6].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收': 'sum'}).reset_index()
tmp3 = tmp3.rename(columns = {'order_number': '表现月末逾期客户数dpd30+','剩余总应收': '表现月末逾期dpd30+剩余总应收（应收租金+买断金）'})
tmp4 = df_mob_total_tmp[conditions_7].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收': 'sum'}).reset_index()
tmp4 = tmp4.rename(columns = {'order_number': '表现月末逾期客户数dpd60+','剩余总应收': '表现月末逾期dpd60+剩余总应收（应收租金+买断金）'})
tmp5 = df_mob_total_tmp[conditions_8].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收': 'sum'}).reset_index()
tmp5 = tmp5.rename(columns = {'order_number': '表现月末逾期客户数dpd90+','剩余总应收': '表现月末逾期dpd90+剩余总应收（应收租金+买断金）'})
tmp = tmp1.merge(tmp2, on = ['search_time','当前期数'], how = 'outer')
tmp = tmp.merge(tmp3, on = ['search_time','当前期数'], how = 'outer')
tmp = tmp.merge(tmp4, on = ['search_time','当前期数'], how = 'outer')
tmp = tmp.merge(tmp5, on = ['search_time','当前期数'], how = 'outer')
tmp.shape
var = ['表现月末未结清客户数','截至表现月份剩余总应收（租金）','截至表现月份剩余总应收（买断金）',	'表现月末逾期客户数dpd1+','表现月末逾期客户数dpd30+','表现月末逾期客户数dpd60+',	
'表现月末逾期客户数dpd90+',	'表现月末逾期dpd1+剩余总应收（应收租金+买断金）','表现月末逾期dpd30+剩余总应收（应收租金+买断金）','表现月末逾期dpd60+剩余总应收（应收租金+买断金）','表现月末逾期dpd90+剩余总应收（应收租金+买断金）']
tmp = tmp[['search_time','当前期数'] + var]
tmp = tmp.sort_values(by = ['search_time','当前期数'])
tmp.to_excel('tmp.xlsx', index = False)



tmp1 = df_mob_total_tmp[conditions_4].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收租金':'sum','剩余总应收买断金':'sum'}).reset_index()
tmp1 = tmp1.rename(columns = {'order_number': '表现月末未结清客户数','剩余总应收租金': '截至表现月份剩余总应收（租金）','剩余总应收买断金':'截至表现月份剩余总应收（买断金）'})
tmp2 = df_mob_total_tmp[conditions_9].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收': 'sum'}).reset_index()
tmp2 = tmp2.rename(columns = {'order_number': '表现月末逾期客户数dpd1+','剩余总应收': '表现月末逾期dpd1+剩余总应收（应收租金+买断金）'})
tmp3 = df_mob_total_tmp[conditions_10].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收': 'sum'}).reset_index()
tmp3 = tmp3.rename(columns = {'order_number': '表现月末逾期客户数dpd30+','剩余总应收': '表现月末逾期dpd30+剩余总应收（应收租金+买断金）'})
tmp4 = df_mob_total_tmp[conditions_11].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收': 'sum'}).reset_index()
tmp4 = tmp4.rename(columns = {'order_number': '表现月末逾期客户数dpd60+','剩余总应收': '表现月末逾期dpd60+剩余总应收（应收租金+买断金）'})
tmp5 = df_mob_total_tmp[conditions_12].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收': 'sum'}).reset_index()
tmp5 = tmp5.rename(columns = {'order_number': '表现月末逾期客户数dpd90+','剩余总应收': '表现月末逾期dpd90+剩余总应收（应收租金+买断金）'})
tmp = tmp1.merge(tmp2, on = ['search_time','当前期数'], how = 'outer')
tmp = tmp.merge(tmp3, on = ['search_time','当前期数'], how = 'outer')
tmp = tmp.merge(tmp4, on = ['search_time','当前期数'], how = 'outer')
tmp = tmp.merge(tmp5, on = ['search_time','当前期数'], how = 'outer')
var = ['表现月末未结清客户数','截至表现月份剩余总应收（租金）','截至表现月份剩余总应收（买断金）',	'表现月末逾期客户数dpd1+','表现月末逾期客户数dpd30+','表现月末逾期客户数dpd60+',	
'表现月末逾期客户数dpd90+',	'表现月末逾期dpd1+剩余总应收（应收租金+买断金）','表现月末逾期dpd30+剩余总应收（应收租金+买断金）','表现月末逾期dpd60+剩余总应收（应收租金+买断金）','表现月末逾期dpd90+剩余总应收（应收租金+买断金）']
tmp = tmp[['search_time','当前期数'] + var]
tmp = tmp.sort_values(by = ['search_time','当前期数'])
tmp.to_excel('tmp1.xlsx', index = False)

# 区分渠道 搜索和租物的基本都是12期的
# 导数给鑫仲统计 客户地区 金额分布
# df_mob_total_tmp[(df_mob_total_tmp.当前期数 == 1)][['order_number','search_time', '应付总租金','买断尾款']].to_excel('tmp11.xlsx', index = False)

# 还款计划表 异常租赁场景（提前买断+提前退租）
var = ['当前应付租金','实付金额','当月已收买断金','提前状态']
conditions_13 = (df_mob_total_tmp.最大期数 == 12) & (df_mob_total_tmp.当前期数 <= 12)
tmp1 = df_mob_total_tmp[conditions_13].groupby(['search_time','当前期数']).agg({'当前应付租金':'sum', '实付金额':'sum','当月已收买断金':'sum'}).reset_index()
tmp2 = df_mob_total_tmp[conditions_13 & (df_mob_total_tmp.提前状态.isin(['提前买断']))].groupby(['search_time','当前期数']).agg({'当月已收买断金':'sum'}).reset_index()
df_mob_total_tmp[conditions_13 & (df_mob_total_tmp.提前状态.isin(['提前买断']))].groupby(['search_time']).agg({'order_number':'count','当月已收买断金':'sum'}).reset_index()

tmp2 = tmp2.rename(columns = {'当月已收买断金' : '提前结清金额'})
tmp = tmp1.merge(tmp2, on = ['search_time','当前期数'], how = 'outer')
tmp = tmp.sort_values(by = ['search_time','当前期数'])
# 提前买断、提前完成

# 当前应付租金
tmp3 = df_mob_total_tmp[['order_number','search_time']].drop_duplicates(subset = ['order_number'])
tmp3 = tmp3.merge(df_xz_concat[['order_number','当前期数','当前应付租金','实付金额']], on = 'order_number', how = 'inner')
tmp3 = tmp3.rename(columns = {'当前应付租金' : '当前应付租金_ori','实付金额':'实付金额_ori'})
tmp3 = tmp3[tmp3.当前期数 <= 12]
tmp4 = tmp3.groupby(['search_time','当前期数']).agg({'当前应付租金_ori':'sum','实付金额_ori':'sum'}).reset_index()
tmp5 = tmp4.merge(tmp, on = ['search_time','当前期数'], how = 'outer')
tmp5.to_excel('tmp12.xlsx', index = False)
# 提前支付租金
# tmp5[tmp5.search_time == '2024-08']
# tmp3[(tmp3.search_time == '2025-01')&(tmp3.当前期数>2)&(tmp3.实付金额_ori>0)&(tmp3.当前期数<11)]
# tmp3[tmp3.order_number == 'A20250102113617265']

# 客户来源--区分渠道 搜索和租物的基本都是12期的
df_mob_total_tmp[(df_mob_total_tmp.当前期数 == 1)].groupby(['归属渠道','search_time']).agg({'应付总租金':'sum','买断尾款':'sum','order_number':'count'}).reset_index().to_excel('tmp13.xlsx', index = False)


,search_time,order_number,应付总租金,买断尾款,purchase_amount
0,2022-07,16,75361.98,62877.02,85451.00
1,2022-08,47,235660.21,208449.79,300846.00
2,2022-09,43,221262.22,168652.78,268822.75
3,2022-10,41,192968.21,144999.79,249543.09
4,2022-11,40,220898.00,181722.00,313698.81
5,2022-12,58,367806.20,279475.00,494830.00
6,2023-01,178,898620.00,766642.52,1167130.00
7,2023-02,302,1523754.66,1408651.08,1988861.82
8,2023-03,269,1289629.29,1326443.71,1778019.00
9,2023-04,543,2594588.54,2993672.46,3701872.50


,search_time,order_number,应付总租金,买断尾款,purchase_amount
0,2022-07,6,57452.10,0.0,39954.00
1,2022-08,9,83607.08,0.0,57605.00
2,2022-09,1,12739.00,0.0,9160.00
3,2022-11,3,37437.00,0.0,30560.00
4,2022-12,8,93072.00,0.0,71446.59
5,2023-01,43,495304.10,0.0,366329.00
6,2023-02,30,275087.15,0.0,192691.00
7,2023-03,47,444628.38,0.0,305002.00
8,2023-04,91,923200.37,0.0,615779.00
9,2023-05,79,848911.11,0.0,527809.00


(360, 13)

,search_time,order_number,当月已收买断金
0,2022-07,1,6287.60
1,2022-08,4,3040.42
2,2022-09,4,15879.01
3,2022-10,2,9816.50
4,2022-11,6,24262.90
5,2022-12,11,42900.30
6,2023-01,29,75085.95
7,2023-02,46,145009.77
8,2023-03,41,102603.91
9,2023-04,83,203322.39


In [92]:
# （5）统计 
# 另外一个资方的数据
# （5-1）
df_mob_total_tmp['当前累计已收买断金'] = df_mob_total_tmp['当前累计已收买断金'].fillna(0)
df_mob_total_tmp['当前累计实付租金'] = df_mob_total_tmp['当前累计实付租金'].fillna(0)
df_mob_total_tmp['当前累计实付'] = df_mob_total_tmp['当前累计已收买断金'] + df_mob_total_tmp['当前累计实付租金']
df_mob_total_tmp['总应收'] = df_mob_total_tmp['应付总租金'] + df_mob_total_tmp['买断尾款']

conditions_1 = (df_mob_total_tmp.当前期数 == 1)
conditions_2 = (df_mob_total_tmp.overdue_days_mob > 30 )
conditions_3 = (df_mob_total_tmp.overdue_days_mob > 90 )

df_mob_total_tmp[conditions_1].groupby(['search_time']).agg({'order_number':'count', '应付总租金':'sum','买断尾款':'sum','purchase_amount':'sum'}).reset_index()

tmp3 = df_mob_total_tmp[conditions_2].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收': 'sum','当前累计实付':'sum','总应收':'sum'}).reset_index()
tmp3 = tmp3.rename(columns = {'order_number': '表现月末逾期客户数dpd30+','剩余总应收': '表现月末逾期dpd30+剩余总应收（应收租金+买断金）','当前累计实付':'表现月末逾期dpd30+累计实付','总应收':'表现月末逾期dpd30+总应收'})
tmp5 = df_mob_total_tmp[conditions_3].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收': 'sum','当前累计实付':'sum','总应收':'sum'}).reset_index()
tmp5 = tmp5.rename(columns = {'order_number': '表现月末逾期客户数dpd90+','剩余总应收': '表现月末逾期dpd90+剩余总应收（应收租金+买断金）','当前累计实付':'表现月末逾期dpd90+累计实付','总应收':'表现月末逾期dpd90+总应收'})

tmp = tmp3.merge(tmp5, on = ['search_time','当前期数'], how = 'outer')
tmp.shape
# var = ['表现月末逾期客户数dpd30+','表现月末逾期客户数dpd90+','表现月末逾期dpd30+剩余总应收（应收租金+买断金）','表现月末逾期dpd90+剩余总应收（应收租金+买断金）','表现月末逾期dpd30+累计实付','表现月末逾期dpd90+累计实付']
# tmp = tmp[['search_time','当前期数'] + var]
# tmp = tmp.sort_values(by = ['search_time','当前期数'])
# tmp.to_excel('tmp0.xlsx', index = False)

tmp.pivot(index="search_time", columns="当前期数", values="表现月末逾期客户数dpd30+")
tmp.pivot(index="search_time", columns="当前期数", values="表现月末逾期客户数dpd90+")

tmp.pivot(index="search_time", columns="当前期数", values="表现月末逾期dpd30+剩余总应收（应收租金+买断金）")
tmp.pivot(index="search_time", columns="当前期数", values="表现月末逾期dpd90+剩余总应收（应收租金+买断金）")
# 分子用 表现月末逾期dpd30+剩余总应收（应收租金+买断金），不用 总应收-累计实付
# tmp.pivot(index="search_time", columns="当前期数", values="表现月末逾期dpd30+累计实付")
# tmp.pivot(index="search_time", columns="当前期数", values="表现月末逾期dpd90+累计实付")

# tmp.pivot(index="search_time", columns="当前期数", values="表现月末逾期dpd30+总应收")
# tmp.pivot(index="search_time", columns="当前期数", values="表现月末逾期dpd90+总应收")



,search_time,order_number,应付总租金,买断尾款,purchase_amount
0,2022-07,22,132814.08,62877.02,125405.00
1,2022-08,56,319267.29,208449.79,358451.00
2,2022-09,44,234001.22,168652.78,277982.75
3,2022-10,41,192968.21,144999.79,249543.09
4,2022-11,43,258335.00,181722.00,344258.81
5,2022-12,66,460878.20,279475.00,566276.59
6,2023-01,221,1393924.10,766642.52,1533459.00
7,2023-02,332,1798841.81,1408651.08,2181552.82
8,2023-03,316,1734257.67,1326443.71,2083021.00
9,2023-04,634,3517788.91,2993672.46,4317651.50


(299, 10)

当前期数,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0
search_time,,,,,,,,,,,,,
2022-07,2.0,6.0,6.0,6.0,7.0,8.0,8.0,8.0,8.0,8.0,8.0,11.0,11.0
2022-08,7.0,10.0,12.0,15.0,16.0,18.0,21.0,20.0,23.0,21.0,21.0,33.0,33.0
2022-09,3.0,5.0,7.0,11.0,11.0,11.0,10.0,11.0,10.0,10.0,10.0,20.0,19.0
2022-10,2.0,2.0,4.0,5.0,7.0,5.0,7.0,7.0,8.0,8.0,8.0,17.0,16.0
2022-11,4.0,4.0,5.0,5.0,6.0,7.0,8.0,8.0,8.0,8.0,8.0,14.0,14.0
2022-12,3.0,4.0,10.0,11.0,11.0,14.0,14.0,17.0,17.0,15.0,15.0,21.0,21.0
2023-01,12.0,21.0,27.0,32.0,45.0,46.0,51.0,60.0,61.0,65.0,65.0,83.0,82.0
2023-02,23.0,34.0,43.0,55.0,57.0,65.0,73.0,74.0,78.0,76.0,77.0,114.0,108.0
2023-03,22.0,35.0,47.0,55.0,66.0,66.0,74.0,85.0,89.0,88.0,88.0,116.0,114.0


当前期数,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0
search_time,,,,,,,,,,,,,
2022-07,NaN,NaN,2.0,6.0,6.0,6.0,7.0,8.0,8.0,8.0,8.0,8.0,8.0
2022-08,NaN,NaN,7.0,10.0,12.0,14.0,16.0,18.0,19.0,18.0,21.0,21.0,21.0
2022-09,NaN,NaN,3.0,5.0,6.0,10.0,10.0,10.0,9.0,10.0,10.0,10.0,10.0
2022-10,NaN,NaN,2.0,2.0,4.0,3.0,5.0,5.0,6.0,7.0,7.0,8.0,8.0
2022-11,NaN,NaN,3.0,4.0,5.0,4.0,4.0,5.0,6.0,8.0,8.0,8.0,8.0
2022-12,NaN,NaN,1.0,3.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,15.0,15.0
2023-01,NaN,NaN,8.0,17.0,25.0,30.0,38.0,42.0,49.0,55.0,60.0,65.0,65.0
2023-02,NaN,NaN,18.0,33.0,41.0,50.0,54.0,59.0,64.0,66.0,72.0,76.0,76.0
2023-03,NaN,NaN,22.0,34.0,43.0,49.0,58.0,61.0,71.0,79.0,83.0,87.0,87.0


当前期数,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0
search_time,,,,,,,,,,,,,
2022-07,17171.94,53553.40,53553.40,53553.40,58709.32,62440.08,62440.08,62440.08,62440.08,62440.08,62440.08,69019.34,69019.34
2022-08,65283.97,96366.94,116317.54,135016.69,140377.93,151932.86,161810.12,155965.10,168757.67,162693.90,162693.90,215369.45,215369.45
2022-09,36927.89,52876.79,63907.19,87206.78,89608.97,89608.97,81431.18,85069.58,74356.58,74356.58,74356.58,115620.61,110622.82
2022-10,18639.80,18231.40,27727.40,35956.40,52770.60,40463.40,54662.80,54334.50,60789.70,59810.50,59810.50,86066.00,85097.00
2022-11,38151.90,38040.00,45118.80,45118.80,51079.40,59255.80,63697.50,63046.80,62514.20,62514.20,62514.20,97013.20,97013.20
2022-12,31560.40,41632.60,92479.50,99216.20,98743.10,119525.10,122969.60,139295.40,137208.90,119404.40,119404.40,147762.20,147762.20
2023-01,99268.60,157112.00,202262.45,233263.40,325277.85,318159.30,343409.65,384783.45,388389.05,400096.00,400096.00,479899.35,474102.05
2023-02,231693.93,345190.65,414785.44,506187.88,517911.54,562165.47,604793.18,598909.06,611774.76,597213.49,602406.63,777403.34,753446.14
2023-03,213156.79,305672.54,404971.18,477003.27,552474.98,547582.79,599836.27,641578.12,661083.31,649648.79,649648.79,779789.57,769079.67


当前期数,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0
search_time,,,,,,,,,,,,,
2022-07,NaN,NaN,17171.94,53553.40,53553.40,53553.40,58709.32,62440.08,62440.08,62440.08,62440.08,62440.08,62440.08
2022-08,NaN,NaN,65283.97,96366.94,116317.54,130537.71,141618.20,151180.45,154022.78,148616.85,162693.90,162693.90,162693.90
2022-09,NaN,NaN,36927.89,52876.79,58504.79,82067.18,81431.18,80981.18,70268.18,74356.58,74356.58,74356.58,74356.58
2022-10,NaN,NaN,18231.40,18231.40,27727.40,24385.00,40463.40,40463.40,47432.40,53742.90,53742.90,59810.50,59810.50
2022-11,NaN,NaN,30165.90,38040.00,45118.80,38040.00,38040.00,43327.50,50880.30,62514.20,62514.20,62514.20,62514.20
2022-12,NaN,NaN,11811.50,31184.60,83044.60,91737.60,98743.10,106789.60,114474.60,113606.10,119404.40,119404.40,119404.40
2023-01,NaN,NaN,66115.75,122217.95,182044.50,217037.20,268119.45,289902.75,326709.85,359846.05,384908.45,400096.00,400096.00
2023-02,NaN,NaN,183037.64,334684.85,393729.84,463277.13,480456.05,509928.07,542770.96,543746.43,574512.31,597213.49,594059.01
2023-03,NaN,NaN,205798.68,300130.10,374212.74,431885.28,495463.39,520640.22,579160.77,605268.55,626745.74,646429.41,646429.41


In [94]:
# （6）联合运营数据
# df_xz_concat.columns
var = ['order_number','下单日期','租赁方案','当前期数','应付日期',
       '应付总租金','买断尾款','当前应付租金','overdue_days_mob','剩余总应收','每期最大逾期天数','实付金额']
# df_mob_total_tmp[df_mob_total_tmp.order_number == 'A202207171820512'][var]
# df_xz_concat[df_xz_concat.order_number == 'A202207171820512'][['order_number','当前期数','逾期天数','实付日期new','order_finish_date','finish_date_new','now_date']]

tmp1 = df_xz_concat[['order_number','finish_date_new','now_date']].drop_duplicates(subset = 'order_number')
tmp1['订单完成日期'] = np.where(pd.notnull(tmp1.finish_date_new), tmp1.finish_date_new, tmp1.now_date)
df_mob_total_tmp_1 = df_mob_total_tmp[pd.isnull(df_mob_total_tmp.当前应付租金)].merge(tmp1[['order_number','订单完成日期']], on = 'order_number', how = 'inner')
df_mob_total_tmp_2 = df_mob_total_tmp[pd.notnull(df_mob_total_tmp.当前应付租金)].merge(df_xz_concat[['order_number','当前期数','逾期天数']], on = ['order_number','当前期数'], how = 'inner')

# tmp1[tmp1.order_number == 'A2022120222510415']
df_mob_total_tmp.shape
df_mob_total_tmp_1.shape
df_mob_total_tmp_2.shape

df_mob_total_tmp_1['每期最大逾期天数'] = (pd.to_datetime(df_mob_total_tmp_1['订单完成日期']) - pd.to_datetime(df_mob_total_tmp_1['应付日期'])).dt.days
df_mob_total_tmp_2 = df_mob_total_tmp_2.rename(columns = {'逾期天数': '每期最大逾期天数'})
df_mob_total_tmp_1 = df_mob_total_tmp_1.drop(columns = ['订单完成日期'])

df_mob_total_tmp_3 = pd.concat([df_mob_total_tmp_2, df_mob_total_tmp_1])
df_mob_total_tmp_3.shape
df_mob_total_tmp_3[df_mob_total_tmp_3.order_number == 'A202207181530165'][var]

(239651, 27)

(11342, 28)

(228309, 28)

(239651, 28)

,order_number,下单日期,租赁方案,当前期数,应付日期,应付总租金,买断尾款,当前应付租金,overdue_days_mob,剩余总应收,每期最大逾期天数,实付金额
67,A202207181530165,2022-07-18,租完归还,1.0,2022-07-18,3519.24,2197.76,146.64,0.0,4957.16,0,146.64
68,A202207181530165,2022-07-18,租完归还,2.0,2022-08-18,3519.24,2197.76,306.60,0.0,4957.16,0,306.60
69,A202207181530165,2022-07-18,租完归还,3.0,2022-09-18,3519.24,2197.76,306.60,0.0,4343.96,0,306.60
70,A202207181530165,2022-07-18,租完归还,4.0,2022-10-18,3519.24,2197.76,306.60,0.0,4343.96,0,306.60
71,A202207181530165,2022-07-18,租完归还,5.0,2022-11-18,3519.24,2197.76,306.60,12.0,4343.96,17,306.60
72,A202207181530165,2022-07-18,租完归还,6.0,2022-12-18,3519.24,2197.76,306.60,13.0,4037.36,23,306.60
73,A202207181530165,2022-07-18,租完归还,7.0,2023-01-18,3519.24,2197.76,306.60,13.0,3730.76,767,0.00
74,A202207181530165,2022-07-18,租完归还,8.0,2023-02-18,3519.24,2197.76,306.60,41.0,3730.76,736,0.00
75,A202207181530165,2022-07-18,租完归还,9.0,2023-03-18,3519.24,2197.76,306.60,72.0,3730.76,708,0.00
76,A202207181530165,2022-07-18,租完归还,10.0,2023-04-18,3519.24,2197.76,306.60,102.0,3730.76,677,0.00


In [ ]:
# tmp3 = df_mob_total_tmp[['order_number']].drop_duplicates(subset = ['order_number'])
# tmp3.shape
# tmp3 = df_mob_total_tmp[['order_number','search_time']].drop_duplicates(subset = ['order_number'])
# tmp3 = tmp3.merge(df_xz_concat[['order_number','当前期数','当前应付租金']], on = 'order_number', how = 'inner')
# tmp3 = tmp3.rename(columns = {'当前应付租金' : '当前应付租金_ori'})
# tmp3[:2]
# tmp3 = tmp3[tmp3.当前期数 <= 12]
# tmp4 = tmp3.groupby(['search_time','当前期数']).agg({'当前应付租金_ori':'sum'}).reset_index()
# tmp5 = tmp4.merge(tmp, on = ['search_time','当前期数'], how = 'outer')
# tmp.to_excel('tmp12.xlsx', index = False)
# df_mob_total_tmp[df_mob_total_tmp.order_number == 'A202207171820512'][['当前期数','overdue_days_mob','下单日期']]
# df_mob_total_tmp.提前状态.value_counts()
# df_mob_total_tmp[(df_mob_total_tmp.search_time == '2022-08')&(df_mob_total_tmp.当前期数 == 1)].租赁方案.value_counts()
# df_mob_total_tmp[(df_mob_total_tmp.search_time == '2022-08')&(df_mob_total_tmp.当前期数 == 1)][['order_number','租赁方案','当前期数','overdue_days_mob','下单日期','订单状态_new','最大期数']].sort_values(by = '租赁方案')

# conditions_5 = ((df_mob_total_tmp.租赁方案 == '租完归还') & (df_mob_total_tmp.最大期数 == 12) & (df_mob_total_tmp.overdue_days_mob > 1 ))
# conditions_6 = ((df_mob_total_tmp.租赁方案 == '租完归还') & (df_mob_total_tmp.最大期数 == 12) & (df_mob_total_tmp.overdue_days_mob > 30 ))
# conditions_7 = ((df_mob_total_tmp.租赁方案 == '租完归还') & (df_mob_total_tmp.最大期数 == 12) & (df_mob_total_tmp.overdue_days_mob > 60 ))
# conditions_8 = ((df_mob_total_tmp.租赁方案 == '租完归还') & (df_mob_total_tmp.最大期数 == 12) & (df_mob_total_tmp.overdue_days_mob > 90 ))

# df_mob_total_tmp[conditions_6 & (df_mob_total_tmp.search_time == '2022-07')&(df_mob_total_tmp.当前期数 == 14)][['order_number','租赁方案','当前期数','overdue_days_mob','下单日期','订单状态_new','最大期数']]


# df_mob_total[df_mob_total.order_number == 'A202207151707153']
# var3 = ['order_id','当前期数','应付日期','当前应付租金','实付日期new','实付金额','finish_

,order_number,search_time,当前期数,当前应付租金_ori
0,A202207050851373,2022-07,1.0,357.95
1,A202207050851373,2022-07,2.0,357.95


In [320]:
# df_mob_total[df_mob_total.order_number == 'A202207151707153']
# var3 = ['order_id','当前期数','应付日期','当前应付租金','实付日期new','实付金额','finish_date_new','订单状态_new','首期应还日期','买断价','应付总租金','买断尾款','订单状态_new','real_pay_money_new','剩余总应收买断金']
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024060200290566'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024063001194047'])][var3]
# 订单已完成
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024031113200319'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202308262317036'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207050851373'])][var3]
# # 在15期之前买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024020317504938'])][var3]
# 在15期之后买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207171820512'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207181530165'])][var3] --2个12期
# 已买断但 买断尾款	!= real_pay_money_new  可能使用优惠劵
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207151707153'])][var3]

df_mob_total_tmp[['当前累计实付租金','实付金额']].tail()

,当前累计实付租金,实付金额
250313,719.04,1.00
250314,1082.94,1082.94
250315,455.93,1.00
250316,455.93,1.00
250317,744.53,744.53


# 6.all

In [93]:
# （1）首逾统计
# sheet '首逾统计'-12期
# 放款月份	当月放款客户数	当月放款总应收（租金）	当月放款总实收（租金）	
# 第一期1+逾期剩余应收租金	第一期逾期1+客户数	第一期3+逾期剩余应收租金	第一期逾期3+客户数	第一期7+逾期剩余应收租金	第一期逾期7+客户数
conditions_1 = ((df_mob_total_tmp.最大期数 == 12) & (df_mob_total_tmp.当前期数 == 1))
conditions_2 = ((df_mob_total_tmp.最大期数 == 12) & (df_mob_total_tmp.当前期数 == 2))
df_mob_total_tmp[conditions_1].groupby(['search_time']).agg({'order_number':'count', '应付总租金':'sum','当前累计实付租金':'sum'}).reset_index()
# df_mob_total_tmp[:1]
# df_mob_total_tmp.columns

df_fstq = df_xz_concat[["order_number","当前期数","逾期天数"]].pivot(index="order_number", columns="当前期数", values="逾期天数")
df_fstq = df_fstq.reset_index()
df_fstq.shape
df_fstq['FPD'] = df_fstq[2]
df_fstq = df_fstq[['order_number', 'FPD']]
df_fstq.shape

temp = df_mob_total_tmp.merge(df_fstq, on = 'order_number', how = 'left')
temp[conditions_2 & (temp.FPD > 1)].groupby(['search_time']).agg({'剩余总应收租金':'sum','order_number':'count'}).reset_index()
temp[conditions_2 & (temp.FPD > 3)].groupby(['search_time']).agg({'剩余总应收租金':'sum','order_number':'count'}).reset_index()
temp[conditions_2 & (temp.FPD > 7)].groupby(['search_time']).agg({'剩余总应收租金':'sum','order_number':'count'}).reset_index()

,search_time,order_number,应付总租金,当前累计实付租金
0,2022-07,22,132814.08,21587.16
1,2022-08,56,319267.29,72083.99
2,2022-09,44,234001.22,59957.01
3,2022-10,41,192968.21,36790.37
4,2022-11,43,258335.00,56603.60
5,2022-12,66,460878.20,90884.71
6,2023-01,221,1393924.10,242908.10
7,2023-02,332,1798841.81,329134.23
8,2023-03,316,1734257.67,278889.76
9,2023-04,634,3517788.91,409761.55


(32818, 37)

(32818, 2)

,search_time,剩余总应收租金,order_number
0,2022-07,14151.88,3
1,2022-08,64531.63,15
2,2022-09,42838.29,9
3,2022-10,18677.00,5
4,2022-11,56661.70,10
5,2022-12,55667.70,10
6,2023-01,212156.00,36
7,2023-02,253333.75,58
8,2023-03,169031.57,36
9,2023-04,337858.09,66


,search_time,剩余总应收租金,order_number
0,2022-07,14151.88,3
1,2022-08,54894.13,12
2,2022-09,33298.29,7
3,2022-10,12617.00,3
4,2022-11,30277.60,6
5,2022-12,50475.60,9
6,2023-01,133455.90,25
7,2023-02,198557.28,45
8,2023-03,160995.97,34
9,2023-04,302810.72,56


,search_time,剩余总应收租金,order_number
0,2022-07,14151.88,3
1,2022-08,38684.97,9
2,2022-09,29223.09,6
3,2022-10,12617.00,3
4,2022-11,23811.90,4
5,2022-12,49760.40,8
6,2023-01,83907.70,16
7,2023-02,160549.93,36
8,2023-03,141868.95,29
9,2023-04,171613.30,33


In [94]:
# （2-1）月末统计
# sheet '逾期余额统计' 月末情况
# 资方数据加工
# 截至统计月末剩余应收租金的客户数	截至统计月末剩余总应收租金	截至统计月末剩余总应收买断金
# 20250211
df_xz_concat_1 = df_xz_concat.copy()
df_xz_concat_1.loc[:,"租赁方案"]=np.where(df_xz_concat_1["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df_xz_concat_1['finish_date_new'] = pd.to_datetime(df_xz_concat_1['finish_date_new'])

tmp5 = df_xz_concat_1.groupby(['order_number']).agg({'应付日期':'max'}).reset_index()
tmp5 = tmp5.rename(columns = {'应付日期': '最大应还日期'})
df_xz_concat_1 = df_xz_concat_1.merge(tmp5, on = 'order_number', how = 'left')
df_xz_concat_1.shape

# （2）提取买断金等信息，区分续租和非续租 买断价，续租的买断价包含了首租的买断价
# 续租买断尾款 为续租总租金，续租买断尾款其实为0
# 首租订单，首租买断尾款为续租总租金，实际中，续租总租金 大于 首租买断尾款
# all_money：续租的已完成的话，为首租+续租总租金；续租的未完成的话，为首租的总租  'A202308262317036','A202207050851373'
sql = '''
SELECT tprm.order_id, tprm.order_number, tprm.order_create_time 下单日期
    ,tod.actual_money 首租买断价
    ,tod.new_actual_money 续租买断价
    , tprm.all_deposit 总押金
    , tprm.total_receivable 总应收
    , tprm.total_received 总已收
    # , om.all_money 首租总租金
    # , (tod.new_actual_money-om.all_money) 续租总租金
    # , (tod.actual_money-om.all_money) 首租买断尾款
    , tprm.purchase_amount 采购金额
    , om.has_actual 是否买断
    , om.relet_status 是否续租
    , tprm.paid_periods 已还期数
    , tod.early_completion_mark 是否提前完成
    , tmu.id_card_num 身份证号
    , tol.delivery_province_name 地区
    , pa.type 活动类型
    , pa.type, too.money 滞纳金, tprm.liquidated_damages 违约金
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_digua_business.t_platform_activity pa on om.activity_id=pa.id
left join db_digua_business.t_member_user tmu on om.user_id=tmu.id
left join db_digua_business.t_order_details tod on tod.order_id=tprm.order_id
left join db_digua_business.t_order_logistics tol on tol.order_id=tprm.order_id
left join (SELECT t.order_id,sum(case when t.capital_type =6 and t.`status`=2 then t.money end) as other_money 
,sum(case when t.capital_type != 5 AND  t.`status` = 2 then t.money end) as money
from db_digua_business.t_order_other t GROUP BY 1 ) too on too.order_id=om.id
'''

df_temp_1 = query(sql)
df_temp_1.shape
df_temp_1 = df_temp_1.drop_duplicates(subset = ['order_number'])
df_temp_1.shape

df_temp_1_1 = df_xz_concat_1[(df_xz_concat_1.is_relet == 0)].merge(df_temp_1[['order_number','首租买断价']], on = 'order_number', how = 'left')
df_temp_1_1 = df_temp_1_1.rename(columns = {'首租买断价': '买断价'})
df_temp_1_1.shape

df_temp_1_2 = df_xz_concat_1[(df_xz_concat_1.is_relet != 0)].merge(df_temp_1[['order_number','续租买断价']], on = 'order_number', how = 'left')
df_temp_1_2 = df_temp_1_2.rename(columns = {'续租买断价': '买断价'})
df_temp_1_2.shape

df_xz_concat_1 = pd.concat([df_temp_1_1, df_temp_1_2])

# （3）加工应付总租金，区分续租和非续租 应付总租金
df_temp_2 = df_xz_concat_1[(df_xz_concat_1.is_relet == 0)].groupby(['order_number']).agg({'当前应付租金':'sum'}).reset_index()
df_temp_2 = df_temp_2.rename(columns = {'当前应付租金': '应付总租金'})
df_temp_2.shape

df_temp_3 = df_xz_concat_1[(df_xz_concat_1.is_relet != 0)].groupby(['order_number']).agg({'当前应付租金':'sum'}).reset_index()
df_temp_3 = df_temp_3.rename(columns = {'当前应付租金': '应付总租金'})
df_temp_3.shape

df_temp_4 = df_xz_concat_1[(df_xz_concat_1.is_relet == 0)].merge(df_temp_2, on = 'order_number', how = 'left')
df_temp_4.shape

df_temp_5 = df_xz_concat_1[(df_xz_concat_1.is_relet != 0)].merge(df_temp_3, on = 'order_number', how = 'left')
df_temp_5.shape

df_xz_concat_1 = pd.concat([df_temp_4, df_temp_5])


#（4）买断金 续租的买断尾款为 0 
df_xz_concat_1['买断尾款'] = df_xz_concat_1['买断价'] - df_xz_concat_1['应付总租金']
df_xz_concat_1['买断尾款'] = np.where(df_xz_concat_1.is_relet == 0, df_xz_concat_1['买断尾款'], 0)
df_xz_concat_1.shape

del df_temp_1_1
del df_temp_1_2
del df_temp_1
del df_temp_2
del df_temp_3
del df_temp_4
del df_temp_5

# （5）补数据  为了关联买断数据
# 当前期数、应付日期 改变，其他属性不变
tmp1 = df_xz_concat_1.groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
var = ['order_number','order_id','is_relet','当前期数', '下单日期', '应付日期', '买断价', '应付总租金', '买断尾款','租赁方案','rembursement_status','finish_date_new']
tmp1.shape

# tmp2 = tmp1.merge(df_xz_concat_1[var], on = ['order_number', '当前期数'], how = 'inner')
# tmp2['当前期数'] = tmp2['当前期数'] + 1
# tmp2['应付日期'] = tmp2['应付日期'] + pd.DateOffset(months = 1)

# tmp3 = tmp1.merge(df_xz_concat_1[var], on = ['order_number', '当前期数'], how = 'inner')
# tmp3['当前期数'] = tmp3['当前期数'] + 2
# tmp3['应付日期'] = tmp3['应付日期'] + pd.DateOffset(months = 2)

# tmp4 = tmp1.merge(df_xz_concat_1[var], on = ['order_number', '当前期数'], how = 'inner')
# tmp4['当前期数'] = tmp4['当前期数'] + 3
# tmp4['应付日期'] = tmp4['应付日期'] + pd.DateOffset(months = 3)

# tmp5 = pd.concat([tmp2, tmp3, tmp4])
# tmp5.shape

for num in range(1,28):
    tmp2 = tmp1.merge(df_xz_concat_1[var], on = ['order_number', '当前期数'], how = 'inner')
    tmp2['当前期数'] = tmp2['当前期数'] + num
    tmp2['应付日期'] = tmp2['应付日期'] + pd.DateOffset(months = num)
    df_xz_concat_1 = pd.concat([df_xz_concat_1, tmp2])

df_xz_concat_1.shape

# （6）订单状态定位到 当月
df_xz_concat_1['date_tmp_1'] = df_xz_concat_1['应付日期'].dt.strftime('%Y-%m')
df_xz_concat_1['date_tmp_1'] = pd.to_datetime(df_xz_concat_1['date_tmp_1'])
df_xz_concat_1['date_tmp_2'] = np.where(df_xz_concat_1['finish_date_new'].notnull(), pd.to_datetime(df_xz_concat_1['finish_date_new']).dt.strftime('%Y-%m'), np.nan)
df_xz_concat_1['date_tmp_2'] = pd.to_datetime(df_xz_concat_1['date_tmp_2'])

# 判断订单结束日期非空的订单是买断还是非买断，未结束的订单不判断（资方所需的是是否结清，分买断或完成）
df_xz_concat_1['订单状态_new'] = df_xz_concat_1.apply(lambda x: '已买断' if x.date_tmp_1 == x.date_tmp_2 and x.rembursement_status == '已买断'
                                              else '已完成' if x.date_tmp_1 == x.date_tmp_2 and x.rembursement_status == '已完成'
                                              else '还款中' if x.date_tmp_1 < x.date_tmp_2
                                              else '已结束' if x.date_tmp_1 > x.date_tmp_2
                                              else '其他', axis = 1
                                              )

df_xz_concat_1['提前状态'] = df_xz_concat_1.apply(lambda x: '提前买断' if x.finish_date_new < x.最大应还日期 and x.订单状态_new == '已买断'
                                              else '已完成' if x.finish_date_new < x.最大应还日期 and x.订单状态_new == '已完成'
                                              else '其他', axis = 1
                                              )
# df_xz_concat_1 = df_xz_concat_1.drop(columns = ['date_tmp_1', 'date_tmp_2'])
df_xz_concat_1.shape

# (7)关联买断数据 
sql1 = '''-- 买断信息   
SELECT too.order_id
,too.real_pay_money 
,too.pay_date
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断
and too.real_pay_money > 0 and too.pay_date is not null
'''
df_out_1 = query(sql1)
df_out_1.shape
df_out_1["pay_date"] = df_out_1['pay_date'].dt.date
df_out_1["pay_date"] = pd.to_datetime(df_out_1["pay_date"],errors="coerce")
df_out_1['pay_date_m'] = df_out_1['pay_date'].dt.strftime('%Y-%m')
df_out_1['pay_date_m'] = pd.to_datetime(df_out_1['pay_date_m'])

df_out_2 = df_out_1.groupby(['order_id','pay_date_m']).agg({'real_pay_money':'sum'}).reset_index()
df_out_2 = df_out_2.rename(columns = {'real_pay_money': 'real_pay_money_new'})
df_out_2.shape

df_xz_concat_1 = df_xz_concat_1.merge(df_out_2[['order_id', 'real_pay_money_new', 'pay_date_m']], left_on = ['order_id', 'date_tmp_1'], right_on = ['order_id','pay_date_m'], how = 'left')
df_xz_concat_1 = df_xz_concat_1.drop(columns = ['date_tmp_1', 'date_tmp_2', 'pay_date_m'])

# 计算剩余买断尾款
df_xz_concat_1['real_pay_money_new'] = df_xz_concat_1['real_pay_money_new'].fillna(0)
df_xz_concat_1 = df_xz_concat_1.rename(columns = {'real_pay_money_new': '当月已收买断金'})
df_xz_concat_1 = df_xz_concat_1.sort_values(by = ['order_number','当前期数'])
df_xz_concat_1['当前累计已收买断金'] = df_xz_concat_1.groupby(['order_number'])['当月已收买断金'].cumsum()

df_xz_concat_1['剩余总应收买断金'] = df_xz_concat_1['买断尾款'] - df_xz_concat_1['当前累计已收买断金'] 
df_xz_concat_1['剩余总应收买断金'] = np.where(df_xz_concat_1['剩余总应收买断金'] < 0 , 0, df_xz_concat_1['剩余总应收买断金'])
df_xz_concat_1['剩余总应收买断金'] = np.where(df_xz_concat_1['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_xz_concat_1['剩余总应收买断金'])

df_xz_concat_1.shape
df_xz_concat_1[:1]
	
# var1 = ['order_number','is_relet','当前期数', '下单日期', '应付日期', '当前应付租金', '实付日期new','实付金额','rembursement_status','订单状态','if_outpay','状态','订单状态_new']
# var2 = ['mob_date_11','paid_date_mob11','paid_money_mob11','mob_date_12','paid_date_mob12','paid_money_mob12','mob_date_13','paid_date_mob13','paid_money_mob13','mob_date_14','paid_date_mob14','paid_money_mob14','mob_date_15','paid_date_mob15','paid_money_mob15']
# var3 = ['order_id','当前期数','应付日期','实付日期new','实付金额','finish_date_new','订单状态_new','首期应还日期','买断价','应付总租金','买断尾款','订单状态_new','real_pay_money_new','剩余总应收买断金']
# var = var1 + var2 + var3

# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024060200290566'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024063001194047'])][var3]
# 订单已完成
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024031113200319'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202308262317036'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207050851373'])][var3]
# 在15期之前买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024020317504938'])][var3]
# 在15期之后买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207171820512'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207181530165'])][var3]
# 已买断但 买断尾款	!= real_pay_money_new  可能使用优惠劵
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207151707153'])][var3]
# 续租的实付买断金非0


(435119, 163)

(34992, 19)

(34989, 19)

(394350, 164)

(40769, 164)

...

(1321205, 170)

(2964, 3)

(2883, 3)

(1321205, 171)

,id_x,order_id,当前应付租金,实付金额,interest_money,实付日期,status_x,当前期数,order_pay_id,create_time_x,...,租赁方案,最大应还日期,买断价,应付总租金,买断尾款,订单状态_new,提前状态,当月已收买断金,当前累计已收买断金,剩余总应收买断金
0,6506.0,8924,357.95,357.95,0.0,2022-07-05,3.0,1.0,9576,2022-07-05 08:51:37,...,租完归还,2024-05-31,7530.0,4295.4,3234.6,还款中,其他,0.0,0.0,3234.6


In [95]:
# （2-2）首租
# （1）非续租租期数据
# (384318, 161)
# df_fq_concat = df_xz_concat[df_xz_concat.is_relet==0]
# 2025.2.9
df_xz_concat_2 = df_xz_concat.merge(df_xz_concat_1[['order_number', '订单状态_new','当前期数']],on = ['order_number','当前期数'], how = 'inner')
# conditions_1 = (df_xz_concat_2.订单状态_new != '已结束') | ((df_xz_concat_2.订单状态_new == '已结束') & (df_xz_concat_2.实付金额 > 0))
# df_xz_concat_2 = df_xz_concat_2[conditions_1]
df_fq_concat = df_xz_concat_2[df_xz_concat_2.is_relet==0]
df_fq_concat['当前应付租金'] = np.where((df_fq_concat.订单状态_new == '已结束') & (df_fq_concat.实付金额 == 0), 0, df_fq_concat.当前应付租金)

df_fq_concat.shape
df_xz_concat.is_relet.value_counts()

# note 20241205 首个应付日期更新为下单日期，避免 首个应付日期为下个月的订单没有 把mob0的已付统计在内，首个应收日期为下个月的订单，mob1没有账单日
df_fq_concat['应付日期_new'] = np.where(df_fq_concat.当前期数 == 1, df_fq_concat.下单日期, df_fq_concat.应付日期)
# note 20241206 更新 应付日期，提前还款的统计在内
df_fq_concat['应付日期_new'] = np.where((df_fq_concat['实付日期new'].notna())&((df_fq_concat['应付日期_new'] - df_fq_concat['实付日期new']).dt.days >= 0),df_fq_concat['实付日期new'],df_fq_concat['应付日期_new'])
df_fq_concat.shape

# （2）12期加工
# note 20241205 每月28.29.30号下单的没有统计mob_0已付的
df_mob_list = []
for i in range(0,13):
# 2025.2.9
# for i in range(0,16):
    # note 20241205 增加 (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])，不然预收的没有在第1期体现出来， A2024063001194047 下单日期!=实付日期new（第一期）
    # note 20241205 增加=，不然28或27号的没有mob0，导致 paid_money_mob 计算不正确，即还款日为月末的计算不正确
    # df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days > 0) 
    df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期_new']).dt.days >= 0) 
                        #   & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                        # 2025.2.9
                          & ((df_fq_concat['date_str'] - df_fq_concat['应付日期_new']).dt.days > 0) 
                          & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                          | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])]  # &((dfzh1['date_str'] - dfzh1[f'mob_date_{i}']).dt.days > 0)
# df_mob2 = dfzh1[((dfzh1['mob_date_2'] - dfzh1['应付日期']).dt.days > 0) & (dfzh1.状态 != '已取消') & ((dfzh1['date_str'] - dfzh1['mob_date_2']).dt.days > 0)]

    # df_mob['overdue_days_mob'] = df_mob.apply(lambda x:
    #                                           0 if x['逾期天数']==0  
    #                                           else 0  if pd.notnull(x[f'paid_date_mob{i}']) 
    #                                           else (x['date_str'] - x['应付日期']).days  if  ((x[f'mob_date_{i}'] - x['date_str']).days > 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else (x[f'mob_date_{i}'] - x['应付日期']).days if ((x[f'mob_date_{i}'] - x['date_str']).days <= 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else x['逾期天数'] ,axis=1 )
    # # np.where(df_mob[f'paid_date_mob{i}'].isnull(),(np.minimum(df_mob['date_str'],df_mob[f'mob_date_{i}']) - df_mob['应付日期']).dt.days  ,(df_mob[f'paid_date_mob{i}'] - df_mob['应付日期']).dt.days)
    # df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # df_mob_group['MOB'] = f'mob_{i}'

    # df_mob_list.append(df_mob_group)

    conditions = [ ( ( df_mob['逾期天数']==0) | (pd.notnull(df_mob[f'paid_date_mob{i}']))),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days > 0)  & (pd.isnull(df_mob[f'paid_date_mob{i}'])) ),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days <= 0) & ( pd.isnull(df_mob[f'paid_date_mob{i}'])))]  
  
    choices = [0,(df_mob['date_str'] - df_mob['应付日期']).dt.days,(df_mob[f'mob_date_{i}'] - df_mob['应付日期']).dt.days]  
  
    # 使用numpy.where来应用条件  
    # note 20241127 overdue_days_mob 截止每个mob观察月的逾期天数，月末状态，若月末不逾期了，则为0
    df_mob['overdue_days_mob'] = np.select(conditions, choices, default = df_mob['逾期天数'])

    df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # note 20241210 修改 提前支付的剔除未到期应收
    df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    df_mob_group_1 = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days >= 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                      | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])
                      ].groupby(by=['order_number']).agg({'当前应付租金':'sum'}).reset_index()
    
    # 2025.2.11，当前最大期数不用 (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])
    df_mob_group_1_1 = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days >= 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                      ].groupby(by=['order_number']).agg({'当前期数':'max'}).reset_index()
    
    df_mob_group_1 = df_mob_group_1.merge(df_mob_group_1_1, on = 'order_number', how = 'inner')

    df_mob_group = df_mob_group.merge(df_mob_group_1, on = 'order_number', how = 'left')

    df_mob_group['MOB'] = f'mob_{i}'

    df_mob_list.append(df_mob_group)

df_mob_total = pd.concat(df_mob_list, ignore_index=True)

# 2025.2.9
df_mob_total.shape
df_mob_total = df_mob_total[df_mob_total.当前应付租金 >= 0]
df_mob_total = df_mob_total.drop_duplicates(subset = ['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','当前期数'])
df_mob_total.shape
df_mob_total = df_mob_total.sort_values(by = ['order_number','当前期数','overdue_days_mob'])
df_mob_total = df_mob_total.drop_duplicates(subset = ['order_number','当前期数'])

# df_mob_total[df_mob_total.order_number == 'A202207151707153']
# var3 = ['order_id','当前期数','应付日期','当前应付租金','实付日期new','实付金额','finish_date_new','订单状态_new','首期应还日期','买断价','应付总租金','买断尾款','订单状态_new','real_pay_money_new','剩余总应收买断金']
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024060200290566'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024063001194047'])][var3]
# 订单已完成
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024031113200319'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202308262317036'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207050851373'])][var3]
# # 在15期之前买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024020317504938'])][var3]
# 在15期之后买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207171820512'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207181530165'])][var3] --2个12期
# 已买断但 买断尾款	!= real_pay_money_new  可能使用优惠劵
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207151707153'])][var3]

# paid_money_mob：当前累计实付租金  当前应付租金：当前累计应付租金
df_mob_total = df_mob_total.rename(columns = {'当前应付租金': '当前累计应付租金', 'paid_money_mob': '当前累计实付租金'})
df_mob_total.tail(10)

df_xz_concat_1.shape
df_xz_concat.shape
df_fq_concat.shape
df_mob_total.shape

# （2）
# note 20241205 每月28.29.30号下单的没有统计mob_0已付的
df_mob_list = []
for i in range(0,13):
# 2025.2.9
# for i in range(0,16):
    # note 20241205 增加 (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])，不然预收的没有在第1期体现出来， A2024063001194047 下单日期!=实付日期new（第一期）
    # note 20241205 增加=，不然28或27号的没有mob0，导致 paid_money_mob 计算不正确，即还款日为月末的计算不正确
    # df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days > 0) 
    df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期_new']).dt.days >= 0) 
                        #   & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                        # 2025.2.9
                          & ((df_fq_concat['date_str'] - df_fq_concat['应付日期_new']).dt.days > 0) 
                          & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                          | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])]  # &((dfzh1['date_str'] - dfzh1[f'mob_date_{i}']).dt.days > 0)
# df_mob2 = dfzh1[((dfzh1['mob_date_2'] - dfzh1['应付日期']).dt.days > 0) & (dfzh1.状态 != '已取消') & ((dfzh1['date_str'] - dfzh1['mob_date_2']).dt.days > 0)]

    # df_mob['overdue_days_mob'] = df_mob.apply(lambda x:
    #                                           0 if x['逾期天数']==0  
    #                                           else 0  if pd.notnull(x[f'paid_date_mob{i}']) 
    #                                           else (x['date_str'] - x['应付日期']).days  if  ((x[f'mob_date_{i}'] - x['date_str']).days > 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else (x[f'mob_date_{i}'] - x['应付日期']).days if ((x[f'mob_date_{i}'] - x['date_str']).days <= 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else x['逾期天数'] ,axis=1 )
    # # np.where(df_mob[f'paid_date_mob{i}'].isnull(),(np.minimum(df_mob['date_str'],df_mob[f'mob_date_{i}']) - df_mob['应付日期']).dt.days  ,(df_mob[f'paid_date_mob{i}'] - df_mob['应付日期']).dt.days)
    # df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # df_mob_group['MOB'] = f'mob_{i}'

    # df_mob_list.append(df_mob_group)

    conditions = [ ( ( df_mob['逾期天数']==0) | (pd.notnull(df_mob[f'paid_date_mob{i}']))),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days > 0)  & (pd.isnull(df_mob[f'paid_date_mob{i}'])) ),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days <= 0) & ( pd.isnull(df_mob[f'paid_date_mob{i}'])))]  
  
    choices = [0,(df_mob['date_str'] - df_mob['应付日期']).dt.days,(df_mob[f'mob_date_{i}'] - df_mob['应付日期']).dt.days]  
  
    # 使用numpy.where来应用条件  
    # note 20241127 overdue_days_mob 截止每个mob观察月的逾期天数，月末状态，若月末不逾期了，则为0
    df_mob['overdue_days_mob'] = np.select(conditions, choices, default = df_mob['逾期天数'])

    df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # note 20241210 修改 提前支付的剔除未到期应收
    df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    df_mob_group_1 = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days >= 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                      | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])
                      ].groupby(by=['order_number']).agg({'当前应付租金':'sum'}).reset_index()
    
    # 2025.2.11，当前最大期数不用 (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])
    df_mob_group_1_1 = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days >= 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                      ].groupby(by=['order_number']).agg({'当前期数':'max'}).reset_index()
    
    df_mob_group_1 = df_mob_group_1.merge(df_mob_group_1_1, on = 'order_number', how = 'inner')

    df_mob_group = df_mob_group.merge(df_mob_group_1, on = 'order_number', how = 'left')

    df_mob_group['MOB'] = f'mob_{i}'

    df_mob_list.append(df_mob_group)

df_mob_total = pd.concat(df_mob_list, ignore_index=True)

# 2025.2.9
df_mob_total.shape
df_mob_total = df_mob_total[df_mob_total.当前应付租金 >= 0]
df_mob_total = df_mob_total.drop_duplicates(subset = ['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','当前期数'])
df_mob_total.shape
df_mob_total = df_mob_total.sort_values(by = ['order_number','当前期数','overdue_days_mob'])
df_mob_total = df_mob_total.drop_duplicates(subset = ['order_number','当前期数'])

# df_mob_total[df_mob_total.order_number == 'A202207151707153']
# var3 = ['order_id','当前期数','应付日期','当前应付租金','实付日期new','实付金额','finish_date_new','订单状态_new','首期应还日期','买断价','应付总租金','买断尾款','订单状态_new','real_pay_money_new','剩余总应收买断金']
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024060200290566'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024063001194047'])][var3]
# 订单已完成
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024031113200319'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202308262317036'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207050851373'])][var3]
# # 在15期之前买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024020317504938'])][var3]
# 在15期之后买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207171820512'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207181530165'])][var3] --2个12期
# 已买断但 买断尾款	!= real_pay_money_new  可能使用优惠劵
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207151707153'])][var3]

# paid_money_mob：当前累计实付租金  当前应付租金：当前累计应付租金
df_mob_total = df_mob_total.rename(columns = {'当前应付租金': '当前累计应付租金', 'paid_money_mob': '当前累计实付租金'})
df_mob_total.tail(10)

df_xz_concat_1.shape
df_xz_concat.shape
df_fq_concat.shape
df_mob_total.shape

# (520033, 170)
# (424063, 161)
# (384318, 163)
# (234387, 7)

# （3）关联多期的表
# 2025.2.9
var_1 = ['order_number','下单日期','租赁方案','当前期数','应付日期', '当前应付租金', '实付金额','实付日期new','买断价','应付总租金','买断尾款','订单状态_new','当月已收买断金','当前累计已收买断金','剩余总应收买断金','提前状态']
df_mob_total_tmp = df_xz_concat_1[df_xz_concat_1.is_relet == 0][var_1].merge(df_mob_total[['order_number','当前期数','当前累计应付租金','当前累计实付租金','overdue_days_mob']], on = ['order_number','当前期数'], how = 'left')
df_xz_concat_1[df_xz_concat_1.is_relet == 0].shape
df_mob_total_tmp.shape

tmp_2 = df_xz_concat.groupby(['order_number']).agg({'is_relet':'max'}).reset_index()
tmp_2 = tmp_2.rename(columns = {'is_relet': 'is_relet_max'})
tmp_2.shape

df_mob_total_tmp = df_mob_total_tmp.merge(tmp_2, on = 'order_number', how ='inner')
df_mob_total_tmp.shape
del tmp_2

# 加工 df_mob_total_tmp
# 续租的订单只有12期，第12期的订单状态修改为已完成
df_mob_total_tmp['订单状态_new'] = np.where((df_mob_total_tmp.is_relet_max >= 1) & (df_mob_total_tmp.当前期数 == 12), '已完成', df_mob_total_tmp['订单状态_new'])
df_mob_total_tmp['剩余总应收买断金'] = np.where(df_mob_total_tmp['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_mob_total_tmp['剩余总应收买断金'])
# 删去订单已结束的订单
df_mob_total_tmp = df_mob_total_tmp[df_mob_total_tmp.订单状态_new != '已结束']
df_mob_total_tmp.shape
# 订单未结束的，删去未到期的，最新时间为 2025.2.10
df_mob_total_tmp = df_mob_total_tmp[df_mob_total_tmp.应付日期 <= '2025-01-31']
df_mob_total_tmp.shape
# 订单未结束的，到期未结束的则取完15期的数据，若租期一直逾期，则衍生到15期，若租期逾期，则默认不逾期
# A2024020317504938 2025.2 买断，还没表现出来
# 'A202207171820512' -- 9683，在15期之外买断，不加工租期逾期
# 'A202207181530165' --一直逾期，直至租期到了之后也逾期

df_mob_total_tmp['overdue_days_mob'] = df_mob_total_tmp['overdue_days_mob'].fillna(0)
df_mob_total_tmp['当前累计应付租金'] = df_mob_total_tmp['当前累计应付租金'].fillna(0)
df_mob_total_tmp['当前累计实付租金'] = df_mob_total_tmp['当前累计实付租金'].fillna(0)

tmp_2 = df_mob_total.groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp_2 = tmp_2.rename(columns = {'当前期数': '当前期数'})
tmp_3 = df_mob_total[['order_number','当前期数','当前累计实付租金','当前累计应付租金','overdue_days_mob']].merge(tmp_2, on = ['order_number','当前期数'], how = 'inner')
tmp_2.shape
tmp_3.shape

# tmp_3['当前期数'] = tmp_3['当前期数'] + 1
# df_mob_total_tmp = df_mob_total_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp['overdue_days_mob_y'] > 0, df_mob_total_tmp['overdue_days_mob_y'] + 30, df_mob_total_tmp['overdue_days_mob_x'])
# df_mob_total_tmp['当前累计应付租金'] = np.where(df_mob_total_tmp['当前累计应付租金_y'] > df_mob_total_tmp['当前累计应付租金_x'], df_mob_total_tmp['当前累计应付租金_y'], df_mob_total_tmp['当前累计应付租金_x'])
# df_mob_total_tmp['当前累计实付租金'] = np.where(df_mob_total_tmp['当前累计实付租金_y'] > df_mob_total_tmp['当前累计实付租金_x'], df_mob_total_tmp['当前累计实付租金_y'], df_mob_total_tmp['当前累计实付租金_x'])
# df_mob_total_tmp = df_mob_total_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

# tmp_3['当前期数'] = tmp_3['当前期数'] + 1
# df_mob_total_tmp = df_mob_total_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp['overdue_days_mob_y'] > 0, df_mob_total_tmp['overdue_days_mob_y'] + 60, df_mob_total_tmp['overdue_days_mob_x'])
# df_mob_total_tmp['当前累计应付租金'] = np.where(df_mob_total_tmp['当前累计应付租金_y'] > df_mob_total_tmp['当前累计应付租金_x'], df_mob_total_tmp['当前累计应付租金_y'], df_mob_total_tmp['当前累计应付租金_x'])
# df_mob_total_tmp['当前累计实付租金'] = np.where(df_mob_total_tmp['当前累计实付租金_y'] > df_mob_total_tmp['当前累计实付租金_x'], df_mob_total_tmp['当前累计实付租金_y'], df_mob_total_tmp['当前累计实付租金_x'])
# df_mob_total_tmp = df_mob_total_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

# tmp_3['当前期数'] = tmp_3['当前期数'] + 1
# df_mob_total_tmp = df_mob_total_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp['overdue_days_mob_y'] > 0, df_mob_total_tmp['overdue_days_mob_y'] + 90, df_mob_total_tmp['overdue_days_mob_x'])
# df_mob_total_tmp['当前累计应付租金'] = np.where(df_mob_total_tmp['当前累计应付租金_y'] > df_mob_total_tmp['当前累计应付租金_x'], df_mob_total_tmp['当前累计应付租金_y'], df_mob_total_tmp['当前累计应付租金_x'])
# df_mob_total_tmp['当前累计实付租金'] = np.where(df_mob_total_tmp['当前累计实付租金_y'] > df_mob_total_tmp['当前累计实付租金_x'], df_mob_total_tmp['当前累计实付租金_y'], df_mob_total_tmp['当前累计实付租金_x'])
# df_mob_total_tmp = df_mob_total_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

for num in range(1,28):
    tmp_3['当前期数'] = tmp_3['当前期数'] + 1
    df_mob_total_tmp = df_mob_total_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
    # df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp['overdue_days_mob_y'] > 0, df_mob_total_tmp['overdue_days_mob_y'] + num*30, df_mob_total_tmp['overdue_days_mob_x'])
    df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp['overdue_days_mob_y'] >= 0, df_mob_total_tmp['overdue_days_mob_y'] + num*30, df_mob_total_tmp['overdue_days_mob_x'])
    df_mob_total_tmp['当前累计应付租金'] = np.where(df_mob_total_tmp['当前累计应付租金_y'] > df_mob_total_tmp['当前累计应付租金_x'], df_mob_total_tmp['当前累计应付租金_y'], df_mob_total_tmp['当前累计应付租金_x'])
    df_mob_total_tmp['当前累计实付租金'] = np.where(df_mob_total_tmp['当前累计实付租金_y'] > df_mob_total_tmp['当前累计实付租金_x'], df_mob_total_tmp['当前累计实付租金_y'], df_mob_total_tmp['当前累计实付租金_x'])
    df_mob_total_tmp = df_mob_total_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})


df_mob_total_tmp.shape
df_mob_total_tmp['search_time'] = df_mob_total_tmp['下单日期'].dt.strftime('%Y-%m')

tmp = df_xz_concat[df_xz_concat.is_relet == 0].groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp = tmp.rename(columns = {'当前期数': '最大期数'})
df_mob_total_tmp = df_mob_total_tmp.merge(tmp, on = 'order_number', how = 'left')
df_mob_total_tmp.shape

df_mob_total_tmp['买断尾款'] = np.where(df_mob_total_tmp.租赁方案 == '租完即送', 0, df_mob_total_tmp.买断尾款)
df_mob_total_tmp['剩余总应收买断金'] = np.where(df_mob_total_tmp.租赁方案 == '租完即送', 0, df_mob_total_tmp.剩余总应收买断金)

df_mob_total_tmp['应付总租金'] = df_mob_total_tmp['应付总租金'].fillna(0)
df_mob_total_tmp['当前累计实付租金'] = df_mob_total_tmp['当前累计实付租金'].fillna(0)
df_mob_total_tmp['剩余总应收租金'] = df_mob_total_tmp['应付总租金'] - df_mob_total_tmp['当前累计实付租金']
df_mob_total_tmp['剩余总应收租金'] = np.where(df_mob_total_tmp['剩余总应收租金'] < 0, 0, df_mob_total_tmp['剩余总应收租金'])
df_mob_total_tmp['剩余总应收租金'] = np.where(df_mob_total_tmp['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_mob_total_tmp['剩余总应收租金'])

df_mob_total_tmp['剩余总应收'] = df_mob_total_tmp['剩余总应收租金'] + df_mob_total_tmp['剩余总应收买断金']
df_mob_total_tmp['剩余总应收'] = np.where(df_mob_total_tmp['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_mob_total_tmp['剩余总应收'])
df_mob_total_tmp['剩余总应收买断金'] = np.where(df_mob_total_tmp['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_mob_total_tmp['剩余总应收买断金'])

# A2023112717153031
df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp.订单状态_new.isin(['已买断','已完成','已结束']), 0 , df_mob_total_tmp.overdue_days_mob)
df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp.当前期数 == 1, 0 , df_mob_total_tmp.overdue_days_mob)
df_mob_total_tmp[:1]

(394350, 162)

is_relet
0.0    394350
1.0     40705
2.0        64
Name: count, dtype: int64

(394350, 163)

(405058, 7)

(236931, 7)

...

(31695, 2)

(31695, 5)

(250318, 20)

(250318, 22)

,order_number,下单日期,租赁方案,当前期数,应付日期,当前应付租金,实付金额,实付日期new,买断价,应付总租金,...,剩余总应收买断金,提前状态,is_relet_max,overdue_days_mob,当前累计应付租金,当前累计实付租金,search_time,最大期数,剩余总应收租金,剩余总应收
0,A202207050851373,2022-07-05,租完归还,1.0,2022-07-05,357.95,357.95,2022-07-05,7530.0,4295.4,...,3234.6,其他,2.0,0.0,357.95,357.95,2022-07,12.0,3937.45,7172.05


In [96]:
# （2-3）续租
# （1）续租数据
# (39745, 162)
# df_relet = df_xz_concat[df_xz_concat.is_relet>=1]
# df_relet.shape

# # 2025.2.11
df_xz_concat_2 = df_xz_concat.merge(df_xz_concat_1[['order_number', '订单状态_new','当前期数']],on = ['order_number','当前期数'], how = 'inner')
# conditions_1 = (df_xz_concat_2.订单状态_new != '已结束') | ((df_xz_concat_2.订单状态_new == '已结束') & (df_xz_concat_2.实付金额 > 0))
# df_xz_concat_2 = df_xz_concat_2[conditions_1]
df_relet = df_xz_concat_2[df_xz_concat_2.is_relet >= 1]
df_relet['当前应付租金'] = np.where((df_relet.订单状态_new == '已结束') & (df_relet.实付金额 == 0), 0, df_relet.当前应付租金)

df_relet.shape
df_xz_concat.is_relet.value_counts()

# （2）每个mob对应的月底时间，有些订单mob1不用还，故 n 放到 24
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    for n in range(13,25):  
        # note 20241209 修改 月末的的跳级了，所以不要加1，mob对齐
        # df[f'mob_date_{n}'] = np.where(df["下单日期"].dt.day.isin([28,29,30,31]), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = np.where((df["下单日期"].dt.day == 31)|(df["下单日期"].isin(['2023-02-28','2024-02-29','2025-02-28']))|((df["下单日期"].dt.month.isin([4,6,9,11]))&(df["下单日期"].dt.day == 30)), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        # df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1) 
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}']) 
    return df

df_relet = mob_date(df_relet)
df_relet.shape
df_relet[['下单日期','mob_date_15','mob_date_21','mob_date_20','mob_date_13','实付金额','paid_money_mob13','paid_date_mob15','paid_money_mob18']][:2]

# （3）每个mob对应的逾期天数  
# note 20241209 更新 应付日期，提前还款的统计在内
df_relet['应付日期_new'] = np.where((df_relet['实付日期new'].notna())&((df_relet['应付日期'] - df_relet['实付日期new']).dt.days >= 0),df_relet['实付日期new'],df_relet['应付日期'])
df_mob_list = []

for i in range(12,25): 
    # note 20241209 增加 =
    df_mob = df_relet[((df_relet[f'mob_date_{i}'] - df_relet['应付日期_new']).dt.days >= 0) 
                      & ((df_relet['date_str'] - df_relet['应付日期']).dt.days > 0) 
                    #   2025.2.11
                    #   & ((df_relet['date_str'] - df_relet['应付日期_new']).dt.days > 0) 
                      & ((df_relet['date_str'] - df_relet[f'mob_date_{i}']).dt.days > 0)
                      ]
   
    conditions = [ ( ( df_mob['逾期天数']==0) | (pd.notnull(df_mob[f'paid_date_mob{i}']))),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days > 0)  & (pd.isnull(df_mob[f'paid_date_mob{i}'])) ),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days <= 0) & ( pd.isnull(df_mob[f'paid_date_mob{i}'])))]  
  
    choices = [0,(df_mob['date_str'] - df_mob['应付日期']).dt.days,(df_mob[f'mob_date_{i}'] - df_mob['应付日期']).dt.days]  
  
    # 使用numpy.where来应用条件  
    df_mob['overdue_days_mob'] = np.select(conditions, choices, default = df_mob['逾期天数'])

    df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']
    # df_mob

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # note 20241210 修改 提前支付的剔除未到期应收
    df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    df_mob_group_1 = df_relet[((df_relet[f'mob_date_{i}'] - df_relet['应付日期']).dt.days >= 0) 
                      & ((df_relet['date_str'] - df_relet['应付日期']).dt.days > 0) 
                      & ((df_relet['date_str'] - df_relet[f'mob_date_{i}']).dt.days > 0)
                      ].groupby(by=['order_number']).agg({'当前应付租金':'sum','当前期数':'max'}).reset_index()
    df_mob_group = df_mob_group.merge(df_mob_group_1, on = 'order_number', how = 'left')


    df_mob_group['MOB'] = f'mob_{i}'

    df_mob_list.append(df_mob_group)

df_mob_relet = pd.concat(df_mob_list, ignore_index=True)

# 数据检查
# order_number order_id
# A202207050851373    8924     已买断，买断金额没有体现在实付金额中，买断后剩余期数的实付金额为0；2023-07-05、2023-10-3的还款记录在系统没有找到
# A2023073114301076   508404  只有mob13-16，mob12在非续租那里了，2023.8不用还款，隔月了，故非续租为13个月长度
# A2023070210395232   432132  只有mob12-16，其他的未到期
# A20231026115543149  726194  只有mob12、mob13，其他的未到期

# df_mob_relet[df_mob_relet.order_number == 'A202207050851373']
# 有mob12但无mob13
# A2023110100321259  738560  23
# 有mob13但无mob12
# A202208291922283    18163  26  2023.2-2023.5 的账单没有还，但2023.6-2023.8的账单还了
# A202207282047042    11630  25

# order_number_param = 'A202207282047042'
# var = ['mob_date_0','mob_date_11','mob_date_12','mob_date_13','mob_date_21','mob_date_23']
# var1 = ['mob_date_0','mob_date_1','mob_date_2','mob_date_3','mob_date_4','mob_date_11','mob_date_12']
# df_xz_concat[df_xz_concat.order_number == order_number_param][['order_number','下单日期','应付日期','实付日期new','实付金额] + var1]
# df_relet[df_relet.order_number == order_number_param][['order_number','order_id','下单日期','状态','应付日期','当前应付租金','实付日期new','实付金额','逾期天数'] + var]
# df_mob_relet[df_mob_relet.order_number == order_number_param][['order_number','下单日期','当前应付租金','paid_money_mob','overdue_days_mob','MOB']]

# 2025.2.9
df_mob_relet.shape
df_mob_relet = df_mob_relet[df_mob_relet.当前应付租金 >= 0]
df_mob_relet = df_mob_relet.drop_duplicates(subset = ['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','当前期数'])
df_mob_relet.shape
df_mob_relet = df_mob_relet.sort_values(by = ['order_number','当前期数','overdue_days_mob'])
df_mob_relet = df_mob_relet.drop_duplicates(subset = ['order_number','当前期数'])

df_mob_relet = df_mob_relet.rename(columns = {'当前应付租金': '当前累计应付租金', 'paid_money_mob': '当前累计实付租金'})
df_mob_relet.tail(10)

# （4）关联多期的表
# 2025.2.11
var_1 = ['order_number','下单日期','租赁方案','当前期数','应付日期', '当前应付租金', '实付金额','实付日期new','买断价','应付总租金','买断尾款','订单状态_new','当月已收买断金','当前累计已收买断金','剩余总应收买断金','提前状态']
df_mob_relet_tmp = df_xz_concat_1[df_xz_concat_1.is_relet >= 1][var_1].merge(df_mob_relet[['order_number','当前期数','当前累计应付租金','当前累计实付租金','overdue_days_mob']], on = ['order_number','当前期数'], how = 'left')
df_xz_concat_1[df_xz_concat_1.is_relet >= 1].shape
df_mob_relet_tmp.shape

# tmp_2 = df_xz_concat.groupby(['order_number']).agg({'is_relet':'max'}).reset_index()
# tmp_2 = tmp_2.rename(columns = {'is_relet': 'is_relet_max'})
# tmp_2.shape

# df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp_2, on = 'order_number', how ='inner')
# df_mob_relet_tmp.shape
# del tmp_2

# 加工 df_mob_relet_tmp
# 续租的订单只有12期，第12期的订单状态修改为已完成
# df_mob_relet_tmp['订单状态_new'] = np.where((df_mob_relet_tmp.is_relet_max >= 1) & (df_mob_relet_tmp.当前期数 == 12), '已完成', df_mob_relet_tmp['订单状态_new'])
# 删去订单已结束的订单
df_mob_relet_tmp = df_mob_relet_tmp[df_mob_relet_tmp.订单状态_new != '已结束']
df_mob_relet_tmp.shape
# 订单未结束的，删去未到期的，最新时间为 2025.2.10
df_mob_relet_tmp = df_mob_relet_tmp[df_mob_relet_tmp.应付日期 <= '2025-01-31']
df_mob_relet_tmp.shape
# 订单未结束的，到期未结束的则取完15期的数据，若租期一直逾期，则衍生到15期，若租期逾期，则默认不逾期
# A2024020317504938 2025.2 买断，还没表现出来
# 'A202207171820512' -- 9683，在15期之外买断，不加工租期逾期
# 'A202207181530165' --一直逾期，直至租期到了之后也逾期

df_mob_relet_tmp['overdue_days_mob'] = df_mob_relet_tmp['overdue_days_mob'].fillna(0)
df_mob_relet_tmp['当前累计应付租金'] = df_mob_relet_tmp['当前累计应付租金'].fillna(0)
df_mob_relet_tmp['当前累计实付租金'] = df_mob_relet_tmp['当前累计实付租金'].fillna(0)

tmp_2 = df_mob_relet.groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp_2 = tmp_2.rename(columns = {'当前期数': '当前期数'})
tmp_3 = df_mob_relet[['order_number','当前期数','当前累计实付租金','当前累计应付租金','overdue_days_mob']].merge(tmp_2, on = ['order_number','当前期数'], how = 'inner')
tmp_2.shape
tmp_3.shape

# tmp_3['当前期数'] = tmp_3['当前期数'] + 1
# df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] > 0, df_mob_relet_tmp['overdue_days_mob_y'] + 30, df_mob_relet_tmp['overdue_days_mob_x'])
# df_mob_relet_tmp['当前累计应付租金'] = np.where(df_mob_relet_tmp['当前累计应付租金_y'] > df_mob_relet_tmp['当前累计应付租金_x'], df_mob_relet_tmp['当前累计应付租金_y'], df_mob_relet_tmp['当前累计应付租金_x'])
# df_mob_relet_tmp['当前累计实付租金'] = np.where(df_mob_relet_tmp['当前累计实付租金_y'] > df_mob_relet_tmp['当前累计实付租金_x'], df_mob_relet_tmp['当前累计实付租金_y'], df_mob_relet_tmp['当前累计实付租金_x'])
# df_mob_relet_tmp = df_mob_relet_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

# tmp_3['当前期数'] = tmp_3['当前期数'] + 1
# df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] > 0, df_mob_relet_tmp['overdue_days_mob_y'] + 60, df_mob_relet_tmp['overdue_days_mob_x'])
# df_mob_relet_tmp['当前累计应付租金'] = np.where(df_mob_relet_tmp['当前累计应付租金_y'] > df_mob_relet_tmp['当前累计应付租金_x'], df_mob_relet_tmp['当前累计应付租金_y'], df_mob_relet_tmp['当前累计应付租金_x'])
# df_mob_relet_tmp['当前累计实付租金'] = np.where(df_mob_relet_tmp['当前累计实付租金_y'] > df_mob_relet_tmp['当前累计实付租金_x'], df_mob_relet_tmp['当前累计实付租金_y'], df_mob_relet_tmp['当前累计实付租金_x'])
# df_mob_relet_tmp = df_mob_relet_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

# tmp_3['当前期数'] = tmp_3['当前期数'] + 1
# df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] > 0, df_mob_relet_tmp['overdue_days_mob_y'] + 90, df_mob_relet_tmp['overdue_days_mob_x'])
# df_mob_relet_tmp['当前累计应付租金'] = np.where(df_mob_relet_tmp['当前累计应付租金_y'] > df_mob_relet_tmp['当前累计应付租金_x'], df_mob_relet_tmp['当前累计应付租金_y'], df_mob_relet_tmp['当前累计应付租金_x'])
# df_mob_relet_tmp['当前累计实付租金'] = np.where(df_mob_relet_tmp['当前累计实付租金_y'] > df_mob_relet_tmp['当前累计实付租金_x'], df_mob_relet_tmp['当前累计实付租金_y'], df_mob_relet_tmp['当前累计实付租金_x'])
# df_mob_relet_tmp = df_mob_relet_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

for num in range(1,28):
    tmp_3['当前期数'] = tmp_3['当前期数'] + 1
    df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
    # df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] > 0, df_mob_relet_tmp['overdue_days_mob_y'] + num*30, df_mob_relet_tmp['overdue_days_mob_x'])
    df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] >= 0, df_mob_relet_tmp['overdue_days_mob_y'] + num*30, df_mob_relet_tmp['overdue_days_mob_x'])
    df_mob_relet_tmp['当前累计应付租金'] = np.where(df_mob_relet_tmp['当前累计应付租金_y'] > df_mob_relet_tmp['当前累计应付租金_x'], df_mob_relet_tmp['当前累计应付租金_y'], df_mob_relet_tmp['当前累计应付租金_x'])
    df_mob_relet_tmp['当前累计实付租金'] = np.where(df_mob_relet_tmp['当前累计实付租金_y'] > df_mob_relet_tmp['当前累计实付租金_x'], df_mob_relet_tmp['当前累计实付租金_y'], df_mob_relet_tmp['当前累计实付租金_x'])
    df_mob_relet_tmp = df_mob_relet_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

df_mob_relet_tmp.shape
df_mob_relet_tmp['search_time'] = df_mob_relet_tmp['下单日期'].dt.strftime('%Y-%m')

tmp = df_xz_concat[df_xz_concat.is_relet >= 1].groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp1 = df_xz_concat[df_xz_concat.is_relet == 0].groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp.shape
tmp1.shape
tmp = tmp.merge(tmp1, on = 'order_number', how = 'left')
tmp.shape
tmp['续租总期数'] = tmp['当前期数_x'] - tmp['当前期数_y']
tmp = tmp[['order_number','续租总期数']]

# tmp = tmp.rename(columns = {'当前期数': '最大期数'})
df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp, on = 'order_number', how = 'left')
df_mob_relet_tmp.shape

# df_mob_relet_tmp['买断尾款'] = np.where(df_mob_relet_tmp.租赁方案 == '租完即送', 0, df_mob_relet_tmp.买断尾款)
# df_mob_relet_tmp['剩余总应收买断金'] = np.where(df_mob_relet_tmp.租赁方案 == '租完即送', 0, df_mob_relet_tmp.剩余总应收买断金)

df_mob_relet_tmp['应付总租金'] = df_mob_relet_tmp['应付总租金'].fillna(0)
df_mob_relet_tmp['当前累计实付租金'] = df_mob_relet_tmp['当前累计实付租金'].fillna(0)
df_mob_relet_tmp['剩余总应收租金'] = df_mob_relet_tmp['应付总租金'] - df_mob_relet_tmp['当前累计实付租金']
df_mob_relet_tmp['剩余总应收租金'] = np.where(df_mob_relet_tmp['剩余总应收租金'] < 0, 0, df_mob_relet_tmp['剩余总应收租金'])
df_mob_relet_tmp['剩余总应收租金'] = np.where(df_mob_relet_tmp['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_mob_relet_tmp['剩余总应收租金'])

# df_mob_relet_tmp['剩余总应收'] = df_mob_relet_tmp['剩余总应收租金'] + df_mob_relet_tmp['剩余总应收买断金']

# A2023112717153031
df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp.订单状态_new.isin(['已买断','已完成','已结束']), 0 , df_mob_relet_tmp.overdue_days_mob)
df_mob_relet_tmp[:1]

(40769, 162)

is_relet
0.0    394350
1.0     40705
2.0        64
Name: count, dtype: int64

(40769, 198)

,下单日期,mob_date_15,mob_date_21,mob_date_20,mob_date_13,实付金额,paid_money_mob13,paid_date_mob15,paid_money_mob18
12,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,354.0,354.0,2023-07-07,354.0
13,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,354.0,354.0,2023-08-07,354.0


(18675, 7)

...

(3869, 2)

(32818, 2)

(3869, 3)

(17061, 21)

,order_number,下单日期,租赁方案,当前期数,应付日期,当前应付租金,实付金额,实付日期new,买断价,应付总租金,...,当月已收买断金,当前累计已收买断金,剩余总应收买断金,提前状态,overdue_days_mob,当前累计应付租金,当前累计实付租金,search_time,续租总期数,剩余总应收租金
0,A202207050851373,2022-07-05,租完归还,13.0,2023-07-05,354.0,354.0,2023-07-07,8176.92,3953.0,...,0.0,0.0,0.0,其他,0.0,354.0,354.0,2022-07,12.0,3599.0


In [97]:
# （2-4）统计
df_mob_total_tmp.shape
df_mob_relet_tmp.shape

df_mob_temp = pd.concat([df_mob_total_tmp, df_mob_relet_tmp])
df_mob_temp.shape
# df_mob_temp[:1]

df_mob_temp['订单状态_new'] = np.where((df_mob_temp['订单状态_new'] == '已完成') & (df_mob_temp.is_relet_max >= 1), '还款中', df_mob_temp['订单状态_new'])

# order_number_need = ['A2023112717153031','A2024060200290566','A2024063001194047','A2024031113200319','A202308262317036','A202207050851373','A2024020317504938','A202207171820512','A202207181530165','A202207151707153']
# df_mob_temp[df_mob_temp.order_number.isin(order_number_need)].to_excel('tmp3.xlsx', index = False)

df_mob_temp['view_month'] = df_mob_temp['应付日期'].dt.strftime('%Y-%m')
# df_mob_temp['search_time_tmp'] = df_mob_temp['应付日期'].dt.strftime('%Y-%m')
# df_mob_temp['view_month'] = df_mob_temp.apply(lambda x: '1.2022-07' if x.search_time_tmp <= '2022-07' and x.当前期数 <= 1
#                                               else '2.2022-08' if x.search_time_tmp <= '2022-08' and x.当前期数 <= 2
#                                               else '3.2022-09' if x.search_time_tmp <= '2022-09' and x.当前期数 <= 3
#                                               else '4.2022-10' if x.search_time_tmp <= '2022-10' and x.当前期数 <= 4
#                                               else '5.2022-11' if x.search_time_tmp <= '2022-11' and x.当前期数 <= 5
#                                               else '6.2022-12' if x.search_time_tmp <= '2022-12' and x.当前期数 <= 6

#                                               else '7.2023-01' if x.search_time_tmp <= '2023-01' and x.当前期数 <= 7
#                                               else '8.2023-02' if x.search_time_tmp <= '2023-02' and x.当前期数 <= 8
#                                               else '9.2023-03' if x.search_time_tmp <= '2023-03' and x.当前期数 <= 9
#                                               else '10.2023-04' if x.search_time_tmp <= '2023-04' and x.当前期数 <= 10
#                                               else '11.2023-05' if x.search_time_tmp <= '2023-05' and x.当前期数 <= 11
#                                               else '12.2023-06' if x.search_time_tmp <= '2023-06' and x.当前期数 <= 12
#                                               else '13.2023-07' if x.search_time_tmp <= '2023-07' and x.当前期数 <= 13
#                                               else '14.2023-08' if x.search_time_tmp <= '2023-08' and x.当前期数 <= 14
#                                               else '15.2023-09' if x.search_time_tmp <= '2023-09' and x.当前期数 <= 15
#                                               else '16.2023-10' if x.search_time_tmp <= '2023-10' and x.当前期数 <= 16
#                                               else '17.2023-11' if x.search_time_tmp <= '2023-11' and x.当前期数 <= 17
#                                               else '18.2023-12' if x.search_time_tmp <= '2023-12' and x.当前期数 <= 18
                                              
#                                               else '19.2024-01' if x.search_time_tmp <= '2024-01' and x.当前期数 <= 19
#                                               else '20.2024-02' if x.search_time_tmp <= '2024-02' and x.当前期数 <= 20
#                                               else '21.2024-03' if x.search_time_tmp <= '2024-03' and x.当前期数 <= 21
#                                               else '22.2024-04' if x.search_time_tmp <= '2024-04' and x.当前期数 <= 22
#                                               else '23.2024-05' if x.search_time_tmp <= '2024-05' and x.当前期数 <= 23
#                                               else '24.2024-06' if x.search_time_tmp <= '2024-06' and x.当前期数 <= 24
#                                               else '25.2024-07' if x.search_time_tmp <= '2024-07' and x.当前期数 <= 25
#                                               else '26.2024-08' if x.search_time_tmp <= '2024-08' and x.当前期数 <= 26
#                                               else '27.2024-09' if x.search_time_tmp <= '2024-09' and x.当前期数 <= 27
#                                               else '28.2024-10' if x.search_time_tmp <= '2024-10' and x.当前期数 <= 28
#                                               else '29.2024-11' if x.search_time_tmp <= '2024-11' and x.当前期数 <= 29
#                                               else '30.2024-12' if x.search_time_tmp <= '2024-12' and x.当前期数 <= 30

#                                               else '31.2025-01' if x.search_time_tmp <= '2025-01' and x.当前期数 <= 31
#                                               else 'other'

                                            #   , axis = 1)

conditions_1 = (~df_mob_temp.订单状态_new.isin(['已买断','已完成','已结束']))
conditions_2 = (df_mob_temp.剩余总应收租金 > 0)
# 剩余总应收租金 大于0 的订单的 剩余总应收租金、剩余总应收买断金，租归逾期 的未统计到
df_mob_temp[conditions_2].groupby(['view_month']).agg({'order_number':'count', '剩余总应收租金':'sum','剩余总应收买断金':'sum'}).reset_index()
# df_mob_temp.groupby(['view_month']).agg({'order_number':'count', '剩余总应收租金':'sum','剩余总应收买断金':'sum'}).reset_index()

df_mob_temp[conditions_1 & (df_mob_temp.overdue_days_mob > 1)].groupby(['view_month']).agg({'order_number':'count', '剩余总应收租金':'sum','剩余总应收买断金':'sum'}).reset_index()
df_mob_temp[conditions_1 & (df_mob_temp.overdue_days_mob > 30)].groupby(['view_month']).agg({'order_number':'count', '剩余总应收租金':'sum','剩余总应收买断金':'sum'}).reset_index()
df_mob_temp[conditions_1 & (df_mob_temp.overdue_days_mob > 90)].groupby(['view_month']).agg({'order_number':'count', '剩余总应收租金':'sum','剩余总应收买断金':'sum'}).reset_index()
df_mob_temp[conditions_1 & (df_mob_temp.overdue_days_mob > 180)].groupby(['view_month']).agg({'order_number':'count', '剩余总应收租金':'sum','剩余总应收买断金':'sum'}).reset_index()

# 有些订单一个月2条应还 A2023052913481677
# df_mob_temp[df_mob_temp.view_month == '2024-05'].shape
# df_mob_temp[df_mob_temp.view_month == '2024-05'].drop_duplicates(subset = ['order_number']).shape



# 另一个资方的需求
df_mob_temp[df_mob_temp.view_month == '2025-01'].groupby(['search_time']).agg({'剩余总应收租金':'sum','剩余总应收买断金':'sum'}).reset_index()

(250318, 24)

(17061, 22)

(267379, 25)

,view_month,order_number,剩余总应收租金,剩余总应收买断金
0,2022-07,21,104939.21,5.699734e+04
1,2022-08,77,340016.40,2.654471e+05
2,2022-09,121,482844.46,4.340999e+05
3,2022-10,160,590797.36,5.695617e+05
4,2022-11,203,738986.88,7.512837e+05
5,2022-12,268,1034287.32,1.024879e+06
6,2023-01,485,2074247.01,1.770966e+06
7,2023-02,775,3162112.72,2.970839e+06
8,2023-03,1085,4219426.44,4.317061e+06
9,2023-04,1682,6809436.16,7.148622e+06


,view_month,order_number,剩余总应收租金,剩余总应收买断金
0,2022-08,2,11592.92,5579.02
1,2022-09,13,74227.57,41843.91
2,2022-10,23,132753.76,86075.73
3,2022-11,32,164007.82,126939.81
4,2022-12,40,195465.96,146585.95
5,2023-01,52,246739.87,190642.66
6,2023-02,68,311747.03,244336.95
7,2023-03,121,537575.91,456212.44
8,2023-04,169,765297.42,679286.98
9,2023-05,228,1013941.11,932553.97


,view_month,order_number,剩余总应收租金,剩余总应收买断金
0,2022-09,2,11592.92,5579.02
1,2022-10,13,75925.90,42911.47
2,2022-11,19,113729.15,73119.08
3,2022-12,25,139831.95,101555.58
4,2023-01,35,179477.31,134539.19
5,2023-02,46,225052.85,162299.74
6,2023-03,63,295695.47,230262.84
7,2023-04,103,475016.88,384784.06
8,2023-05,139,658546.83,551482.12
9,2023-06,193,874197.84,784835.10


,view_month,order_number,剩余总应收租金,剩余总应收买断金
0,2022-11,2,11592.92,5579.02
1,2022-12,13,75925.90,42911.47
2,2023-01,19,113729.15,73119.08
3,2023-02,25,139423.55,101555.58
4,2023-03,32,170818.89,125330.23
5,2023-04,43,208175.93,155528.43
6,2023-05,55,264373.86,197482.00
7,2023-06,87,409365.10,321649.47
8,2023-07,129,621998.20,508379.22
9,2023-08,184,838084.70,751407.24


,view_month,order_number,剩余总应收租金,剩余总应收买断金
0,2023-02,2,11592.92,5579.02
1,2023-03,13,75925.90,42911.47
2,2023-04,19,113729.15,73119.08
3,2023-05,25,138984.46,101555.58
4,2023-06,30,165799.88,118249.58
5,2023-07,40,203743.02,144460.34
6,2023-08,54,263545.75,193190.14
7,2023-09,85,406094.72,313139.97
8,2023-10,126,609681.96,494637.10
9,2023-11,169,775888.53,682457.09


,view_month,order_number,剩余总应收租金,剩余总应收买断金
0,2022-07,22,104939.21,5.699734e+04
1,2022-08,77,340016.40,2.654471e+05
2,2022-09,121,482844.46,4.340999e+05
3,2022-10,162,590797.36,5.749507e+05
4,2022-11,204,738986.88,7.566727e+05
5,2022-12,270,1034287.32,1.030268e+06
6,2023-01,490,2074247.01,1.786633e+06
7,2023-02,783,3162112.72,3.004760e+06
8,2023-03,1093,4219426.44,4.351077e+06
9,2023-04,1708,6809436.16,7.244517e+06


In [102]:

# df_mob_temp[df_mob_temp.order_number == 'A2023090309334066'][['当前期数','view_month','search_time','当前期数']]
# df_mob_temp[df_mob_temp.view_month == '15.2023-09'].order_number.value_counts()
# df_mob_temp[df_mob_temp.order_number == 'A2023052913481677'].to_excel('tmp4.xlsx', index = False)
# 另一个资方的需求
df_mob_temp[df_mob_temp.view_month == '2025-01'].groupby(['search_time']).agg({'剩余总应收租金':'sum','剩余总应收买断金':'sum'}).reset_index()

,search_time,剩余总应收租金,剩余总应收买断金
0,2022-07,50011.04,18778.37
1,2022-08,93534.82,129081.49
2,2022-09,47154.00,77234.32
3,2022-10,31032.61,57180.79
4,2022-11,47261.36,58668.60
5,2022-12,78634.20,79114.70
6,2023-01,258870.10,248620.35
7,2023-02,301029.82,473430.85
8,2023-03,344588.04,489116.43
9,2023-04,691787.95,1053187.95


In [72]:
# （5）数据检查
# df_mob_total_tmp[df_mob_total_tmp.order_number.isin(['A2023112717153031'])][['剩余总应收租金','应付总租金','当前累计实付租金','当前应付租金','实付金额','应付日期','实付日期new','overdue_days_mob','订单状态_new','当前期数']]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024063001194047'])][var3]
# 订单已完成
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024031113200319'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202308262317036'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207050851373'])][var3]
# # 在15期之前买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024020317504938'])][var3] --
# 在15期之后买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207171820512'])][var3] -- 9683，在15期之外买断，不加工租期逾期
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207181530165'])][var3] --一直逾期，直至租期到了之后也逾期
# 已买断但 买断尾款	!= real_pay_money_new  可能使用优惠劵
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207151707153'])][var3] --

# order_number = ['A2023112717153031','A2024060200290566','A2024063001194047','A2024031113200319','A202308262317036','A202207050851373','A2024020317504938','A202207171820512','A202207181530165','A202207151707153']
# df_mob_total_tmp.to_excel('tmp.xlsx', index = False)
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024060200290566'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024063001194047'])][var3]
# 订单已完成
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024031113200319'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202308262317036'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207050851373'])][var3]
# 在15期之前买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024020317504938'])][var3]
# 在15期之后买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207171820512'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207181530165'])][var3]
# 已买断但 买断尾款	!= real_pay_money_new  可能使用优惠劵
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207151707153'])][var3]

# 7.续租订单

<!-- 注意 20241205 20241206 修改非续租的内容 -->
<!-- code未修正 -->

In [73]:
# （1）续租数据
# (39745, 162)
# df_relet = df_xz_concat[df_xz_concat.is_relet>=1]
# df_relet.shape

# # 2025.2.11
df_xz_concat_2 = df_xz_concat.merge(df_xz_concat_1[['order_number', '订单状态_new','当前期数']],on = ['order_number','当前期数'], how = 'inner')
# conditions_1 = (df_xz_concat_2.订单状态_new != '已结束') | ((df_xz_concat_2.订单状态_new == '已结束') & (df_xz_concat_2.实付金额 > 0))
# df_xz_concat_2 = df_xz_concat_2[conditions_1]
df_relet = df_xz_concat_2[df_xz_concat_2.is_relet >= 1]
df_relet['当前应付租金'] = np.where((df_relet.订单状态_new == '已结束') & (df_relet.实付金额 == 0), 0, df_relet.当前应付租金)

df_relet.shape
df_xz_concat.is_relet.value_counts()

(40936, 162)

is_relet
0.0    398010
1.0     40872
2.0        64
Name: count, dtype: int64

In [74]:
# （2）每个mob对应的月底时间，有些订单mob1不用还，故 n 放到 24
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    for n in range(13,25):  
        # note 20241209 修改 月末的的跳级了，所以不要加1，mob对齐
        # df[f'mob_date_{n}'] = np.where(df["下单日期"].dt.day.isin([28,29,30,31]), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = np.where((df["下单日期"].dt.day == 31)|(df["下单日期"].isin(['2023-02-28','2024-02-29','2025-02-28']))|((df["下单日期"].dt.month.isin([4,6,9,11]))&(df["下单日期"].dt.day == 30)), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        # df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1) 
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}']) 
    return df

df_relet = mob_date(df_relet)
df_relet.shape
df_relet[['下单日期','mob_date_15','mob_date_21','mob_date_20','mob_date_13','实付金额','paid_money_mob13','paid_date_mob15','paid_money_mob18']][:2]

(40936, 198)

,下单日期,mob_date_15,mob_date_21,mob_date_20,mob_date_13,实付金额,paid_money_mob13,paid_date_mob15,paid_money_mob18
12,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,354.0,354.0,2023-07-07,354.0
13,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,354.0,354.0,2023-08-07,354.0


In [75]:
# （3）每个mob对应的逾期天数  
# note 20241209 更新 应付日期，提前还款的统计在内
df_relet['应付日期_new'] = np.where((df_relet['实付日期new'].notna())&((df_relet['应付日期'] - df_relet['实付日期new']).dt.days >= 0),df_relet['实付日期new'],df_relet['应付日期'])
df_mob_list = []

for i in range(12,25): 
    # note 20241209 增加 =
    df_mob = df_relet[((df_relet[f'mob_date_{i}'] - df_relet['应付日期_new']).dt.days >= 0) 
                      & ((df_relet['date_str'] - df_relet['应付日期']).dt.days > 0) 
                    #   2025.2.11
                    #   & ((df_relet['date_str'] - df_relet['应付日期_new']).dt.days > 0) 
                      & ((df_relet['date_str'] - df_relet[f'mob_date_{i}']).dt.days > 0)
                      ]
   
    conditions = [ ( ( df_mob['逾期天数']==0) | (pd.notnull(df_mob[f'paid_date_mob{i}']))),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days > 0)  & (pd.isnull(df_mob[f'paid_date_mob{i}'])) ),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days <= 0) & ( pd.isnull(df_mob[f'paid_date_mob{i}'])))]  
  
    choices = [0,(df_mob['date_str'] - df_mob['应付日期']).dt.days,(df_mob[f'mob_date_{i}'] - df_mob['应付日期']).dt.days]  
  
    # 使用numpy.where来应用条件  
    df_mob['overdue_days_mob'] = np.select(conditions, choices, default = df_mob['逾期天数'])

    df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']
    # df_mob

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # note 20241210 修改 提前支付的剔除未到期应收
    df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    df_mob_group_1 = df_relet[((df_relet[f'mob_date_{i}'] - df_relet['应付日期']).dt.days >= 0) 
                      & ((df_relet['date_str'] - df_relet['应付日期']).dt.days > 0) 
                      & ((df_relet['date_str'] - df_relet[f'mob_date_{i}']).dt.days > 0)
                      ].groupby(by=['order_number']).agg({'当前应付租金':'sum','当前期数':'max'}).reset_index()
    df_mob_group = df_mob_group.merge(df_mob_group_1, on = 'order_number', how = 'left')


    df_mob_group['MOB'] = f'mob_{i}'

    df_mob_list.append(df_mob_group)

df_mob_relet = pd.concat(df_mob_list, ignore_index=True)

# 数据检查
# order_number order_id
# A202207050851373    8924     已买断，买断金额没有体现在实付金额中，买断后剩余期数的实付金额为0；2023-07-05、2023-10-3的还款记录在系统没有找到
# A2023073114301076   508404  只有mob13-16，mob12在非续租那里了，2023.8不用还款，隔月了，故非续租为13个月长度
# A2023070210395232   432132  只有mob12-16，其他的未到期
# A20231026115543149  726194  只有mob12、mob13，其他的未到期

# df_mob_relet[df_mob_relet.order_number == 'A202207050851373']
# 有mob12但无mob13
# A2023110100321259  738560  23
# 有mob13但无mob12
# A202208291922283    18163  26  2023.2-2023.5 的账单没有还，但2023.6-2023.8的账单还了
# A202207282047042    11630  25

# order_number_param = 'A202207282047042'
# var = ['mob_date_0','mob_date_11','mob_date_12','mob_date_13','mob_date_21','mob_date_23']
# var1 = ['mob_date_0','mob_date_1','mob_date_2','mob_date_3','mob_date_4','mob_date_11','mob_date_12']
# df_xz_concat[df_xz_concat.order_number == order_number_param][['order_number','下单日期','应付日期','实付日期new','实付金额] + var1]
# df_relet[df_relet.order_number == order_number_param][['order_number','order_id','下单日期','状态','应付日期','当前应付租金','实付日期new','实付金额','逾期天数'] + var]
# df_mob_relet[df_mob_relet.order_number == order_number_param][['order_number','下单日期','当前应付租金','paid_money_mob','overdue_days_mob','MOB']]

# 2025.2.9
df_mob_relet.shape
df_mob_relet = df_mob_relet[df_mob_relet.当前应付租金 >= 0]
df_mob_relet = df_mob_relet.drop_duplicates(subset = ['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','当前期数'])
df_mob_relet.shape
df_mob_relet = df_mob_relet.sort_values(by = ['order_number','当前期数','overdue_days_mob'])
df_mob_relet = df_mob_relet.drop_duplicates(subset = ['order_number','当前期数'])

df_mob_relet = df_mob_relet.rename(columns = {'当前应付租金': '当前累计应付租金', 'paid_money_mob': '当前累计实付租金'})
df_mob_relet.tail(10)

(18675, 7)

(17136, 7)

,order_number,下单日期,当前累计实付租金,overdue_days_mob,当前累计应付租金,当前期数,MOB
3344,A2024012710264619,2024-01-27,608.17,0,608.17,13.0,mob_12
3346,A2024012716204933,2024-01-27,0.00,1,394.94,13.0,mob_12
3348,A2024012721174916,2024-01-27,608.17,0,608.17,13.0,mob_12
3349,A202401281010128,2024-01-28,810.86,0,810.86,13.0,mob_12
3350,A2024012812064410,2024-01-28,670.76,0,670.76,13.0,mob_12
3351,A202401281305253,2024-01-28,658.81,0,658.81,13.0,mob_12
3352,A2024012814461158,2024-01-28,670.76,0,670.76,13.0,mob_12
3353,A2024012815251522,2024-01-28,0.00,0,670.76,13.0,mob_12
3354,A202401281702473,2024-01-28,354.28,0,354.28,13.0,mob_12
3360,A202401291601122,2024-01-29,669.63,0,669.63,13.0,mob_12


In [76]:
# （4）关联27期的表
# 2025.2.11
var_1 = ['order_number','下单日期','租赁方案','当前期数','应付日期', '当前应付租金', '实付金额','实付日期new','首期应还日期','买断价','应付总租金','买断尾款','订单状态_new','当月已收买断金','当前累计已收买断金','剩余总应收买断金']
df_mob_relet_tmp = df_xz_concat_1[df_xz_concat_1.is_relet >= 1][var_1].merge(df_mob_relet[['order_number','当前期数','当前累计应付租金','当前累计实付租金','overdue_days_mob']], on = ['order_number','当前期数'], how = 'left')
df_xz_concat_1[df_xz_concat_1.is_relet >= 1].shape
df_mob_relet_tmp.shape

# tmp_2 = df_xz_concat.groupby(['order_number']).agg({'is_relet':'max'}).reset_index()
# tmp_2 = tmp_2.rename(columns = {'is_relet': 'is_relet_max'})
# tmp_2.shape

# df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp_2, on = 'order_number', how ='inner')
# df_mob_relet_tmp.shape
# del tmp_2

# 加工 df_mob_relet_tmp
# 续租的订单只有12期，第12期的订单状态修改为已完成
# df_mob_relet_tmp['订单状态_new'] = np.where((df_mob_relet_tmp.is_relet_max >= 1) & (df_mob_relet_tmp.当前期数 == 12), '已完成', df_mob_relet_tmp['订单状态_new'])
# 删去订单已结束的订单
df_mob_relet_tmp = df_mob_relet_tmp[df_mob_relet_tmp.订单状态_new != '已结束']
df_mob_relet_tmp.shape
# 订单未结束的，删去未到期的，最新时间为 2025.2.10
df_mob_relet_tmp = df_mob_relet_tmp[df_mob_relet_tmp.应付日期 <= '2025-01-31']
df_mob_relet_tmp.shape
# 订单未结束的，到期未结束的则取完15期的数据，若租期一直逾期，则衍生到15期，若租期逾期，则默认不逾期
# A2024020317504938 2025.2 买断，还没表现出来
# 'A202207171820512' -- 9683，在15期之外买断，不加工租期逾期
# 'A202207181530165' --一直逾期，直至租期到了之后也逾期

df_mob_relet_tmp['overdue_days_mob'] = df_mob_relet_tmp['overdue_days_mob'].fillna(0)
df_mob_relet_tmp['当前累计应付租金'] = df_mob_relet_tmp['当前累计应付租金'].fillna(0)
df_mob_relet_tmp['当前累计实付租金'] = df_mob_relet_tmp['当前累计实付租金'].fillna(0)

tmp_2 = df_mob_relet.groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp_2 = tmp_2.rename(columns = {'当前期数': '当前期数'})
tmp_3 = df_mob_relet[['order_number','当前期数','当前累计实付租金','当前累计应付租金','overdue_days_mob']].merge(tmp_2, on = ['order_number','当前期数'], how = 'inner')
tmp_2.shape
tmp_3.shape
tmp_3['当前期数'] = tmp_3['当前期数'] + 1
df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] > 0, df_mob_relet_tmp['overdue_days_mob_y'] + 30, df_mob_relet_tmp['overdue_days_mob_x'])
df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] >= 0, df_mob_relet_tmp['overdue_days_mob_y'] + 30, df_mob_relet_tmp['overdue_days_mob_x'])
df_mob_relet_tmp['当前累计应付租金'] = np.where(df_mob_relet_tmp['当前累计应付租金_y'] > df_mob_relet_tmp['当前累计应付租金_x'], df_mob_relet_tmp['当前累计应付租金_y'], df_mob_relet_tmp['当前累计应付租金_x'])
df_mob_relet_tmp['当前累计实付租金'] = np.where(df_mob_relet_tmp['当前累计实付租金_y'] > df_mob_relet_tmp['当前累计实付租金_x'], df_mob_relet_tmp['当前累计实付租金_y'], df_mob_relet_tmp['当前累计实付租金_x'])
df_mob_relet_tmp = df_mob_relet_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

tmp_3['当前期数'] = tmp_3['当前期数'] + 1
df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] > 0, df_mob_relet_tmp['overdue_days_mob_y'] + 60, df_mob_relet_tmp['overdue_days_mob_x'])
df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] >= 0, df_mob_relet_tmp['overdue_days_mob_y'] + 60, df_mob_relet_tmp['overdue_days_mob_x'])
df_mob_relet_tmp['当前累计应付租金'] = np.where(df_mob_relet_tmp['当前累计应付租金_y'] > df_mob_relet_tmp['当前累计应付租金_x'], df_mob_relet_tmp['当前累计应付租金_y'], df_mob_relet_tmp['当前累计应付租金_x'])
df_mob_relet_tmp['当前累计实付租金'] = np.where(df_mob_relet_tmp['当前累计实付租金_y'] > df_mob_relet_tmp['当前累计实付租金_x'], df_mob_relet_tmp['当前累计实付租金_y'], df_mob_relet_tmp['当前累计实付租金_x'])
df_mob_relet_tmp = df_mob_relet_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

tmp_3['当前期数'] = tmp_3['当前期数'] + 1
df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] > 0, df_mob_relet_tmp['overdue_days_mob_y'] + 90, df_mob_relet_tmp['overdue_days_mob_x'])
df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] >= 0, df_mob_relet_tmp['overdue_days_mob_y'] + 90, df_mob_relet_tmp['overdue_days_mob_x'])
df_mob_relet_tmp['当前累计应付租金'] = np.where(df_mob_relet_tmp['当前累计应付租金_y'] > df_mob_relet_tmp['当前累计应付租金_x'], df_mob_relet_tmp['当前累计应付租金_y'], df_mob_relet_tmp['当前累计应付租金_x'])
df_mob_relet_tmp['当前累计实付租金'] = np.where(df_mob_relet_tmp['当前累计实付租金_y'] > df_mob_relet_tmp['当前累计实付租金_x'], df_mob_relet_tmp['当前累计实付租金_y'], df_mob_relet_tmp['当前累计实付租金_x'])
df_mob_relet_tmp = df_mob_relet_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

df_mob_relet_tmp.shape
df_mob_relet_tmp['search_time'] = df_mob_relet_tmp['下单日期'].dt.strftime('%Y-%m')

tmp = df_xz_concat[df_xz_concat.is_relet >= 1].groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp1 = df_xz_concat[df_xz_concat.is_relet == 0].groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp.shape
tmp1.shape
tmp = tmp.merge(tmp1, on = 'order_number', how = 'left')
tmp.shape
tmp['续租总期数'] = tmp['当前期数_x'] - tmp['当前期数_y']
tmp = tmp[['order_number','续租总期数']]

# tmp = tmp.rename(columns = {'当前期数': '最大期数'})
df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp, on = 'order_number', how = 'left')
df_mob_relet_tmp.shape

# df_mob_relet_tmp['买断尾款'] = np.where(df_mob_relet_tmp.租赁方案 == '租完即送', 0, df_mob_relet_tmp.买断尾款)
# df_mob_relet_tmp['剩余总应收买断金'] = np.where(df_mob_relet_tmp.租赁方案 == '租完即送', 0, df_mob_relet_tmp.剩余总应收买断金)

df_mob_relet_tmp['应付总租金'] = df_mob_relet_tmp['应付总租金'].fillna(0)
df_mob_relet_tmp['当前累计实付租金'] = df_mob_relet_tmp['当前累计实付租金'].fillna(0)
df_mob_relet_tmp['剩余总应收租金'] = df_mob_relet_tmp['应付总租金'] - df_mob_relet_tmp['当前累计实付租金']
df_mob_relet_tmp['剩余总应收租金'] = np.where(df_mob_relet_tmp['剩余总应收租金'] < 0, 0, df_mob_relet_tmp['剩余总应收租金'])
df_mob_relet_tmp['剩余总应收租金'] = np.where(df_mob_relet_tmp['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_mob_relet_tmp['剩余总应收租金'])

# df_mob_relet_tmp['剩余总应收'] = df_mob_relet_tmp['剩余总应收租金'] + df_mob_relet_tmp['剩余总应收买断金']

# A2023112717153031
df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp.订单状态_new.isin(['已买断','已完成','已结束']), 0 , df_mob_relet_tmp.overdue_days_mob)
# df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp.当前期数 == 13, 0 , df_mob_relet_tmp.overdue_days_mob)

df_mob_relet_tmp[:1]

(52600, 172)

(52600, 19)

(49905, 19)

(16716, 19)

(3584, 2)

...

(3888, 2)

(33120, 2)

(3888, 3)

(16716, 21)

,order_number,下单日期,租赁方案,当前期数,应付日期,当前应付租金,实付金额,实付日期new,首期应还日期,买断价,...,订单状态_new,当月已收买断金,当前累计已收买断金,剩余总应收买断金,overdue_days_mob,当前累计应付租金,当前累计实付租金,search_time,续租总期数,剩余总应收租金
0,A202207050851373,2022-07-05,租完归还,13.0,2023-07-05,354.0,354.0,2023-07-07,2023-07-05,8176.92,...,还款中,0.0,0.0,0.0,0.0,354.0,354.0,2022-07,12.0,3599.0


In [314]:
# （5）统计
# df_mob_relet_tmp[(df_mob_relet_tmp.续租总期数 == 12)].当前期数.min()
# df_mob_relet_tmp.买断尾款.max()
# df_mob_relet_tmp.剩余总应收买断金.max()

conditions_1 = ((df_mob_relet_tmp.续租总期数 == 12) & (df_mob_relet_tmp.当前期数 == 13))
conditions_4 = ((df_mob_relet_tmp.续租总期数 == 12) & (~df_mob_relet_tmp.订单状态_new.isin(['已买断','已完成','已结束'])))

conditions_9 = ( (df_mob_relet_tmp.续租总期数 == 12) & (df_mob_relet_tmp.overdue_days_mob > 1 ))
conditions_10 = ((df_mob_relet_tmp.续租总期数 == 12) & (df_mob_relet_tmp.overdue_days_mob > 30 ))
conditions_11 = ((df_mob_relet_tmp.续租总期数 == 12) & (df_mob_relet_tmp.overdue_days_mob > 60 ))
conditions_12 = ((df_mob_relet_tmp.续租总期数 == 12) & (df_mob_relet_tmp.overdue_days_mob > 90 ))


# '应付总租金'
df_mob_relet_tmp[conditions_1].groupby(['search_time']).agg({'order_number':'count', '应付总租金':'sum'}).reset_index()

# df_mob_relet_tmp[conditions_2].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收租金':'sum'}).reset_index()

# df_mob_relet_tmp[conditions_3].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收租金': 'sum'}).reset_index()
# df_mob_relet_tmp[conditions_4].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收租金': 'sum'}).reset_index()
# df_mob_relet_tmp[conditions_5].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收租金': 'sum'}).reset_index()
# df_mob_relet_tmp[conditions_6].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收租金': 'sum'}).reset_index()


tmp1 = df_mob_relet_tmp[conditions_4].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收租金':'sum','剩余总应收买断金':'sum'}).reset_index()
tmp1 = tmp1.rename(columns = {'order_number': '表现月末未结清客户数','剩余总应收租金': '截至表现月份剩余总应收（租金）','剩余总应收买断金':'截至表现月份剩余总应收（买断金）'})

tmp2 = df_mob_relet_tmp[conditions_9].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收租金': 'sum'}).reset_index()
tmp2 = tmp2.rename(columns = {'order_number': '表现月末逾期客户数dpd1+','剩余总应收租金': '表现月末逾期dpd1+剩余总应收（应收租金+买断金）'})

tmp3 = df_mob_relet_tmp[conditions_10].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收租金': 'sum'}).reset_index()
tmp3 = tmp3.rename(columns = {'order_number': '表现月末逾期客户数dpd30+','剩余总应收租金': '表现月末逾期dpd30+剩余总应收（应收租金+买断金）'})

tmp4 = df_mob_relet_tmp[conditions_11].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收租金': 'sum'}).reset_index()
tmp4 = tmp4.rename(columns = {'order_number': '表现月末逾期客户数dpd60+','剩余总应收租金': '表现月末逾期dpd60+剩余总应收（应收租金+买断金）'})

tmp5 = df_mob_relet_tmp[conditions_12].groupby(['search_time','当前期数']).agg({'order_number':'count', '剩余总应收租金': 'sum'}).reset_index()
tmp5 = tmp5.rename(columns = {'order_number': '表现月末逾期客户数dpd90+','剩余总应收租金': '表现月末逾期dpd90+剩余总应收（应收租金+买断金）'})

tmp = tmp1.merge(tmp2, on = ['search_time','当前期数'], how = 'outer')
tmp = tmp.merge(tmp3, on = ['search_time','当前期数'], how = 'outer')
tmp = tmp.merge(tmp4, on = ['search_time','当前期数'], how = 'outer')
tmp = tmp.merge(tmp5, on = ['search_time','当前期数'], how = 'outer')
var = ['表现月末未结清客户数','截至表现月份剩余总应收（租金）','截至表现月份剩余总应收（买断金）',	'表现月末逾期客户数dpd1+','表现月末逾期客户数dpd30+','表现月末逾期客户数dpd60+',	
'表现月末逾期客户数dpd90+',	'表现月末逾期dpd1+剩余总应收（应收租金+买断金）','表现月末逾期dpd30+剩余总应收（应收租金+买断金）','表现月末逾期dpd60+剩余总应收（应收租金+买断金）','表现月末逾期dpd90+剩余总应收（应收租金+买断金）']
tmp = tmp[['search_time','当前期数'] + var]
tmp = tmp.sort_values(by = ['search_time','当前期数'])
tmp.to_excel('tmp2.xlsx', index = False)

# 2023-11	15.0	35	129453.64

,search_time,order_number,应付总租金
0,2022-07,1,3953.00
1,2022-08,1,4420.80
2,2022-09,3,13415.70
3,2022-10,2,9599.86
4,2023-01,1,5544.00
5,2023-02,1,5759.61
6,2023-04,25,174510.07
7,2023-05,138,914493.31
8,2023-06,219,1512620.42
9,2023-07,322,2332133.56


In [99]:
# （6）联合运营数据
# A202207181530165
var = ['order_number','下单日期','租赁方案','当前期数','应付日期',
       '应付总租金','买断尾款','当前应付租金','剩余总应收租金','overdue_days_mob','实付金额']
# df_mob_relet_tmp[df_mob_relet_tmp.order_number == 'A2022072220482114'][var]
# df_xz_concat[df_xz_concat.order_number == 'A2022072220482114'][['order_number','is_relet','逾期天数']]
# 13 ，续租的开始期数是13，即首租 12期的才有续租
# df_xz_concat[df_xz_concat.is_relet >= 1].当前期数.min()

temp1 = df_mob_relet_tmp[df_mob_relet_tmp.当前期数 <= 15][var].merge(df_xz_concat[['order_number','当前期数','逾期天数']], on = ['order_number','当前期数'], how = 'inner')
temp1 = temp1.rename(columns = {'逾期天数':'每期最大逾期天数','剩余总应收租金':'剩余总应收'})
# 9473
temp1.shape
# temp1.买断尾款.max()

var = ['order_number','下单日期','租赁方案','当前期数','应付日期',
       '应付总租金','买断尾款','当前应付租金','overdue_days_mob','剩余总应收','每期最大逾期天数','实付金额']
df_mob_total_tmp_3[var].shape

# 完整数据，15期，不区分是否续租
df_data_15 = pd.concat([df_mob_total_tmp_3[var], temp1])
df_data_15.shape

temp2 = df_xz_concat.drop_duplicates(subset = ['order_number'])[['order_number','归属渠道']]
df_data_15 = df_data_15.merge(temp2, on = 'order_number', how = 'inner')
df_data_15.shape

df_data_15 = df_data_15[(df_data_15.归属渠道 == '芝麻租物') & (df_data_15.下单日期 >= '2023-11-01') & (df_data_15.下单日期 <= '2024-10-31')]
df_data_15.shape
# df_data_15.drop_duplicates(subset = ['order_number','当前期数']).shape

# 取 外部订单号
# （6-1）取 trace_id
sql1='''-- trace关联表 
SELECT trace_id, order_number
from db_credit.order_association
;
'''

trace_df =query(sql1)
trace_df.shape

# （6-2）关联 out_order_no
sql1='''-- 芝麻租物风控专业版
-- 95184	95184	95184
SELECT trace_id,out_order_no
from db_credit.risk_alipay_auth_query
;
'''
zm_fk_df = query(sql1)
zm_fk_df.shape

# （6-3）表关联
trace_df = trace_df.merge(zm_fk_df, on = 'trace_id', how = 'left')
trace_df.shape
trace_df = trace_df[trace_df.out_order_no.notnull()]
trace_df = trace_df[['order_number','out_order_no']]
trace_df = trace_df.drop_duplicates(subset = 'order_number')
trace_df.shape

df_data_15 = df_data_15.merge(trace_df, on = 'order_number', how = 'inner')
df_data_15.shape

temp3 = df_xz_concat[df_xz_concat.is_relet == 0].groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
temp3 = temp3.rename(columns = {'当前期数': '租期'})
# df_data_15.租期.min()
df_data_15 = df_data_15.merge(temp3, on = 'order_number', how = 'inner')
df_data_15['应付总租金'] = df_data_15['应付总租金'].fillna(0)
df_data_15['买断尾款'] = df_data_15['买断尾款'].fillna(0)
df_data_15['商品价值'] = df_data_15['应付总租金'] + df_data_15['买断尾款']
df_data_15['每期最大逾期天数'] = np.where(df_data_15['每期最大逾期天数'] < 0, 0, df_data_15['每期最大逾期天数'])
df_data_15.shape


df_data_15[df_data_15.order_number == 'A20231128114426146']

df_data_15 = df_data_15.rename(columns = {'out_order_no': '外部订单号', '当前应付租金': '每期租金', '应付日期': '第X期应扣款时间', '每期最大逾期天数':'第X期历史最大逾期天数', '实付金额': '实付租金'})
# var = ['外部订单号','租期','当前期数','每期租金','第X期应扣款时间','第X期历史最大逾期天数','实付租金','剩余总应收','商品价值']

# path = 'C:/Users/Administrator/Desktop/mine/4_专题分析/5_联合运营/1_样本发送/'
# df_data_15[var].to_excel(path + 'lhyy_sample_data_发送_20250224.xlsx', index = False)


# 随机抽取
df_data_15.drop_duplicates(subset = 'order_number').shape
# 11357*0.9 = 10221
tmp1 = df_data_15.drop_duplicates(subset = 'order_number')[['order_number']]
df_data_sample = tmp1.sample(n = 10221)
df_data_sample.shape


df_data_sample = df_data_sample.merge(df_data_15, on = 'order_number', how = 'inner')
df_data_sample.shape
var = ['外部订单号','租期','当前期数','每期租金','第X期应扣款时间','第X期历史最大逾期天数','实付租金','剩余总应收','商品价值']
path = 'C:/Users/Administrator/Desktop/mine/4_专题分析/5_联合运营/1_样本发送/'
df_data_sample[var].to_excel(path + 'lhyy_sample_data_发送_20250225.xlsx', index = False)

del trace_df
del zm_fk_df

# 关联不到 外部订单号
# 93751 - 93714

(9473, 12)

(239651, 12)

(249124, 12)

(249124, 13)

(95564, 13)

...

(1701189, 3)

(308034, 2)

(93714, 14)

(93714, 16)

,order_number,下单日期,租赁方案,当前期数,应付日期,应付总租金,买断尾款,当前应付租金,overdue_days_mob,剩余总应收,每期最大逾期天数,实付金额,归属渠道,out_order_no,租期,商品价值
13250,A20231128114426146,2023-11-28,租完归还,1.0,2023-12-01,6497.29,6499.41,541.32,0.0,10829.97,0,541.32,芝麻租物,bb8d60c954cb4efd96942733207f45b8,12.0,12996.70
13251,A20231128114426146,2023-11-28,租完归还,2.0,2024-01-01,6497.29,6499.41,541.32,0.0,10288.65,0,541.32,芝麻租物,bb8d60c954cb4efd96942733207f45b8,12.0,12996.70
13252,A20231128114426146,2023-11-28,租完归还,3.0,2024-02-01,6497.29,6499.41,541.32,0.0,9747.33,0,541.32,芝麻租物,bb8d60c954cb4efd96942733207f45b8,12.0,12996.70
13253,A20231128114426146,2023-11-28,租完归还,4.0,2024-03-01,6497.29,6499.41,541.32,0.0,9206.01,0,541.32,芝麻租物,bb8d60c954cb4efd96942733207f45b8,12.0,12996.70
13254,A20231128114426146,2023-11-28,租完归还,5.0,2024-04-01,6497.29,6499.41,541.32,0.0,8664.69,0,541.32,芝麻租物,bb8d60c954cb4efd96942733207f45b8,12.0,12996.70
13255,A20231128114426146,2023-11-28,租完归还,6.0,2024-05-01,6497.29,6499.41,541.32,0.0,8123.37,0,541.32,芝麻租物,bb8d60c954cb4efd96942733207f45b8,12.0,12996.70
13256,A20231128114426146,2023-11-28,租完归还,7.0,2024-06-01,6497.29,6499.41,541.32,0.0,7582.05,0,541.32,芝麻租物,bb8d60c954cb4efd96942733207f45b8,12.0,12996.70
13257,A20231128114426146,2023-11-28,租完归还,8.0,2024-07-01,6497.29,6499.41,541.32,0.0,7040.73,0,541.32,芝麻租物,bb8d60c954cb4efd96942733207f45b8,12.0,12996.70
13258,A20231128114426146,2023-11-28,租完归还,9.0,2024-08-01,6497.29,6499.41,541.32,0.0,6499.41,0,541.32,芝麻租物,bb8d60c954cb4efd96942733207f45b8,12.0,12996.70
13259,A20231128114426146,2023-11-28,租完归还,10.0,2024-09-01,6497.29,6499.41,541.32,0.0,6499.41,0,541.32,芝麻租物,bb8d60c954cb4efd96942733207f45b8,12.0,12996.70


(11357, 16)

10221.300000000001

(10221, 1)

(84408, 16)

In [119]:
df_data_sample.drop_duplicates(subset = 'order_number').shape
df_data_sample.drop_duplicates(subset = '外部订单号').shape
# df_data_sample.外部订单号.value_counts()
tmp1 = df_data_sample.drop_duplicates(subset = ['外部订单号','order_number'])[['外部订单号','order_number']]
tmp2 = tmp1.groupby(['外部订单号']).agg({'order_number':'count'}).reset_index()
tmp2 = tmp2.sort_values(by = 'order_number', ascending = False)
tmp2[:16]
# tmp1.外部订单号.value_counts()
# df_data_15[df_data_15.外部订单号 == '83c6f45526f74cf293ca1dc90d12cc5b']
# 10221-10206

(10221, 16)

(10206, 16)

,外部订单号,order_number
6967,A20240925105927333,2
4138,A20240728175944140,2
6799,A2024092213135130,2
7713,A20241027155607258,2
5253,A2024082418240696,2
3227,83c6f45526f74cf293ca1dc90d12cc5b,2
6164,A2024091114145591,2
7369,A2024101117063831,2
6689,A2024092005290550,2
7166,A20241002114211148,2


In [100]:
# df_data_15[df_data_15.每期最大逾期天数>30]
# A20241031103751165
# A2025010615080173
# df_data_15[df_data_15.order_number == 'A2025010615080173']
# 实付金额 当月已收买断金
# df_mob_total_tmp[(df_mob_total_tmp.当前应付租金 != df_mob_total_tmp.实付金额) & (df_mob_total_tmp.实付金额 > 1)]
# df_mob_total_tmp[df_mob_total_tmp.order_number == 'A20241228144711236'][['当前期数','当前应付租金','实付金额']]

# df_mob_total_tmp_3[(df_mob_total_tmp_3.每期最大逾期天数 > 0) & (df_mob_total_tmp_3.实付金额 > 0) & (df_mob_total_tmp_3.当前应付租金 != df_mob_total_tmp_3.实付金额)]
# df_mob_total_tmp_3[df_mob_total_tmp_3.order_number == 'A2023032622384654'][['当前期数','当前应付租金','实付金额','每期最大逾期天数','应付日期']]
# 
# df_data_15[df_data_15.外部订单号 == '4a874e7712b4457794ba062af3cb5467']

,order_number,下单日期,租赁方案,当前期数,第X期应扣款时间,应付总租金,买断尾款,每期租金,overdue_days_mob,剩余总应收,第X期历史最大逾期天数,实付租金,归属渠道,外部订单号,租期,商品价值
22997,A2024010211365571,2024-01-02,租完归还,1.0,2024-01-05,7597.24,5399.46,632.96,0.0,11096.10,0,632.96,芝麻租物,4a874e7712b4457794ba062af3cb5467,12.0,12996.7
22998,A2024010211365571,2024-01-02,租完归还,2.0,2024-02-05,7597.24,5399.46,632.96,0.0,11096.10,0,632.96,芝麻租物,4a874e7712b4457794ba062af3cb5467,12.0,12996.7
22999,A2024010211365571,2024-01-02,租完归还,3.0,2024-03-05,7597.24,5399.46,632.96,0.0,10463.14,0,632.96,芝麻租物,4a874e7712b4457794ba062af3cb5467,12.0,12996.7
23000,A2024010211365571,2024-01-02,租完归还,4.0,2024-04-05,7597.24,5399.46,632.96,0.0,9830.18,0,632.96,芝麻租物,4a874e7712b4457794ba062af3cb5467,12.0,12996.7
23001,A2024010211365571,2024-01-02,租完归还,5.0,2024-05-05,7597.24,5399.46,632.96,0.0,9197.22,0,632.96,芝麻租物,4a874e7712b4457794ba062af3cb5467,12.0,12996.7
23002,A2024010211365571,2024-01-02,租完归还,6.0,2024-06-05,7597.24,5399.46,632.96,0.0,8564.26,0,632.96,芝麻租物,4a874e7712b4457794ba062af3cb5467,12.0,12996.7
23003,A2024010211365571,2024-01-02,租完归还,7.0,2024-07-05,7597.24,5399.46,632.96,0.0,7298.34,0,632.96,芝麻租物,4a874e7712b4457794ba062af3cb5467,12.0,12996.7
23004,A2024010211365571,2024-01-02,租完归还,8.0,2024-08-05,7597.24,5399.46,632.96,0.0,7298.34,0,632.96,芝麻租物,4a874e7712b4457794ba062af3cb5467,12.0,12996.7
23005,A2024010211365571,2024-01-02,租完归还,9.0,2024-09-05,7597.24,5399.46,632.96,0.0,6665.38,0,632.96,芝麻租物,4a874e7712b4457794ba062af3cb5467,12.0,12996.7
23006,A2024010211365571,2024-01-02,租完归还,10.0,2024-10-05,7597.24,5399.46,632.96,0.0,5399.46,0,632.96,芝麻租物,4a874e7712b4457794ba062af3cb5467,12.0,12996.7


In [102]:
df_data_15[df_data_15.order_number == 'A2024031300252114']

,order_number,下单日期,租赁方案,当前期数,第X期应扣款时间,应付总租金,买断尾款,每期租金,overdue_days_mob,剩余总应收,第X期历史最大逾期天数,实付租金,归属渠道,外部订单号,租期,商品价值
33560,A2024031300252114,2024-03-13,租完归还,1.0,2024-03-16,1038.9,649.5,86.56,0.0,1515.10,0,86.56,芝麻租物,c5ab6802af194c9c8bcab01332fa2df6,12.0,1688.4
33561,A2024031300252114,2024-03-13,租完归还,2.0,2024-04-16,1038.9,649.5,86.56,0.0,1428.54,0,86.56,芝麻租物,c5ab6802af194c9c8bcab01332fa2df6,12.0,1688.4
33562,A2024031300252114,2024-03-13,租完归还,3.0,2024-05-16,1038.9,649.5,86.56,0.0,1341.98,3,86.56,芝麻租物,c5ab6802af194c9c8bcab01332fa2df6,12.0,1688.4
33563,A2024031300252114,2024-03-13,租完归还,4.0,2024-06-16,1038.9,649.5,86.56,0.0,1255.42,13,86.56,芝麻租物,c5ab6802af194c9c8bcab01332fa2df6,12.0,1688.4
33564,A2024031300252114,2024-03-13,租完归还,5.0,2024-07-16,1038.9,649.5,86.56,0.0,1168.86,15,86.56,芝麻租物,c5ab6802af194c9c8bcab01332fa2df6,12.0,1688.4
33565,A2024031300252114,2024-03-13,租完归还,6.0,2024-08-16,1038.9,649.5,86.56,15.0,1168.86,30,86.56,芝麻租物,c5ab6802af194c9c8bcab01332fa2df6,12.0,1688.4
33566,A2024031300252114,2024-03-13,租完归还,7.0,2024-09-16,1038.9,649.5,86.56,0.0,995.74,0,86.56,芝麻租物,c5ab6802af194c9c8bcab01332fa2df6,12.0,1688.4
33567,A2024031300252114,2024-03-13,租完归还,8.0,2024-10-16,1038.9,649.5,86.56,15.0,995.74,21,86.56,芝麻租物,c5ab6802af194c9c8bcab01332fa2df6,12.0,1688.4
33568,A2024031300252114,2024-03-13,租完归还,9.0,2024-11-16,1038.9,649.5,86.56,14.0,909.18,57,86.56,芝麻租物,c5ab6802af194c9c8bcab01332fa2df6,12.0,1688.4
33569,A2024031300252114,2024-03-13,租完归还,10.0,2024-12-16,1038.9,649.5,86.56,45.0,909.18,27,86.56,芝麻租物,c5ab6802af194c9c8bcab01332fa2df6,12.0,1688.4


In [111]:
# order_number_need = ['A2023112717153031','A2024060200290566','A2024063001194047','A2024031113200319','A202308262317036','A202207050851373','A2024020317504938','A202207171820512','A202207181530165','A202207151707153']
# df_mob_total_tmp[df_mob_total_tmp.order_number.isin(order_number_need)].to_excel('tmp.xlsx', index = False)
# order_number_need = ['A2023112717153031','A2024060200290566','A2024063001194047','A2024031113200319','A202308262317036','A202207050851373','A2024020317504938','A202207171820512','A202207181530165','A202207151707153']
# df_mob_relet_tmp[df_mob_relet_tmp.order_number.isin(order_number_need)].to_excel('tmp1.xlsx', index = False)